In [ ]:

##gui에서 '신고가 다시 찾기' 눌러서 한번 조회한 단지는 또 눌러도 다시 조회하지 않고 새롭게 등록된 단지만 6년치 신고가 데이터를 찾게 수정해 줘.


import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd
import numpy as np
import os
os.environ['PYTHONHTTPSVERIFY'] = '0'
import json
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import logging
import time
import concurrent.futures
import threading
import schedule
import tkinter.font as tkFont
from plyer import notification  # 윈도우 알림
from PIL import Image, ImageTk, ImageGrab  # 캡쳐
import io
import random
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from tkinter import simpledialog



import webbrowser
import html  # html.escape 사용
# (선택) 월 이동 정확도 개선 시 사용
# from dateutil.relativedelta import relativedelta

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# --------------------------- 공통 네트워킹 유틸 ---------------------------
def build_session():
    """SSL 문제 해결된 세션"""
    import ssl
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    
    s = requests.Session()
    retry = Retry(
        total=5,
        connect=3,
        read=3,
        backoff_factor=0.6,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=frozenset(["GET"]),
        raise_on_status=False
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=20, pool_maxsize=40)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    s.headers.update({"User-Agent": "RealEstateMonitor/1.0"})
    s.verify = False  # SSL 검증 비활성화
    return s

# (connect, read) 타임아웃
API_TIMEOUT = (5, 15)

def jitter_sleep(max_ms=300):
    """0~max_ms ms 사이 임의 지연 (429 완화용)"""
    time.sleep(random.uniform(0, max_ms / 1000.0))

# ------------------------------------------------------------------------

class RealEstateMonitorApp:
    def __init__(self):
        self.palette = {
            'bg':            '#1E1E2F',
            'surface':       '#2A2A3D',
            'text_primary':  '#E0E0E8',
            'text_secondary':'#A0A0A8',
            'accent':        '#3AA6FF',
        }
        self.root = tk.Tk()
        self.root.title("부태리의 실거래가 모니터링")
        self.root.geometry("1500x800")  # 700 -> 850으로 변경
        
        # 공통 HTTP 세션
        self.http = build_session()
        
        # ★★★ API 캐싱 시스템 추가 ★★★
        self.api_cache = {}  # 캐시 저장소
        self.cache_ttl = timedelta(hours=24)  # 캐시 유효시간 24시간
        self.cache_hit_count = 0  # 캐시 히트 통계
        self.api_call_count = 0  # API 호출 통계

        # ▼▼ 추가: '선택시 해당 지역 전체 단지 모니터링' 체크 상태
        self.bulk_monitor_enabled = tk.BooleanVar(value=False)        
        
        # 기본 설정
        self.download_path = "C:\\Download"
        self.lawdong_path = "C:/law-dong/law-dong.txt"
        self.monitored_apts_file = os.path.join(self.download_path, "monitored_apts.json")

        # 백업 관련 설정
        self.backup_dir = os.path.join(self.download_path, "backups")
        self.auto_backup_interval = 24  # 24시간마다 자동 백업
        self.last_auto_backup = None
        os.makedirs(self.backup_dir, exist_ok=True)
        
        # 알림 설정
        self.auto_update_enabled = tk.BooleanVar(value=False)
        self.update_time = tk.StringVar(value="09:10")
        self.setup_fonts()         
        # 설정 불러오기
        self.load_settings()

        # ---- 여러 리스트 관리로 전환 ----
        self.monitored_lists = self.load_monitored_apts()  # {"lists": {...}, "active_list": "기본"} 형태로 로드/마이그레이션
        if "lists" not in self.monitored_lists:
            self.monitored_lists = {"lists": {"기본": []}, "active_list": "기본"}
        
        self.active_list = tk.StringVar(value=self.monitored_lists.get("active_list", "기본"))
        if self.active_list.get() not in self.monitored_lists["lists"]:
            self.monitored_lists["lists"][self.active_list.get()] = []        
        
        # 폴더 생성
        if not os.path.exists(self.download_path):
            os.makedirs(self.download_path)

    
        # 신고가 히스토리 파일 경로 추가
        self.notifications_history_file = os.path.join(self.download_path, "notifications_history.json")
        
        # 신고가 히스토리 로드
        self.notifications_history = self.load_notifications_history()
        
        # 폰트 설정
        self.setup_fonts()
        
        # 법정동 코드 관련 변수 초기화
        self.region_codes = {}
        self.sido_list = []
        self.sigungu_dict = {}
        self.dong_dict = {}
        
        # 법정동 파일 로드
        self.load_lawdong_file()
    
        self.sort_column = "last_max_price"  # 기본 정렬 열: 현재 최고가
        self.sort_reverse = True  # 기본 정렬 방향: 내림차순 (높은 값부터)
        
        # API 키 설정 (환경변수로 빼는 것을 권장)
        self.service_key = "Vs5lXsSo6iEI8no3pP%2FT0udWF9s7Cc8oP1SIWnEI5F4h6dKq92fLvnKmxkoWGJxSeW2%2FSOLQECGxOJzWcjJEXQ%3D%3D"
        
        # 모니터링 중인 아파트 목록

        
        # GUI 설정
        self.setup_gui()
        
        # 스케줄러 설정
        self.setup_scheduler()
        
        # 종료 시 처리
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    @property
    def monitored_apts(self):
        # 현재 활성 리스트의 실제 목록을 반환
        return self.monitored_lists["lists"].setdefault(self.active_list.get(), [])
    
    @monitored_apts.setter
    def monitored_apts(self, new_list):
        # 현재 활성 리스트에 새 목록을 설정
        self.monitored_lists["lists"][self.active_list.get()] = new_list
    
    def get_cached_api_data(self, sigungu_code, deal_ymd, api_type='existing'):
        """캐시된 API 데이터 반환 또는 새로 조회"""
        cache_key = (sigungu_code, deal_ymd, api_type)
        
        # 캐시 확인
        if cache_key in self.api_cache:
            cached_data, timestamp = self.api_cache[cache_key]
            if datetime.now() - timestamp < self.cache_ttl:
                self.cache_hit_count += 1
                logging.info(f"캐시 히트: {cache_key} (총 히트: {self.cache_hit_count})")
                return cached_data
            else:
                del self.api_cache[cache_key]
        
        # API 호출
        self.api_call_count += 1
        logging.info(f"API 호출: {cache_key} (총 호출: {self.api_call_count})")
        
        if api_type == 'existing':
            url = (f"http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
                   f"?serviceKey={self.service_key}"
                   f"&LAWD_CD={sigungu_code}"
                   f"&DEAL_YMD={deal_ymd}"
                   f"&numOfRows=1000")
        else:
            url = (f"http://apis.data.go.kr/1613000/RTMSDataSvcSilvTrade/getRTMSDataSvcSilvTrade"
                   f"?serviceKey={self.service_key}"
                   f"&LAWD_CD={sigungu_code}"
                   f"&DEAL_YMD={deal_ymd}"
                   f"&numOfRows=1000")
        
        try:
            response = self.http.get(url, timeout=API_TIMEOUT)
            jitter_sleep(200)
            
            if response.status_code == 200:
                root = ET.fromstring(response.text)
                items = root.findall('.//item')
                
                parsed_data = []
                for item in items:
                    try:
                        data = {
                            'apt_name': item.findtext('aptNm', '').strip(),
                            'dong': item.findtext('umdNm', '').strip(),
                            'area': float(item.findtext('excluUseAr', '0')),
                            'year': int(item.findtext('dealYear')),
                            'month': int(item.findtext('dealMonth')),
                            'day': int(item.findtext('dealDay', '1')),
                            'price': int(item.findtext('dealAmount').replace(',', '')),
                            'floor': int(item.findtext('floor', '0')),
                            'jibun': item.findtext('jibun', '').strip(),
                            'kaptdong': item.findtext('kaptdong', ''),
                            'build_year': item.findtext('buildYear', '').strip()
                        }
                        parsed_data.append(data)
                    except (ValueError, TypeError):
                        continue
                
                # 캐시 저장
                self.api_cache[cache_key] = (parsed_data, datetime.now())
                
                # 캐시 크기 관리
                if len(self.api_cache) > 100:
                    self.cleanup_old_cache()
                
                return parsed_data
            return []
        except Exception as e:
            logging.error(f"API 호출 중 오류: {str(e)}")
            return []


    def _build_search_index_text(self, apt: dict) -> str:
        """
        한 아파트(dict)를 검색 가능한 문자열로 합쳐줌.
        - 단어 포함(부분일치) 위주 검색, 공백으로 여러 키워드 AND 매칭
        """
        fields = [
            apt.get('apt_name', ''),
            str(apt.get('build_year', '')),
            f"{apt.get('area','')}㎡",
            f"{apt.get('sido','')} {apt.get('sigungu','')} {apt.get('dong','')}",
            str(apt.get('max_price_dong', '')),
            str(apt.get('max_price_floor', '')),
            str(apt.get('prev_max_price','')),
            str(apt.get('prev_max_date','')),
            str(apt.get('last_max_price','')),
            str(apt.get('max_price_date','')),
            str(apt.get('last_update','')),
        ]
        return " ".join(map(str, fields)).lower()
    
    def _filter_apts_by_query(self, apts: list, query: str) -> list:
        """
        공백으로 분리된 여러 키워드를 모두 포함(AND)하는 항목만 반환.
        """
        q = (query or '').strip().lower()
        if not q:
            return apts
        tokens = [t for t in q.split() if t]
        if not tokens:
            return apts
    
        filtered = []
        for apt in apts:
            hay = self._build_search_index_text(apt)
            if all(tok in hay for tok in tokens):
                filtered.append(apt)
        return filtered
    
    def calculate_regional_rankings(self):
        """전체 모니터링 단지 기준 지역구별 타입별 순위 계산"""
        rankings = {}
        
        # 모든 모니터링 단지를 순회하며 지역/타입별로 분류
        for apt in self.monitored_apts:
            sido = apt.get('sido', '')
            sigungu = apt.get('sigungu', '').split('(')[0] if apt.get('sigungu') else ''
            
            if not sido or not sigungu:
                continue
                
            region_key = f"{sido} {sigungu}"
            
            if region_key not in rankings:
                rankings[region_key] = {'84': [], '59': []}
            
            # 면적과 가격 정보 추출
            try:
                area_str = str(apt.get('area', '')).replace('㎡', '').strip()
                area = float(area_str)
                price = apt.get('last_max_price', 0)
                apt_name = apt.get('apt_name', '')
                
                if not apt_name or price <= 0:
                    continue
                
                # 84타입 (82-86㎡)
                if 82 <= area <= 86:
                    rankings[region_key]['84'].append((apt_name, price))
                # 59타입 (57-61㎡)
                elif 57 <= area <= 61:
                    rankings[region_key]['59'].append((apt_name, price))
            except:
                continue
        
        # 각 지역/타입별로 가격순 정렬 후 TOP 5 순위 부여
        final_rankings = {}
        for region_key in rankings:
            final_rankings[region_key] = {'84': {}, '59': {}}
            
            # 84타입 TOP 5
            sorted_84 = sorted(rankings[region_key]['84'], key=lambda x: x[1], reverse=True)[:5]
            for rank, (name, _) in enumerate(sorted_84, 1):
                final_rankings[region_key]['84'][name] = rank
            
            # 59타입 TOP 5
            sorted_59 = sorted(rankings[region_key]['59'], key=lambda x: x[1], reverse=True)[:5]
            for rank, (name, _) in enumerate(sorted_59, 1):
                final_rankings[region_key]['59'][name] = rank
        
        return final_rankings
    
    def backfill_build_year(self, apt_info, months=24, fallback_text=None):
        """
        apt_info에 build_year가 비어 있으면 resolve_build_year로 보정한다.
        - months: 캐시 조회 기간(기본 24개월)
        - fallback_text: 목록 문자열(있으면 '(준공: ####년)' / '분양중' 파싱)
        반환: 보정된 연식 문자열('2012' / '분양' / '') 
        """
        by = (apt_info.get('build_year') or '').strip()
        if by:
            return by
    
        try:
            by = self.resolve_build_year(
                sigungu_code=apt_info.get('sigungu_code', ''),
                dong=apt_info.get('dong', ''),
                apt_name=apt_info.get('apt_name', ''),
                months=months,
                fallback_text=fallback_text
            )
            if by:
                apt_info['build_year'] = by
                return by
        except Exception as e:
            logging.error(f"build_year 보정 중 오류: {e}")
        return ''



    
    def resolve_build_year(self, sigungu_code, dong, apt_name, months=12, fallback_text=None):
        """
        최근 months개월의 캐시에서 해당 단지의 buildYear를 우선 탐색.
        없으면 fallback_text(목록 문자열)에서 (준공: ####년) 또는 (분양중) 파싱.
        반환: '2012' / '분양' / ''(미상)
        """
        import re
        now = datetime.now()
    
        # 1) 캐시된 '기축/신축' API에서 buildYear 찾기
        for m in range(months):
            deal_ymd = (now - timedelta(days=30*m)).strftime("%Y%m")
            # 기축
            try:
                for it in self.get_cached_api_data(sigungu_code, deal_ymd, 'existing'):
                    if it.get('apt_name','').strip() == apt_name and it.get('dong','').strip() == dong:
                        by = (it.get('build_year') or '').strip()
                        if by and by.isdigit():
                            return by
            except Exception:
                pass
            # 신축(분양권)
            try:
                for it in self.get_cached_api_data(sigungu_code, deal_ymd, 'new'):
                    if it.get('apt_name','').strip() == apt_name and it.get('dong','').strip() == dong:
                        by = (it.get('build_year') or '').strip()
                        # 신축 API는 보통 buildYear가 없지만, 있으면 사용. 없으면 '분양' 처리
                        if by and by.isdigit():
                            return by
                        # 신축 표기만 있고 연식이 없으면 '분양'으로 간주
                        if not by:
                            return '분양'
            except Exception:
                pass
    
        # 2) 목록 문자열에서 후순위 파싱
        if fallback_text:
            # (준공: 2012년)
            m = re.search(r'준공:\s*(\d{4})년', fallback_text)
            if m:
                return m.group(1)
            # (분양중)
            if '분양중' in fallback_text:
                return '분양'
    
        return ''



    
    def cleanup_old_cache(self):
        """오래된 캐시 항목 정리"""
        current_time = datetime.now()
        expired_keys = []
        for key, (data, timestamp) in self.api_cache.items():
            if current_time - timestamp >= self.cache_ttl:
                expired_keys.append(key)
        for key in expired_keys:
            del self.api_cache[key]
        logging.info(f"캐시 정리: {len(expired_keys)}개 항목 제거")            

    def filter_apt_data(self, all_data, apt_name, dong, target_area):
        """전체 데이터에서 특정 아파트 정보만 필터링"""
        filtered = []
        for item in all_data:
            if (item['apt_name'] == apt_name and 
                item['dong'] == dong and 
                abs(item['area'] - target_area) <= 1):
                
                building_dong = item['kaptdong']
                if not building_dong and item['jibun'] and '동' in item['jibun']:
                    dong_parts = item['jibun'].split('동')
                    if len(dong_parts) > 0 and dong_parts[0].isdigit():
                        building_dong = dong_parts[0] + '동'
                building_dong = building_dong or '-'
                
                trade = {
                    'date': datetime(item['year'], item['month'], item['day']),
                    'price': item['price'],
                    'floor': item['floor'],
                    'area': item['area'],
                    'dong': building_dong
                }
                filtered.append(trade)
        return filtered
    
    def adjust_column_widths(self):
        """Treeview의 모든 열을 내용에 맞춰 자동으로 너비 조정"""
        font = tkFont.Font(font=self.font_normal)
        for col in self.apt_tree["columns"]:
            header_text = self.apt_tree.heading(col, option="text")
            max_width = font.measure(header_text)
            for iid in self.apt_tree.get_children():
                cell_text = self.apt_tree.set(iid, col)
                max_width = max(max_width, font.measure(cell_text))
            self.apt_tree.column(col, width=max_width + 20)

    def load_notifications_history(self):
        """저장된 신고가 히스토리 로드"""
        if os.path.exists(self.notifications_history_file):
            try:
                with open(self.notifications_history_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for notification in data:
                        if 'timestamp' in notification and isinstance(notification['timestamp'], str):
                            try:
                                notification['timestamp'] = datetime.strptime(notification['timestamp'], '%Y-%m-%d %H:%M:%S')
                            except:
                                notification['timestamp'] = datetime.now()
                    cutoff_date = datetime.now() - timedelta(days=30)
                    filtered_data = [n for n in data if n.get('timestamp', datetime.now()) > cutoff_date]
                    logging.info(f"신고가 히스토리를 불러왔습니다. ({len(filtered_data)}개)")
                    return filtered_data
            except Exception as e:
                logging.error(f"신고가 히스토리 로드 중 오류: {str(e)}")
        return []
    
    def save_notifications_history(self):
        """신고가 히스토리 저장"""
        try:
            save_dir = os.path.dirname(self.notifications_history_file)
            os.makedirs(save_dir, exist_ok=True)
            save_data = []
            for notification in self.notifications_history:
                notification_copy = notification.copy()
                if 'timestamp' in notification_copy and isinstance(notification_copy['timestamp'], datetime):
                    notification_copy['timestamp'] = notification_copy['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
                save_data.append(notification_copy)
            with open(self.notifications_history_file, 'w', encoding='utf-8') as f:
                json.dump(save_data, f, ensure_ascii=False, indent=2)
            logging.info(f"신고가 히스토리가 저장되었습니다. ({len(save_data)}개)")
        except Exception as e:
            logging.error(f"신고가 히스토리 저장 중 오류: {str(e)}")

    def load_settings(self):
        """설정 파일 불러오기"""
        try:
            settings_file = os.path.join(os.getcwd(), 'monitor_settings.json')
            if os.path.exists(settings_file):
                try:
                    with open(settings_file, 'r', encoding='utf-8') as f:
                        settings_data = json.load(f)
                        if 'download_path' in settings_data:
                            self.download_path = settings_data['download_path']
                            self.monitored_apts_file = os.path.join(self.download_path, "monitored_apts.json")
                        if 'lawdong_path' in settings_data:
                            self.lawdong_path = settings_data['lawdong_path']
                        if 'auto_update' in settings_data:
                            self.auto_update_enabled.set(settings_data['auto_update'])
                        if 'update_time' in settings_data:
                            self.update_time.set(settings_data['update_time'])
                except Exception as e:
                    logging.error(f"설정 파일 불러오기 중 오류: {str(e)}")
        except Exception as e:
            logging.error(f"설정 파일 경로 확인 중 오류: {str(e)}")
                
    def setup_fonts(self):
        """폰트 설정 (2pt씩 확대)"""
        self.font_normal = ('Malgun Gothic', 11)
        self.font_large  = ('Malgun Gothic', 13)
        self.font_title  = ('Malgun Gothic', 16, 'bold')
        self.font_button = ('Malgun Gothic', 11)
        
    def load_lawdong_file(self):
        """법정동 코드 파일 로드"""
        try:
            if not os.path.exists(self.lawdong_path):
                messagebox.showerror("오류", "법정동 코드 파일이 존재하지 않습니다.")
                return False
            for encoding in ['cp949', 'euc-kr', 'utf-8']:
                try:
                    with open(self.lawdong_path, 'r', encoding=encoding) as file:
                        law_dong_data = []
                        for line in file:
                            parts = line.strip().split('\t')
                            if len(parts) < 2:
                                continue
                            code = parts[0].strip()
                            name = parts[1].strip()
                            if any('폐지' in part for part in parts):
                                continue
                            sido_code = code[:2]
                            sigungu_code = code[2:5]
                            dong_code = code[5:]
                            law_dong_data.append({
                                'code': code,
                                'name': name,
                                'sido_code': sido_code,
                                'sigungu_code': sigungu_code, 
                                'dong_code': dong_code
                            })
                        self.sido_list = []
                        self.sigungu_dict = {}
                        self.dong_dict = {}
                        self.region_codes = {}
                        self.sigungu_to_full_info = {}
                        self.special_sigungu_names = {}
                        self.gu_info = {}  # 구 정보 저장 (구이름 -> 구코드)
                        # 시도
                        sido_data = [item for item in law_dong_data if item['code'].endswith('00000000')]
                        for sido in sido_data:
                            sido_name = sido['name']
                            self.sido_list.append(sido_name)
                            self.sigungu_dict[sido_name] = []
                        # 시군구 및 구
                        sigungu_data = [item for item in law_dong_data 
                                       if item['dong_code'] == '00000' and not item['code'].endswith('00000000')]
                        sigungu_name_count = {}
                        temp_sigungu_list = []
                        for item in sigungu_data:
                            names = item['name'].split()
                            if len(names) >= 2:
                                if len(names) >= 3 and names[2].endswith('구'):
                                    si_name = names[1]
                                    sigungu_name_count[si_name] = sigungu_name_count.get(si_name, 0) + 1
                                    if si_name not in temp_sigungu_list:
                                        temp_sigungu_list.append(si_name)
                                else:
                                    sigungu_name = names[1]
                                    sigungu_name_count[sigungu_name] = sigungu_name_count.get(sigungu_name, 0) + 1
                                    temp_sigungu_list.append(sigungu_name)
                        duplicate_sigungu_names = {name for name, count in sigungu_name_count.items() if count > 1}
                        processed_si = set()
                        for item in sigungu_data:
                            names = item['name'].split()
                            if len(names) >= 2:
                                sido_name = names[0]
                                if len(names) >= 3 and names[2].endswith('구'):
                                    si_name = names[1]
                                    gu_name = names[2]
                                    gu_code = f"{item['sido_code']}{item['sigungu_code']}"
                                    self.gu_info[f"{sido_name}_{si_name}_{gu_name}"] = gu_code
                                    if (sido_name, si_name) not in processed_si:
                                        processed_si.add((sido_name, si_name))
                                        display_name = si_name
                                        if si_name in duplicate_sigungu_names:
                                            sido_abbr = sido_name[0]
                                            display_name = f"{si_name}({sido_abbr})"
                                        if sido_name in self.sigungu_dict:
                                            self.sigungu_dict[sido_name].append(display_name)
                                            self.dong_dict[display_name] = []
                                            si_code = gu_code[:5]
                                            self.sigungu_to_full_info[display_name] = (sido_name, si_name, si_code)
                                else:
                                    sigungu_name = names[1]
                                    if sido_name in self.sido_list:
                                        sigungu_full_code = f"{item['sido_code']}{item['sigungu_code']}"
                                        display_name = sigungu_name
                                        if sigungu_name in duplicate_sigungu_names:
                                            sido_abbr = sido_name[0]
                                            display_name = f"{sigungu_name}({sido_abbr})"
                                            self.special_sigungu_names[display_name] = (sido_name, sigungu_name)
                                        self.sigungu_to_full_info[display_name] = (sido_name, sigungu_name, sigungu_full_code)
                                        if display_name not in self.sigungu_dict[sido_name]:
                                            self.sigungu_dict[sido_name].append(display_name)
                                            self.dong_dict[display_name] = []
                        # 읍면동
                        for item in law_dong_data:
                            if item['dong_code'] != '00000' and not item['code'].endswith('00000'):
                                names = item['name'].split()
                                if len(names) >= 4 and names[2].endswith('구'):
                                    sido_name = names[0]
                                    si_name = names[1]
                                    gu_name = names[2]
                                    dong_name = names[3]
                                    si_display_name = None
                                    for display_name, (s_name, sg_name, _) in self.sigungu_to_full_info.items():
                                        if s_name == sido_name and sg_name == si_name:
                                            si_display_name = display_name
                                            break
                                    if si_display_name:
                                        if gu_name not in self.dong_dict[si_display_name]:
                                            self.dong_dict[si_display_name].append(gu_name)
                                        gu_key = f"{si_display_name}_{gu_name}"
                                        if gu_key not in self.dong_dict:
                                            self.dong_dict[gu_key] = []
                                        if dong_name not in self.dong_dict[gu_key]:
                                            self.dong_dict[gu_key].append(dong_name)
                                        sigungu_code_5digits = f"{item['sido_code']}{item['sigungu_code']}"
                                        self.region_codes[f"{si_display_name}_{gu_name}_{dong_name}"] = (item['code'], sigungu_code_5digits)
                                elif len(names) >= 3:
                                    sido_name = names[0]
                                    sigungu_name = names[1]
                                    dong_name = names[2]
                                    display_name = None
                                    for d_name, (s_name, sg_name, _) in self.sigungu_to_full_info.items():
                                        if s_name == sido_name and sg_name == sigungu_name:
                                            display_name = d_name
                                            break
                                    if display_name and display_name in self.dong_dict:
                                        if dong_name not in self.dong_dict[display_name]:
                                            self.dong_dict[display_name].append(dong_name)
                                            sigungu_code_5digits = f"{item['sido_code']}{item['sigungu_code']}"
                                            self.region_codes[(sido_name, display_name, dong_name)] = (item['code'], sigungu_code_5digits)
                        self.sido_list = sorted(set(self.sido_list))
                        for sido in self.sido_list:
                            self.sigungu_dict[sido] = sorted(set(self.sigungu_dict[sido]))
                        for key in self.dong_dict:
                            self.dong_dict[key] = sorted(set(self.dong_dict[key]))
                        return True
                except UnicodeDecodeError:
                    continue
            messagebox.showerror("오류", "법정동 코드 파일을 읽을 수 없습니다.")
            return False
        except Exception as e:
            messagebox.showerror("오류", f"법정동 코드 파일 로드 중 오류: {str(e)}")
            import traceback
            traceback.print_exc()
            return False
            
    def load_monitored_apts(self):
        """
        저장된 모니터링 '리스트 모음' 로드
        - 신형: {"lists": {리스트명: [...]}, "active_list": "리스트명"}
        - 구형: [아파트...]  => {"lists": {"기본": [...]}, "active_list": "기본"} 로 마이그레이션
        """
        if os.path.exists(self.monitored_apts_file):
            try:
                with open(self.monitored_apts_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
    
                # 구형(list) -> 신형(dict) 마이그레이션
                if isinstance(data, list):
                    # 날짜 문자열 -> datetime 복구
                    for apt in data:
                        if 'trade_data' in apt:
                            for trade in apt['trade_data']:
                                if 'date' in trade and isinstance(trade['date'], str):
                                    try:
                                        trade['date'] = datetime.strptime(trade['date'], '%Y-%m-%d')
                                    except:
                                        pass
                    return {"lists": {"기본": data}, "active_list": "기본"}
    
                # 신형(dict)
                if isinstance(data, dict) and "lists" in data:
                    # 날짜 복구
                    for lst in data["lists"].values():
                        for apt in lst:
                            if 'trade_data' in apt:
                                for trade in apt['trade_data']:
                                    if 'date' in trade and isinstance(trade['date'], str):
                                        try:
                                            trade['date'] = datetime.strptime(trade['date'], '%Y-%m-%d')
                                        except:
                                            pass
                    if "active_list" not in data:
                        data["active_list"] = "기본"
                    if data["active_list"] not in data["lists"]:
                        data["lists"].setdefault("기본", [])
                        data["active_list"] = "기본"
                    return data
    
            except Exception as e:
                logging.error(f"모니터링 목록 로드 중 오류: {str(e)}")
                import traceback
                logging.error(traceback.format_exc())
        # 기본 골격
        return {"lists": {"기본": []}, "active_list": "기본"}

        
    def save_monitored_apts(self):
        """모니터링 '리스트 모음' 저장"""
        try:
            save_dir = os.path.dirname(self.monitored_apts_file)
            os.makedirs(save_dir, exist_ok=True)
    
            # 날짜를 문자열로 직렬화
            serializable = {"lists": {}, "active_list": self.active_list.get()}
            for list_name, lst in self.monitored_lists["lists"].items():
                lst_copy = []
                for apt in lst:
                    apt_copy = apt.copy()
                    if 'trade_data' in apt_copy:
                        for trade in apt_copy['trade_data']:
                            if 'date' in trade and isinstance(trade['date'], datetime):
                                trade['date'] = trade['date'].strftime('%Y-%m-%d')
                    lst_copy.append(apt_copy)
                serializable["lists"][list_name] = lst_copy
    
            with open(self.monitored_apts_file, 'w', encoding='utf-8') as f:
                json.dump(serializable, f, ensure_ascii=False, indent=2)
    
            logging.info(f"모니터링 리스트 저장 완료 (리스트 수: {len(self.monitored_lists['lists'])})")

            # 자동 백업 체크
            self.auto_backup_check()
            
        except Exception as e:
            logging.error(f"모니터링 목록 저장 중 오류: {str(e)}")
            import traceback
            logging.error(traceback.format_exc())

    def backup_monitored_lists(self, *, parent_window=None):
        """모니터링 리스트를 백업"""
        try:
            from tkinter import filedialog
            
            # 백업 파일명 생성
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            default_filename = f"monitored_lists_backup_{timestamp}.json"
            
            # 저장 위치 선택
            if parent_window:
                filepath = filedialog.asksaveasfilename(
                    parent=parent_window,
                    initialdir=self.backup_dir,
                    initialfile=default_filename,
                    title="모니터링 리스트 백업",
                    defaultextension=".json",
                    filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
                )
            else:
                filepath = filedialog.asksaveasfilename(
                    initialdir=self.backup_dir,
                    initialfile=default_filename,
                    title="모니터링 리스트 백업",
                    defaultextension=".json",
                    filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
                )
            
            if not filepath:
                return False
            
            # 현재 데이터 준비
            backup_data = {
                "version": "2.0",
                "backup_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "lists": {},
                "active_list": self.active_list.get(),
                "statistics": {
                    "total_lists": len(self.monitored_lists["lists"]),
                    "total_apts": sum(len(lst) for lst in self.monitored_lists["lists"].values())
                }
            }
            
            # 데이터 복사 및 직렬화
            for list_name, lst in self.monitored_lists["lists"].items():
                lst_copy = []
                for apt in lst:
                    apt_copy = apt.copy()
                    if 'trade_data' in apt_copy:
                        for trade in apt_copy['trade_data']:
                            if 'date' in trade and isinstance(trade['date'], datetime):
                                trade['date'] = trade['date'].strftime('%Y-%m-%d')
                    lst_copy.append(apt_copy)
                backup_data["lists"][list_name] = lst_copy
            
            # 파일 저장
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(backup_data, f, ensure_ascii=False, indent=2)
            
            messagebox.showinfo("백업 완료", f"모니터링 리스트가 백업되었습니다.\n경로: {filepath}")
            logging.info(f"모니터링 리스트 백업 완료: {filepath}")
            return True
            
        except Exception as e:
            messagebox.showerror("백업 실패", f"백업 중 오류가 발생했습니다:\n{str(e)}")
            logging.error(f"백업 중 오류: {str(e)}")
            return False
    
    def restore_monitored_lists(self, *, parent_window=None):
        """백업된 모니터링 리스트 복원"""
        try:
            from tkinter import filedialog
            
            # 복원할 파일 선택
            if parent_window:
                filepath = filedialog.askopenfilename(
                    parent=parent_window,
                    initialdir=self.backup_dir,
                    title="백업 파일 선택",
                    filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
                )
            else:
                filepath = filedialog.askopenfilename(
                    initialdir=self.backup_dir,
                    title="백업 파일 선택",
                    filetypes=[("JSON files", "*.json"), ("All files", "*.*")]
                )
            
            if not filepath:
                return False
            
            # 현재 데이터 백업 (복원 실패시 롤백용)
            current_backup = {
                "lists": self.monitored_lists["lists"].copy(),
                "active_list": self.active_list.get()
            }
            
            try:
                # 백업 파일 읽기
                with open(filepath, 'r', encoding='utf-8') as f:
                    backup_data = json.load(f)
                
                # 데이터 유효성 검증
                if not isinstance(backup_data, dict):
                    raise ValueError("잘못된 백업 파일 형식입니다.")
                
                # 버전 확인
                version = backup_data.get("version", "1.0")
                
                # 리스트 복원
                if "lists" in backup_data:
                    restored_lists = backup_data["lists"]
                elif isinstance(backup_data, list):
                    # 구형 백업 (단일 리스트)
                    restored_lists = {"기본": backup_data}
                else:
                    raise ValueError("백업 파일에서 리스트를 찾을 수 없습니다.")
                
                # 날짜 문자열을 datetime으로 변환
                for list_name, lst in restored_lists.items():
                    for apt in lst:
                        if 'trade_data' in apt:
                            for trade in apt['trade_data']:
                                if 'date' in trade and isinstance(trade['date'], str):
                                    try:
                                        trade['date'] = datetime.strptime(trade['date'], '%Y-%m-%d')
                                    except:
                                        pass
                
                # 복원 확인
                stats = backup_data.get("statistics", {})
                total_lists = stats.get("total_lists", len(restored_lists))
                total_apts = stats.get("total_apts", sum(len(lst) for lst in restored_lists.values()))
                backup_time = backup_data.get("backup_timestamp", "알 수 없음")
                
                msg = f"백업 정보:\n"
                msg += f"- 백업 시간: {backup_time}\n"
                msg += f"- 리스트 수: {total_lists}개\n"
                msg += f"- 총 아파트 수: {total_apts}개\n\n"
                msg += "현재 데이터를 이 백업으로 교체하시겠습니까?"
                
                if not messagebox.askyesno("복원 확인", msg):
                    return False
                
                # 데이터 복원
                self.monitored_lists["lists"] = restored_lists
                
                # 활성 리스트 설정
                if "active_list" in backup_data and backup_data["active_list"] in restored_lists:
                    self.active_list.set(backup_data["active_list"])
                else:
                    # 첫 번째 리스트 선택
                    if restored_lists:
                        self.active_list.set(list(restored_lists.keys())[0])
                
                # UI 업데이트
                self.refresh_list_combobox_values()
                self.update_apt_tree()
                
                # 복원된 데이터 저장
                self.save_monitored_apts()
                
                messagebox.showinfo("복원 완료", 
                                  f"백업이 성공적으로 복원되었습니다.\n"
                                  f"복원된 리스트: {total_lists}개\n"
                                  f"복원된 아파트: {total_apts}개")
                logging.info(f"백업 복원 완료: {filepath}")
                return True
                
            except Exception as e:
                # 복원 실패시 롤백
                self.monitored_lists["lists"] = current_backup["lists"]
                self.active_list.set(current_backup["active_list"])
                raise e
                
        except Exception as e:
            messagebox.showerror("복원 실패", f"백업 복원 중 오류가 발생했습니다:\n{str(e)}")
            logging.error(f"백업 복원 중 오류: {str(e)}")
            return False
    
    def open_backup_folder(self):
        """백업 폴더 열기"""
        try:
            os.makedirs(self.backup_dir, exist_ok=True)
            if os.name == 'nt':  # Windows
                os.startfile(self.backup_dir)
            elif os.name == 'posix':  # macOS/Linux
                os.system(f'open "{self.backup_dir}"')
            else:
                messagebox.showinfo("경로", f"백업 폴더 경로:\n{self.backup_dir}")
        except Exception as e:
            messagebox.showerror("오류", f"백업 폴더를 열 수 없습니다:\n{str(e)}")
    
    def auto_backup_check(self):
        """자동 백업 체크 (save_monitored_apts 호출시)"""
        try:
            if self.last_auto_backup is None:
                self.last_auto_backup = datetime.now()
                return
            
            hours_passed = (datetime.now() - self.last_auto_backup).total_seconds() / 3600
            if hours_passed >= self.auto_backup_interval:
                # 자동 백업 수행
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                auto_backup_file = os.path.join(self.backup_dir, f"auto_backup_{timestamp}.json")
                
                backup_data = {
                    "version": "2.0",
                    "backup_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "lists": {},
                    "active_list": self.active_list.get(),
                    "auto_backup": True
                }
                
                for list_name, lst in self.monitored_lists["lists"].items():
                    lst_copy = []
                    for apt in lst:
                        apt_copy = apt.copy()
                        if 'trade_data' in apt_copy:
                            for trade in apt_copy['trade_data']:
                                if 'date' in trade and isinstance(trade['date'], datetime):
                                    trade['date'] = trade['date'].strftime('%Y-%m-%d')
                        lst_copy.append(apt_copy)
                    backup_data["lists"][list_name] = lst_copy
                
                with open(auto_backup_file, 'w', encoding='utf-8') as f:
                    json.dump(backup_data, f, ensure_ascii=False, indent=2)
                
                self.last_auto_backup = datetime.now()
                logging.info(f"자동 백업 완료: {auto_backup_file}")
                
                # 오래된 자동 백업 삭제 (7일 이상)
                self.cleanup_old_auto_backups()
                
        except Exception as e:
            logging.error(f"자동 백업 중 오류: {str(e)}")
    
    def cleanup_old_auto_backups(self):
        """7일 이상된 자동 백업 파일 삭제"""
        try:
            cutoff_date = datetime.now() - timedelta(days=7)
            for filename in os.listdir(self.backup_dir):
                if filename.startswith("auto_backup_"):
                    filepath = os.path.join(self.backup_dir, filename)
                    file_time = datetime.fromtimestamp(os.path.getmtime(filepath))
                    if file_time < cutoff_date:
                        os.remove(filepath)
                        logging.info(f"오래된 자동 백업 삭제: {filename}")
        except Exception as e:
            logging.error(f"자동 백업 정리 중 오류: {str(e)}")

    
    
    def setup_gui(self):
        """GUI 구성 (다크 테마)"""
        self.root.configure(bg=self.palette['bg'])
        style = ttk.Style(self.root)
        style.theme_use('clam')
        style.configure('TFrame', background=self.palette['surface'])
        style.configure('TLabelframe', background=self.palette['surface'], borderwidth=0, relief='flat')
        style.configure('TLabelframe.Label', background=self.palette['surface'], foreground=self.palette['text_primary'], font=self.font_large)
        style.configure('TLabel', background=self.palette['surface'], foreground=self.palette['text_primary'])
        style.configure('Title.TLabel', background=self.palette['surface'], foreground=self.palette['accent'], font=self.font_title)
        style.configure('Accent.TButton', background=self.palette['accent'], foreground='white', relief='flat', font=self.font_button)
        style.map('Accent.TButton', background=[('active', '#1A8CFF')])
        style.configure('TEntry', fieldbackground=self.palette['surface'], background=self.palette['surface'], foreground=self.palette['text_primary'])
        style.configure('TCombobox', fieldbackground=self.palette['surface'], background=self.palette['surface'], foreground=self.palette['text_primary'])
        style.map('TCombobox', fieldbackground=[('readonly', self.palette['surface'])], foreground=[('disabled', self.palette['text_secondary'])])
        style.configure('TCheckbutton', background=self.palette['bg'], foreground=self.palette['text_primary'])
        style.configure('Treeview', background=self.palette['surface'], fieldbackground=self.palette['surface'], foreground=self.palette['text_primary'], rowheight=26, font=self.font_normal)
        style.configure('Treeview.Heading', background=self.palette['bg'], foreground=self.palette['accent'], relief='flat', font=self.font_normal)
        style.map('Treeview.Heading', background=[('active', self.palette['bg']), ('pressed', self.palette['bg']), ('!active', self.palette['bg'])])
        main_frame = ttk.Frame(self.root, padding="10", style='TFrame')
        main_frame.pack(fill="both", expand=True)
        top_frame = ttk.Frame(main_frame, style='TFrame')
        top_frame.pack(fill="x")
        title_label = ttk.Label(top_frame, text="부태리의 실거래가 모니터", style='Title.TLabel')
        title_label.pack(side="left", pady=(0, 10))
        button_frame = ttk.Frame(top_frame)
        button_frame.pack(side="right", pady=(0, 10))
        ttk.Button(button_frame, text="⚙️ 설정", style='Accent.TButton', 
                  command=self.show_settings_dialog).pack(side="right", padx=(0, 5))
        ttk.Button(button_frame, text="💾 캐시", style='Accent.TButton',   # ← 추가!
                  command=self.show_cache_statistics).pack(side="right", padx=(0, 5))
        ttk.Button(button_frame, text="📊 이전 신고가", style='Accent.TButton', 
                  command=self.show_previous_notifications).pack(side="right")
        region_frame = ttk.LabelFrame(main_frame, text="지역 검색", padding=10)
        region_frame.pack(fill="x", pady=5)
        region_container = ttk.Frame(region_frame)
        region_container.pack(fill="x")
        ttk.Label(region_container, text="시/도:", style='TLabel').grid(row=0, column=0, sticky="w", padx=(0, 5))
        self.sido_combobox = ttk.Combobox(region_container, values=self.sido_list, state="readonly", width=15, style='TCombobox')
        self.sido_combobox.set("시/도 선택")
        self.sido_combobox.grid(row=0, column=1, padx=(0, 10))
        self.sido_combobox.bind('<<ComboboxSelected>>', self.on_sido_selected)
        ttk.Label(region_container, text="시/군/구:", style='TLabel').grid(row=0, column=2, sticky="w", padx=(0, 5))
        self.sigungu_combobox = ttk.Combobox(region_container, state="readonly", width=15, style='TCombobox')
        self.sigungu_combobox.set("시/군/구 선택")
        self.sigungu_combobox.grid(row=0, column=3, padx=(0, 10))
        self.sigungu_combobox.bind('<<ComboboxSelected>>', self.on_sigungu_selected)
        ttk.Label(region_container, text="읍/면/동:", style='TLabel').grid(row=0, column=4, sticky="w", padx=(0, 5))
        self.dong_combobox = ttk.Combobox(region_container, state="readonly", width=15, style='TCombobox')
        self.dong_combobox.set("읍/면/동 선택")
        self.dong_combobox.grid(row=0, column=5, padx=(0, 10))
        self.dong_combobox.bind('<<ComboboxSelected>>', self.on_dong_selected)
        search_button = ttk.Button(region_container, text="아파트 목록 조회", style='Accent.TButton', command=self.show_apt_list)
        search_button.grid(row=0, column=6, padx=(10, 0))
        
        # ▼▼ 추가: 체크박스 (선택시 해당 지역 전체 단지 모니터링)
        bulk_chk = ttk.Checkbutton(
            region_container,
            text="선택시 해당 지역 전체 단지 모니터링",
            variable=self.bulk_monitor_enabled
        )
        bulk_chk.grid(row=0, column=7, padx=(12, 0), sticky="w")
        
        for i in range(8):  # ← 7 → 8 로 변경 (컬럼 개수 증가)
            region_container.grid_columnconfigure(i, weight=1 if i == 7 else 0)
        # === 리스트 선택/관리 바 ===
        list_bar = ttk.LabelFrame(main_frame, text="모니터링 리스트", padding=10)
        list_bar.pack(fill="x", pady=5)
        
        bar_left = ttk.Frame(list_bar)
        bar_left.pack(side="left")
        ttk.Label(bar_left, text="현재 리스트:", style='TLabel').pack(side="left", padx=(0, 6))
        
        # 콤보박스 (리스트 선택)
        self.list_combobox = ttk.Combobox(
            bar_left,
            values=sorted(self.monitored_lists["lists"].keys()),
            textvariable=self.active_list,
            state="readonly",
            width=20,
            style='TCombobox'
        )
        self.list_combobox.pack(side="left")
        self.list_combobox.bind('<<ComboboxSelected>>', self.on_active_list_changed)
        
        bar_right = ttk.Frame(list_bar)
        bar_right.pack(side="right")
        
        ttk.Button(bar_right, text="➕ 추가", style='Accent.TButton', command=self.create_new_list).pack(side="left", padx=4)
        ttk.Button(bar_right, text="✏️ 이름변경", style='Accent.TButton', command=self.rename_current_list).pack(side="left", padx=4)
        ttk.Button(bar_right, text="🗑 삭제", style='Accent.TButton', command=self.delete_current_list).pack(side="left", padx=4)
        ttk.Button(bar_right, text="↪ 선택 이동", style='Accent.TButton', command=self.move_selected_to_list).pack(side="left", padx=4)



            
        monitored_apt_frame = ttk.LabelFrame(main_frame, text="모니터링 중인 아파트", padding=10)
        monitored_apt_frame.pack(fill="both", expand=True, pady=5)

        # --- 검색 바 ---
        search_bar = ttk.Frame(monitored_apt_frame)
        search_bar.pack(fill="x", pady=(0, 6))
        
        self.search_var = tk.StringVar(value="")
        self.search_count_var = tk.StringVar(value="")
        
        def _on_search_change(*_):
            self.update_apt_tree()
        
        def _clear_search():
            self.search_var.set("")
            self.update_apt_tree()
        
        ttk.Label(search_bar, text="검색:").pack(side="left", padx=(0,6))
        search_entry = ttk.Entry(search_bar, textvariable=self.search_var, width=36)
        search_entry.pack(side="left")
        self.search_var.trace_add('write', _on_search_change)
        
        ttk.Button(search_bar, text="지우기", style='Accent.TButton', command=_clear_search)\
           .pack(side="left", padx=(6,6))
        
        # 결과 개수 표시
        ttk.Label(search_bar, textvariable=self.search_count_var)\
           .pack(side="right")
        
        # 단축키: Ctrl+F 로 검색창 포커스
        def _focus_search(event=None):
            try:
                search_entry.focus_set()
                search_entry.select_range(0, 'end')
                return "break"
            except:
                return
        
        self.root.bind_all("<Control-f>", _focus_search)





        
        list_frame = ttk.Frame(monitored_apt_frame)
        list_frame.pack(fill="both", expand=True)
        columns = ("apt_name", "build_year", "area", "location", "dong", "floor",
                   "prev_max_price", "prev_date", "last_max_price", "avg_py_price", "last_date", "last_update")
        
        self.apt_tree = ttk.Treeview(
            list_frame,
            columns=columns,
            show="headings",
            height=10,            # (네가 앞서 10으로 바꿨다면 그대로 두기)
            style='Treeview'
        )
        
        self.apt_tree.heading("apt_name", text="단지명", anchor='center')
        self.apt_tree.heading("build_year", text="연식", anchor='center')
        self.apt_tree.heading("area", text="전용면적", anchor='center')
        self.apt_tree.heading("location", text="주소", anchor='center')
        self.apt_tree.heading("dong", text="동", anchor='center')
        self.apt_tree.heading("floor", text="층", anchor='center')
        self.apt_tree.heading("prev_max_price", text="이전 신고가", anchor='center')
        self.apt_tree.heading("prev_date", text="거래 날짜", anchor='center')
        self.apt_tree.heading("last_max_price", text="최근 신고가", anchor='center')
        self.apt_tree.heading("avg_py_price", text="평균평단가", anchor='center')   # ★ 추가
        self.apt_tree.heading("last_date", text="날짜", anchor='center')
        self.apt_tree.heading("last_update", text="갱신 날짜", anchor='center')
        
        self.apt_tree.column("apt_name", width=120, anchor='center')
        self.apt_tree.column("build_year", width=60, anchor='center')
        self.apt_tree.column("area", width=60, anchor='center')
        self.apt_tree.column("location", width=100, anchor='center')
        self.apt_tree.column("dong", width=50, anchor='center')
        self.apt_tree.column("floor", width=40, anchor='center')
        self.apt_tree.column("prev_max_price", width=100, anchor='center')
        self.apt_tree.column("prev_date", width=100, anchor='center')
        self.apt_tree.column("last_max_price", width=100, anchor='center')
        self.apt_tree.column("avg_py_price", width=90, anchor='center')            # ★ 추가
        self.apt_tree.column("last_date", width=100, anchor='center')
        self.apt_tree.column("last_update", width=140, anchor='center')

        # 헤더 클릭 정렬 바인딩 (모든 열)
        for col in self.apt_tree["columns"]:
            # lambda의 late-binding 방지: 기본값 c=col 캡처
            self.apt_tree.heading(col, command=lambda c=col: self.treeview_sort_column(
                c,
                # 같은 열을 또 누르면 방향 토글, 다른 열을 누르면 내림차순(또는 원하는 기본값) 시작
                (False if self.sort_column != c else not self.sort_reverse)
            ))

        
        scrollbar = ttk.Scrollbar(list_frame, orient="vertical", command=self.apt_tree.yview)
        self.apt_tree.configure(yscrollcommand=scrollbar.set)
        self.apt_tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        btns = ttk.Frame(monitored_apt_frame)
        btns.pack(fill="x", pady=5)
        ttk.Button(btns, text="선택 삭제", style='Accent.TButton', command=self.delete_selected_apt).pack(side="left", padx=5)
        ttk.Button(btns, text="모두 삭제", style='Accent.TButton', command=self.clear_all_apts).pack(side="left", padx=5)
        
        ttk.Button(btns, text="데이터 갱신", style='Accent.TButton', command=self.update_all_data).pack(side="right", padx=5)
        ttk.Button(btns, text="🔍 신고가 다시 찾기", style='Accent.TButton', command=self.recheck_max_prices).pack(side="right", padx=5)
        
        auto_update_frame = ttk.LabelFrame(main_frame, text="자동 업데이트 설정", padding=10)
        auto_update_frame.pack(fill="x", pady=5)
        ttk.Checkbutton(auto_update_frame, text="자동 업데이트 사용", variable=self.auto_update_enabled, command=self.toggle_auto_update).pack(side="left", padx=5)
        ttk.Label(auto_update_frame, text="업데이트 시간:").pack(side="left", padx=5)
        time_entry = ttk.Entry(auto_update_frame, textvariable=self.update_time, width=8)
        time_entry.pack(side="left", padx=5)
        ttk.Label(auto_update_frame, text="(24시간 형식, 예: 09:10)").pack(side="left", padx=5)
        ttk.Button(auto_update_frame, text="적용", style='Accent.TButton', command=self.apply_update_time).pack(side="left", padx=5)
        self.status_var = tk.StringVar(value="준비 완료")
        status_bar = ttk.Label(self.root, textvariable=self.status_var, relief="sunken", anchor="w", background=self.palette['surface'], foreground=self.palette['text_secondary'])
        status_bar.pack(side="bottom", fill="x")
        self.update_apt_tree()
        self.root.after_idle(self.adjust_column_widths)

        # 트리뷰에 더블클릭 이벤트 바인딩 추가
        self.apt_tree.bind('<Double-Button-1>', self.edit_apt_data)
        
        # 우클릭 메뉴 추가
        self.context_menu = tk.Menu(self.root, tearoff=0)
        self.context_menu.add_command(label="데이터 수정", command=self.edit_selected_apt)
        self.context_menu.add_separator()
        self.context_menu.add_command(label="삭제", command=self.delete_selected_apt)
        
        self.apt_tree.bind('<Button-3>', self.show_context_menu)


    

    def refresh_list_combobox_values(self):
        self.list_combobox['values'] = sorted(self.monitored_lists["lists"].keys())
        # active_list 값이 사라진 경우 보정
        if self.active_list.get() not in self.monitored_lists["lists"]:
            # 아무거나 하나 선택
            any_name = sorted(self.monitored_lists["lists"].keys())[0]
            self.active_list.set(any_name)

    def show_context_menu(self, event):
        """우클릭 메뉴 표시"""
        try:
            # 클릭한 위치의 아이템 선택
            item = self.apt_tree.identify('item', event.x, event.y)
            if item:
                self.apt_tree.selection_set(item)
                self.context_menu.post(event.x_root, event.y_root)
        finally:
            self.context_menu.grab_release()
    
    def edit_selected_apt(self):
        """선택된 아파트 데이터 수정"""
        selection = self.apt_tree.selection()
        if not selection:
            messagebox.showinfo("알림", "수정할 아파트를 선택해주세요.")
            return
        self.edit_apt_data(None)
    
    def edit_apt_data(self, event):
        """아파트 데이터 수정 다이얼로그"""
        selection = self.apt_tree.selection()
        if not selection:
            return
        
        # 선택된 아이템 정보 가져오기
        item_id = selection[0]
        item_values = self.apt_tree.item(item_id, "values")
        
        apt_name = item_values[0]
        area_str = item_values[2]
        
        # 면적 값 추출
        import re
        area_value = re.search(r'(\d+(?:\.\d+)?)', area_str)
        if area_value:
            area = area_value.group(1)
        else:
            area = area_str.replace('㎡', '').strip()
        
        # monitored_apts에서 해당 아파트 찾기
        apt_data = None
        apt_index = None
        for idx, apt in enumerate(self.monitored_apts):
            if apt.get('apt_name') == apt_name:
                apt_area = str(apt.get('area', '')).replace('㎡', '').strip()
                try:
                    if float(apt_area) == float(area):
                        apt_data = apt
                        apt_index = idx
                        break
                except:
                    if apt_area == area:
                        apt_data = apt
                        apt_index = idx
                        break
        
        if not apt_data:
            messagebox.showerror("오류", "아파트 정보를 찾을 수 없습니다.")
            return
        
        # 편집 다이얼로그 창
        edit_window = tk.Toplevel(self.root)
        edit_window.title(f"데이터 수정 - {apt_name}")
        edit_window.geometry("500x600")
        edit_window.transient(self.root)
        edit_window.grab_set()
        
        # 창을 화면 중앙에 배치
        edit_window.update_idletasks()
        width = edit_window.winfo_width()
        height = edit_window.winfo_height()
        x = (edit_window.winfo_screenwidth() // 2) - (width // 2)
        y = (edit_window.winfo_screenheight() // 2) - (height // 2)
        edit_window.geometry(f'{width}x{height}+{x}+{y}')
        
        # 스타일 설정
        frame = ttk.Frame(edit_window, padding="10")
        frame.pack(fill="both", expand=True)
        
        # 제목
        title_label = ttk.Label(frame, text=f"{apt_name} ({area}㎡)", 
                               font=self.font_large)
        title_label.grid(row=0, column=0, columnspan=2, pady=(0, 10))
        
        # 입력 필드들
        ttk.Label(frame, text="현재 최고가 정보", font=self.font_large).grid(
            row=1, column=0, columnspan=2, pady=(10, 5), sticky="w")
        
        ttk.Label(frame, text="최고가 (만원):").grid(row=2, column=0, sticky="w", pady=5)
        current_price_var = tk.StringVar(value=str(apt_data.get('last_max_price', 0)))
        current_price_entry = ttk.Entry(frame, textvariable=current_price_var, width=20)
        current_price_entry.grid(row=2, column=1, sticky="w", pady=5)
        
        ttk.Label(frame, text="거래 날짜:").grid(row=3, column=0, sticky="w", pady=5)
        current_date_var = tk.StringVar(value=apt_data.get('max_price_date', ''))
        current_date_entry = ttk.Entry(frame, textvariable=current_date_var, width=20)
        current_date_entry.grid(row=3, column=1, sticky="w", pady=5)
        ttk.Label(frame, text="(형식: YYYY-MM-DD)", font=('Malgun Gothic', 8)).grid(
            row=3, column=2, sticky="w", pady=5)
        
        ttk.Label(frame, text="층:").grid(row=4, column=0, sticky="w", pady=5)
        current_floor_var = tk.StringVar(value=str(apt_data.get('max_price_floor', '')))
        current_floor_entry = ttk.Entry(frame, textvariable=current_floor_var, width=20)
        current_floor_entry.grid(row=4, column=1, sticky="w", pady=5)
        
        ttk.Label(frame, text="동:").grid(row=5, column=0, sticky="w", pady=5)
        current_dong_var = tk.StringVar(value=apt_data.get('max_price_dong', '-'))
        current_dong_entry = ttk.Entry(frame, textvariable=current_dong_var, width=20)
        current_dong_entry.grid(row=5, column=1, sticky="w", pady=5)
        
        # 이전 최고가 정보
        ttk.Label(frame, text="이전 최고가 정보", font=self.font_large).grid(
            row=6, column=0, columnspan=2, pady=(20, 5), sticky="w")
        
        ttk.Label(frame, text="이전 최고가 (만원):").grid(row=7, column=0, sticky="w", pady=5)
        prev_price_var = tk.StringVar(value=str(apt_data.get('prev_max_price', 0)))
        prev_price_entry = ttk.Entry(frame, textvariable=prev_price_var, width=20)
        prev_price_entry.grid(row=7, column=1, sticky="w", pady=5)
        
        ttk.Label(frame, text="이전 거래 날짜:").grid(row=8, column=0, sticky="w", pady=5)
        prev_date_var = tk.StringVar(value=apt_data.get('prev_max_date', ''))
        prev_date_entry = ttk.Entry(frame, textvariable=prev_date_var, width=20)
        prev_date_entry.grid(row=8, column=1, sticky="w", pady=5)
        
        ttk.Label(frame, text="이전 층:").grid(row=9, column=0, sticky="w", pady=5)
        prev_floor_var = tk.StringVar(value=str(apt_data.get('prev_max_floor', '')))
        prev_floor_entry = ttk.Entry(frame, textvariable=prev_floor_var, width=20)
        prev_floor_entry.grid(row=9, column=1, sticky="w", pady=5)
        
        ttk.Label(frame, text="이전 동:").grid(row=10, column=0, sticky="w", pady=5)
        prev_dong_var = tk.StringVar(value=apt_data.get('prev_max_dong', ''))  # 여기 수정됨
        prev_dong_entry = ttk.Entry(frame, textvariable=prev_dong_var, width=20)
        prev_dong_entry.grid(row=10, column=1, sticky="w", pady=5)
        
        # 버튼 프레임
        button_frame = ttk.Frame(frame)
        button_frame.grid(row=11, column=0, columnspan=3, pady=(20, 0))
        
        def save_changes():
            """변경사항 저장"""
            try:
                # 현재 최고가 정보 업데이트
                new_price = int(current_price_var.get())
                apt_data['last_max_price'] = new_price
                apt_data['max_price_date'] = current_date_var.get()
                apt_data['max_price_floor'] = current_floor_var.get()
                apt_data['max_price_dong'] = current_dong_var.get()
                
                # 이전 최고가 정보 업데이트
                prev_price = int(prev_price_var.get()) if prev_price_var.get() else 0
                apt_data['prev_max_price'] = prev_price
                apt_data['prev_max_date'] = prev_date_var.get()
                apt_data['prev_max_floor'] = prev_floor_var.get()
                apt_data['prev_max_dong'] = prev_dong_var.get()
                
                # 업데이트 시간 갱신
                apt_data['last_update'] = datetime.now().strftime('%Y-%m-%d %H:%M')
                
                # 리스트 업데이트
                self.monitored_apts[apt_index] = apt_data
                
                # 저장 및 화면 갱신
                self.save_monitored_apts()
                self.update_apt_tree()
                
                messagebox.showinfo("완료", "데이터가 수정되었습니다.")
                edit_window.destroy()
                
            except ValueError as e:
                messagebox.showerror("오류", "가격은 숫자로 입력해주세요.")
            except Exception as e:
                messagebox.showerror("오류", f"저장 중 오류: {str(e)}")
        
        def swap_prices():
            """현재/이전 최고가 서로 바꾸기"""
            # 값 임시 저장
            temp_price = current_price_var.get()
            temp_date = current_date_var.get()
            temp_floor = current_floor_var.get()
            temp_dong = current_dong_var.get()
            
            # 현재 <- 이전
            current_price_var.set(prev_price_var.get())
            current_date_var.set(prev_date_var.get())
            current_floor_var.set(prev_floor_var.get())
            current_dong_var.set(prev_dong_var.get())
            
            # 이전 <- 임시
            prev_price_var.set(temp_price)
            prev_date_var.set(temp_date)
            prev_floor_var.set(temp_floor)
            prev_dong_var.set(temp_dong)
        
        ttk.Button(button_frame, text="저장", command=save_changes, 
                  style='Accent.TButton').pack(side="left", padx=5)
        ttk.Button(button_frame, text="↔️ 교환", command=swap_prices,
                  style='Accent.TButton').pack(side="left", padx=5)
        ttk.Button(button_frame, text="취소", command=edit_window.destroy).pack(side="left", padx=5)
        
        # 첫 번째 입력 필드에 포커스
        current_price_entry.focus_set()
        current_price_entry.select_range(0, 'end')

    
    def on_active_list_changed(self, event=None):
        # 리스트 변경 시 트리 갱신 + 저장
        self.save_monitored_apts()
        self.update_apt_tree()
        self.status_var.set(f"리스트 전환: {self.active_list.get()}")
    
    def create_new_list(self):
        name = simpledialog.askstring("새 리스트", "리스트 이름을 입력하세요:", parent=self.root)
        if not name:
            return
        name = name.strip()
        if not name:
            return
        if name in self.monitored_lists["lists"]:
            messagebox.showinfo("알림", "이미 존재하는 리스트 이름입니다.")
            return
        self.monitored_lists["lists"][name] = []
        self.active_list.set(name)
        self.refresh_list_combobox_values()
        self.save_monitored_apts()
        self.update_apt_tree()
        self.status_var.set(f"리스트 생성: {name}")
    
    def rename_current_list(self):
        cur = self.active_list.get()
        new_name = simpledialog.askstring("리스트 이름변경", f"'{cur}'의 새 이름:", parent=self.root, initialvalue=cur)
        if not new_name:
            return
        new_name = new_name.strip()
        if not new_name or new_name == cur:
            return
        if new_name in self.monitored_lists["lists"]:
            messagebox.showinfo("알림", "이미 존재하는 리스트 이름입니다.")
            return
        # 키 변경
        self.monitored_lists["lists"][new_name] = self.monitored_lists["lists"].pop(cur)
        self.active_list.set(new_name)
        self.refresh_list_combobox_values()
        self.save_monitored_apts()
        self.update_apt_tree()
        self.status_var.set(f"리스트 이름 변경: {cur} → {new_name}")
    
    def delete_current_list(self):
        cur = self.active_list.get()
        if len(self.monitored_lists["lists"]) <= 1:
            messagebox.showinfo("알림", "마지막 리스트는 삭제할 수 없습니다.")
            return
        if not messagebox.askyesno("확인", f"'{cur}' 리스트를 삭제하시겠습니까?\n(해당 리스트 내 아파트도 함께 삭제됩니다)"):
            return
        # 다른 리스트 하나로 전환
        names = sorted(self.monitored_lists["lists"].keys())
        fallback = next((n for n in names if n != cur), None)
        self.monitored_lists["lists"].pop(cur, None)
        self.active_list.set(fallback or "기본")
        self.refresh_list_combobox_values()
        self.save_monitored_apts()
        self.update_apt_tree()
        self.status_var.set(f"리스트 삭제: {cur}")
    
    def move_selected_to_list(self):
        selection = self.apt_tree.selection()
        if not selection:
            messagebox.showinfo("알림", "이동할 아파트를 선택해주세요.")
            return
        cur = self.active_list.get()
        names = [n for n in sorted(self.monitored_lists["lists"].keys()) if n != cur]
        if not names:
            messagebox.showinfo("알림", "이동할 대상 리스트가 없습니다. 먼저 리스트를 추가해 주세요.")
            return
        # 대상 리스트 선택
        target = simpledialog.askstring("선택 이동", f"이동할 리스트 이름을 입력하세요.\n가능: {', '.join(names)}", parent=self.root)
        if not target or target not in self.monitored_lists["lists"] or target == cur:
            return
    
        # 선택 항목 -> dict로 찾아 옮기기
        moved = 0
        cur_list = self.monitored_lists["lists"][cur]
        tgt_list = self.monitored_lists["lists"][target]
    
        # 현재 트리에서 선택된 값으로 매칭
        for item_id in selection:
            vals = self.apt_tree.item(item_id, "values")
            apt_name = vals[0]
            area_str = vals[2]
            import re
            m = re.search(r'(\d+(?:\.\d+)?)', area_str)
            area_val = m.group(1) if m else area_str.replace('㎡','').strip()
            # 현재 리스트에서 동일 항목 찾기
            idx_to_move = None
            for idx, apt in enumerate(cur_list):
                if apt.get('apt_name') == apt_name:
                    try:
                        if float(str(apt.get('area','')).replace('㎡','').strip()) == float(area_val):
                            idx_to_move = idx
                            break
                    except:
                        if str(apt.get('area','')).replace('㎡','').strip() == str(area_val):
                            idx_to_move = idx
                            break
            if idx_to_move is not None:
                tgt_list.append(cur_list.pop(idx_to_move))
                moved += 1
    
        if moved > 0:
            self.save_monitored_apts()
            self.update_apt_tree()
            self.status_var.set(f"{moved}개 항목을 '{cur}' → '{target}'로 이동했습니다.")
        else:
            self.status_var.set("이동할 항목을 찾지 못했습니다.")




    
    def show_cache_statistics(self):
        """캐시 통계 표시"""
        stats_window = tk.Toplevel(self.root)
        stats_window.title("API 캐시 통계")
        stats_window.geometry("400x300")
        stats_window.transient(self.root)
        
        frame = ttk.Frame(stats_window, padding="10")
        frame.pack(fill="both", expand=True)
        
        ttk.Label(frame, text="API 캐시 통계", font=self.font_title).pack(pady=(0, 10))
        
        hit_rate = 0
        if (self.api_call_count + self.cache_hit_count) > 0:
            hit_rate = (self.cache_hit_count / (self.api_call_count + self.cache_hit_count)) * 100
        
        stats_text = f"""
        캐시 크기: {len(self.api_cache)}개
        총 API 호출: {self.api_call_count}회
        캐시 히트: {self.cache_hit_count}회
        캐시 히트율: {hit_rate:.1f}%
        캐시 TTL: {self.cache_ttl.total_seconds() / 3600:.1f}시간
        """
        
        ttk.Label(frame, text=stats_text, font=self.font_normal).pack(pady=10)
        
        def clear_cache():
            if messagebox.askyesno("확인", "캐시를 모두 삭제하시겠습니까?"):
                self.api_cache.clear()
                self.cache_hit_count = 0
                self.api_call_count = 0
                messagebox.showinfo("알림", "캐시가 초기화되었습니다.")
                stats_window.destroy()
        
        button_frame = ttk.Frame(frame)
        button_frame.pack(fill="x", pady=(10, 0))
        ttk.Button(button_frame, text="캐시 초기화", command=clear_cache).pack(side="left")
        ttk.Button(button_frame, text="닫기", command=stats_window.destroy).pack(side="right")
    
    def show_previous_notifications(self):
        """이전 신고가 히스토리 표시"""
        if not self.notifications_history:
            messagebox.showinfo("알림", "이전 신고가 기록이 없습니다.")
            return
        history_dialog = tk.Toplevel(self.root)
        history_dialog.title("이전 신고가 기록")
        history_dialog.geometry("600x1000")
        history_dialog.transient(self.root)
        history_dialog.grab_set()
        screen_width = history_dialog.winfo_screenwidth()
        screen_height = history_dialog.winfo_screenheight()
        x = (screen_width - 600) // 2
        y = (screen_height - 400) // 2
        history_dialog.geometry(f"600x600+{x}+{y}")
        top_frame = ttk.Frame(history_dialog, padding="10")
        top_frame.pack(fill="x")
        ttk.Label(top_frame, text="이전 신고가 기록", font=self.font_title).pack(side="left")
        list_frame = ttk.Frame(history_dialog, padding="10")
        list_frame.pack(fill="both", expand=True)
        columns = ("timestamp", "count", "max_increase", "preview")
        history_tree = ttk.Treeview(list_frame, columns=columns, show="headings", height=12)
        history_tree.heading("timestamp", text="발생 시간")
        history_tree.heading("count", text="아파트 수")
        history_tree.heading("max_increase", text="최대 상승률")
        history_tree.heading("preview", text="미리보기")
        history_tree.column("timestamp", width=150)
        history_tree.column("count", width=80)
        history_tree.column("max_increase", width=100)
        history_tree.column("preview", width=250)
        scrollbar = ttk.Scrollbar(list_frame, orient="vertical", command=history_tree.yview)
        history_tree.configure(yscrollcommand=scrollbar.set)
        history_tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        sorted_history = sorted(self.notifications_history, key=lambda x: x.get('timestamp', datetime.now()), reverse=True)
        for i, notification_item in enumerate(sorted_history):
            timestamp = notification_item.get('timestamp', datetime.now())
            if isinstance(timestamp, str):
                try:
                    timestamp = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                except:
                    timestamp = datetime.now()
            apt_list = notification_item.get('apt_list', [])
            count = len(apt_list)
            max_increase = 0
            preview_text = ""
            if apt_list:
                for apt in apt_list:
                    if apt.get('old_price', 0) > 0:
                        increase_percent = ((apt.get('new_price', 0) - apt.get('old_price', 0)) / apt.get('old_price', 0)) * 100
                        max_increase = max(max_increase, increase_percent)
                first_apt = apt_list[0]
                preview_text = f"{first_apt.get('apt_name', '')} {first_apt.get('area', '')}㎡"
                if count > 1:
                    preview_text += f" 외 {count-1}개"
            history_tree.insert("", "end", values=(
                timestamp.strftime('%Y-%m-%d %H:%M'),
                f"{count}개",
                f"+{max_increase:.1f}%" if max_increase > 0 else "-",
                preview_text
            ), tags=(str(i),))
        def on_double_click(event):
            selection = history_tree.selection()
            if selection:
                item = history_tree.item(selection[0])
                tag = item['tags'][0] if item['tags'] else '0'
                idx = int(tag)
                if 0 <= idx < len(sorted_history):
                    selected_notification = sorted_history[idx]
                    apt_list = selected_notification.get('apt_list', [])
                    if apt_list:
                        history_dialog.destroy()
                        original_history_count = len(self.notifications_history)
                        self.show_new_max_notification(apt_list)
                        if len(self.notifications_history) > original_history_count:
                            self.notifications_history = self.notifications_history[:-1]
                            self.save_notifications_history()
        history_tree.bind('<Double-1>', on_double_click)
        button_frame = ttk.Frame(history_dialog, padding="10")
        button_frame.pack(fill="x")
        def show_selected():
            selection = history_tree.selection()
            if not selection:
                messagebox.showinfo("알림", "보려는 기록을 선택해주세요.")
                return
            on_double_click(None)
        def delete_selected():
            selection = history_tree.selection()
            if not selection:
                messagebox.showinfo("알림", "삭제할 기록을 선택해주세요.")
                return
            if not messagebox.askyesno("확인", "선택한 신고가 기록을 삭제하시겠습니까?"):
                return
            try:
                indices_to_delete = []
                for item in selection:
                    tag = history_tree.item(item)['tags'][0] if history_tree.item(item)['tags'] else '0'
                    idx = int(tag)
                    if 0 <= idx < len(sorted_history):
                        target_notification = sorted_history[idx]
                        for orig_idx, orig_notification in enumerate(self.notifications_history):
                            if (orig_notification.get('timestamp') == target_notification.get('timestamp') and
                                len(orig_notification.get('apt_list', [])) == len(target_notification.get('apt_list', []))):
                                indices_to_delete.append(orig_idx)
                                break
                for idx in sorted(set(indices_to_delete), reverse=True):
                    if 0 <= idx < len(self.notifications_history):
                        del self.notifications_history[idx]
                self.save_notifications_history()
                for item in selection:
                    history_tree.delete(item)
                messagebox.showinfo("알림", f"{len(indices_to_delete)}개의 기록이 삭제되었습니다.")
            except Exception as e:
                messagebox.showerror("오류", f"기록 삭제 중 오류가 발생했습니다: {str(e)}")
        def delete_all():
            if not self.notifications_history:
                messagebox.showinfo("알림", "삭제할 기록이 없습니다.")
                return
            if not messagebox.askyesno("확인", f"모든 신고가 기록({len(self.notifications_history)}개)을 삭제하시겠습니까?\n\n이 작업은 되돌릴 수 없습니다."):
                return
            try:
                self.notifications_history = []
                self.save_notifications_history()
                for item in history_tree.get_children():
                    history_tree.delete(item)
                messagebox.showinfo("알림", "모든 신고가 기록이 삭제되었습니다.")
            except Exception as e:
                messagebox.showerror("오류", f"기록 삭제 중 오류: {str(e)}")
        ttk.Button(button_frame, text="선택 삭제", style='Accent.TButton', command=delete_selected).pack(side="left", padx=5)
        ttk.Button(button_frame, text="전체 삭제", style='Accent.TButton', command=delete_all).pack(side="left", padx=5)
        ttk.Button(button_frame, text="보기",      style='Accent.TButton', command=show_selected).pack(side="right", padx=5)
        ttk.Button(button_frame, text="닫기",      style='Accent.TButton', command=history_dialog.destroy).pack(side="right", padx=5)

    def treeview_sort_column(self, col, reverse):
        self.sort_column = col
        self.sort_reverse = reverse
        self.update_apt_tree()
        # 다음 클릭 때 방향 토글되도록 현재 열에 재바인딩
        self.apt_tree.heading(col, command=lambda: self.treeview_sort_column(col, not reverse))

    def show_settings_dialog(self):
        """설정 대화상자"""
        settings = tk.Toplevel(self.root)
        settings.title("설정")
        settings.geometry("800x300")
        settings.resizable(False, False)
        settings.transient(self.root)
        settings.grab_set()
        settings.configure(bg=self.palette['bg'])
        style = ttk.Style(settings)
        style.theme_use('clam')
        ttk.Label(settings, text="다운로드 경로:", background=self.palette['surface'], foreground=self.palette['text_primary']).grid(row=0, column=0, sticky="w", padx=10, pady=10)
        download_path_var = tk.StringVar(value=self.download_path)
        ttk.Entry(settings, textvariable=download_path_var, width=40).grid(row=0, column=1, padx=5, pady=10)
        def select_download_path():
            path = filedialog.askdirectory(initialdir=self.download_path)
            if path:
                download_path_var.set(path)
        ttk.Button(settings, text="찾아보기", style='Accent.TButton', command=select_download_path).grid(row=0, column=2, padx=5, pady=10)
        ttk.Label(settings, text="법정동 파일 경로:", background=self.palette['surface'], foreground=self.palette['text_primary']).grid(row=1, column=0, sticky="w", padx=10, pady=10)
        lawdong_path_var = tk.StringVar(value=self.lawdong_path)
        ttk.Entry(settings, textvariable=lawdong_path_var, width=40).grid(row=1, column=1, padx=5, pady=10)
        def select_lawdong_path():
            path = filedialog.askopenfilename(initialdir=os.path.dirname(self.lawdong_path), title="법정동 코드 파일 선택", filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
            if path:
                lawdong_path_var.set(path)
        ttk.Button(settings, text="찾아보기", style='Accent.TButton', command=select_lawdong_path).grid(row=1, column=2, padx=5, pady=10)
        # 백업/복원 섹션 추가
        ttk.Label(settings, text="모니터링 리스트 백업:", background=self.palette['surface'], foreground=self.palette['text_primary']).grid(row=2, column=0, sticky="w", padx=10, pady=10)
        
        backup_restore_frame = ttk.Frame(settings, style='TFrame')
        backup_restore_frame.grid(row=2, column=1, columnspan=2, sticky="w", padx=5, pady=10)
        
        ttk.Button(backup_restore_frame, text="백업 저장", style='Accent.TButton', 
                  command=lambda: self.backup_monitored_lists(parent_window=settings)).pack(side='left', padx=5)
        ttk.Button(backup_restore_frame, text="백업 불러오기", style='Accent.TButton', 
                  command=lambda: self.restore_monitored_lists(parent_window=settings)).pack(side='left', padx=5)
        ttk.Button(backup_restore_frame, text="자동 백업 폴더 열기", style='Accent.TButton', 
                  command=self.open_backup_folder).pack(side='left', padx=5)


        
        button_frame = ttk.Frame(settings, style='TFrame')
        button_frame.grid(row=3, column=0, columnspan=3, sticky="e", padx=10, pady=20)
        def save_settings():
            new_dp = download_path_var.get()
            if new_dp:
                os.makedirs(new_dp, exist_ok=True)
                self.download_path = new_dp
                self.monitored_apts_file = os.path.join(self.download_path, "monitored_apts.json")
            new_lp = lawdong_path_var.get()
            if new_lp and os.path.exists(new_lp):
                self.lawdong_path = new_lp
            settings_data = {
                'download_path': self.download_path,
                'lawdong_path': self.lawdong_path,
                'auto_update': self.auto_update_enabled.get(),
                'update_time': self.update_time.get()
            }
            with open(os.path.join(os.getcwd(), 'monitor_settings.json'), 'w', encoding='utf-8') as f:
                json.dump(settings_data, f, ensure_ascii=False, indent=2)
            self.load_lawdong_file()
            messagebox.showinfo("알림", "설정이 저장되었습니다.")
            settings.destroy()
        ttk.Button(button_frame, text="취소", style='Accent.TButton', command=settings.destroy).pack(side='right', padx=5)
        ttk.Button(button_frame, text="저장", style='Accent.TButton', command=save_settings).pack(side='right')

    def setup_scheduler(self):
        """스케줄러 설정"""
        self.scheduler_thread = threading.Thread(target=self.run_scheduler, daemon=True)
        self.scheduler_thread.start()
    
    def run_scheduler(self):
        """스케줄러 실행"""
        while True:
            schedule.run_pending()
            time.sleep(1)
    
    def toggle_auto_update(self):
        """자동 업데이트 토글 (태그 관리)"""
        if self.auto_update_enabled.get():
            self.apply_update_time()
        else:
            schedule.clear('auto-update')
            self.status_var.set("자동 업데이트가 비활성화되었습니다.")
    
    def apply_update_time(self):
        """업데이트 시간 적용 (태그 관리)"""
        try:
            schedule.clear('auto-update')
            if self.auto_update_enabled.get():
                update_time = self.update_time.get()
                hours, minutes = map(int, update_time.split(':'))
                if not (0 <= hours < 24 and 0 <= minutes < 60):
                    raise ValueError("올바른 시간 형식이 아닙니다.")
                schedule.every().day.at(update_time).do(self.update_all_data).tag('auto-update')
                self.status_var.set(f"자동 업데이트가 {update_time}에 실행되도록 설정되었습니다.")
        except Exception as e:
            messagebox.showerror("오류", f"업데이트 시간 설정 오류: {str(e)}")
            self.status_var.set("자동 업데이트 설정 중 오류가 발생했습니다.")
        
    def update_apt_tree(self):
        """모니터링 아파트 목록 트리뷰 업데이트 (검색 필터 적용)"""
        # 모두 지우기
        for item in self.apt_tree.get_children():
            self.apt_tree.delete(item)
    
        # 1) 정렬
        sorted_apts = self.get_sorted_apts()
    
        # 2) 검색 필터
        query = getattr(self, 'search_var', tk.StringVar(value="")).get() if hasattr(self, 'search_var') else ""
        filtered_apts = self._filter_apts_by_query(sorted_apts, query)
    
        # 3) 표시

        for apt in filtered_apts:   # 또는 sorted_apts 사용 중이면 거기 맞춰서
            prev_max_price = apt.get('prev_max_price', 0)
            prev_max_price_str = f"{prev_max_price:,}만원" if prev_max_price > 0 else "-"
            prev_date = apt.get('prev_max_date', '')
    
            last_max_price = apt.get('last_max_price', 0)
            last_max_price_str = f"{last_max_price:,}만원" if last_max_price > 0 else "-"
            last_date = apt.get('max_price_date', '-')
            dong_info = apt.get('max_price_dong', '-')
            floor_info = apt.get('max_price_floor', '-')
    
            build_year = apt.get('build_year', '')
            build_year_str = f"{build_year}년" if (build_year and build_year != '분양') else ("분양" if build_year == '분양' else "-")
            location = f"{apt.get('sigungu', '')} {apt.get('dong', '')}"
    
            # ★ 평균평단가 계산: 최근신고가 / 전용면적 * 3.3
            try:
                area_val = float(str(apt.get('area','')).replace('㎡','').strip() or 0)
            except:
                area_val = 0.0
            if last_max_price and area_val > 0:
                avg_py = last_max_price / area_val * 3.3
                # 표시는 반올림 정수(만원/평). 원하면 소수 1자리로 바꿔도 됨: f"{avg_py:,.1f}만원/평"
                avg_py_str = f"{round(avg_py):,}만원/평"
            else:
                avg_py_str = "-"
    
            item_id = self.apt_tree.insert("", "end", values=(
                apt["apt_name"],
                build_year_str,
                f"{apt['area']}㎡",
                location,
                dong_info,
                floor_info,
                prev_max_price_str,
                prev_date,
                last_max_price_str,
                avg_py_str,                 # ★ 추가: 최근 신고가 바로 다음
                last_date,
                apt.get("last_update", "업데이트 필요")
            ))
    
            if prev_max_price > 0 and last_max_price > prev_max_price:
                self.apt_tree.item(item_id, tags=('price_up',))
            elif prev_max_price > 0 and last_max_price < prev_max_price:
                self.apt_tree.item(item_id, tags=('price_down',))
    
        self.apt_tree.tag_configure('price_up', background='#FFDDDD', foreground='#3AA6FF')
        self.apt_tree.tag_configure('price_down', background='#DDDDFF')
    
        # 결과 개수 표시
        try:
            total = len(sorted_apts)
            shown = len(filtered_apts)
            self.search_count_var.set(f"{shown}/{total}개 표시")
        except Exception:
            pass
    
        self.adjust_column_widths()


    def get_sorted_apts(self):
        """정렬된 아파트 목록 반환"""
        sorted_apts = self.monitored_apts.copy()
        def get_sort_key(apt):
            if self.sort_column == "apt_name":
                return apt.get("apt_name", "")
            elif self.sort_column == "build_year":
                try:
                    return int(apt.get("build_year", 0))
                except:
                    return 0
            elif self.sort_column == "area":
                try:
                    return float(apt.get("area", 0))
                except:
                    return 0
            elif self.sort_column == "location":
                return f"{apt.get('sigungu', '')} {apt.get('dong', '')}"
            elif self.sort_column == "dong":
                return apt.get("max_price_dong", "")
            elif self.sort_column == "prev_max_price":
                return apt.get("prev_max_price", 0)
            elif self.sort_column == "prev_date":
                return apt.get("prev_max_date", "")
            elif self.sort_column == "last_max_price":
                return apt.get("last_max_price", 0)
            elif self.sort_column == "last_date":
                return apt.get("max_price_date", "")
            elif self.sort_column == "floor":
                try:
                    return int(apt.get("max_price_floor", 0))
                except:
                    return 0
            elif self.sort_column == "last_update":
                return apt.get("last_update", "")
            elif self.sort_column == "avg_py_price":     # ★ 추가
                try:
                    area_val = float(str(apt.get("area","")).replace("㎡","").strip() or 0)
                except:
                    area_val = 0.0
                last_max = apt.get("last_max_price", 0) or 0
                return (last_max / area_val * 3.3) if (last_max and area_val > 0) else -1
            else:
                return 0
    
        sorted_apts.sort(key=get_sort_key, reverse=self.sort_reverse)
        return sorted_apts

    def on_sido_selected(self, event):
        """시/도 선택"""
        sido = self.sido_combobox.get()
        if sido in self.sigungu_dict:
            self.sigungu_combobox['values'] = sorted(self.sigungu_dict[sido])
            self.sigungu_combobox.set("시/군/구 선택")
            self.dong_combobox.set("읍/면/동 선택")
        
    def on_sigungu_selected(self, event):
        """시/군/구 선택"""
        sigungu = self.sigungu_combobox.get()
        if sigungu in self.dong_dict:
            dong_list = self.dong_dict[sigungu]
            gu_list = [dong for dong in dong_list if dong.endswith('구')]
            if gu_list:
                all_items = []
                for gu in sorted(gu_list):
                    all_items.append(gu)
                    gu_key = f"{sigungu}_{gu}"
                    if gu_key in self.dong_dict:
                        gu_dong_list = self.dong_dict[gu_key]
                        for dong in sorted(gu_dong_list):
                            all_items.append(f"  └ {dong}")
                normal_dong_list = [dong for dong in dong_list if not dong.endswith('구')]
                if normal_dong_list:
                    all_items.extend(sorted(normal_dong_list))
                self.dong_combobox['values'] = all_items
            else:
                self.dong_combobox['values'] = sorted(dong_list)
            self.dong_combobox.set("읍/면/동 선택")
        else:
            self.dong_combobox['values'] = []
            self.dong_combobox.set("읍/면/동 선택")
    
    def on_dong_selected(self, event):
        """읍/면/동 선택"""
        dong = self.dong_combobox.get()
        sigungu = self.sigungu_combobox.get()
        if dong.endswith('구'):
            gu_key = f"{sigungu}_{dong}"
            if gu_key in self.dong_dict:
                dong_list = sorted(self.dong_dict[gu_key])
                self.dong_combobox['values'] = dong_list
                self.dong_combobox.set("읍/면/동 선택")
                self.status_var.set(f"{dong}의 하위 동을 선택해주세요.")
            else:
                self.status_var.set("아파트 목록 조회 버튼을 눌러 진행하세요.")
        else:
            self.status_var.set("아파트 목록 조회 버튼을 눌러 진행하세요.")
    
    def show_apt_list(self):
        """아파트 목록 조회 및 표시"""
        sido = self.sido_combobox.get()
        sigungu = self.sigungu_combobox.get()
        dong = self.dong_combobox.get()
        original_dong = dong
        
        # ⭐ 디버그 로그 추가
        print(f"\n{'='*80}")
        print(f"=== show_apt_list 호출 ===")
        print(f"sido: '{sido}'")
        print(f"sigungu: '{sigungu}'")
        print(f"dong: '{dong}'")
        print(f"체크박스: {self.bulk_monitor_enabled.get()}")
        print(f"'선택' not in [sido, sigungu]: {'선택' not in [sido, sigungu]}")
        print(f"'선택' in dong: {'선택' in dong}")
        print(f"{'='*80}\n")  # ← 올바른 따옴표
        
        # ⭐⭐⭐ 이 부분이 핵심! 순서 변경 ⭐⭐⭐
        # 1️⃣ 먼저 체크박스가 켜져있고 시/군/구까지만 선택된 경우 처리
        if self.bulk_monitor_enabled.get() and "선택" not in [sido, sigungu] and "선택" in dong:
            print(">>> ✅ 시/군/구 전체 등록 분기 진입!")
            # 해당 시/군/구의 모든 동 처리
            if messagebox.askyesno("확인", 
                f"{sido} {sigungu}의 모든 동을 일괄 등록하시겠습니까?\n"
                "이 작업은 오래 걸릴 수 있습니다."):
                self.bulk_add_all_dongs_in_sigungu(sido, sigungu)
            return  # ⭐ 여기서 종료!
        
        # 2️⃣ 체크박스가 켜져있고 시/도만 선택된 경우
        if self.bulk_monitor_enabled.get() and "선택" not in [sido] and "선택" in [sigungu, dong]:
            print(">>> ✅ 시/도 전체 등록 분기 진입!")
            # 해당 시/도의 모든 시군구와 동 처리
            if messagebox.askyesno("확인", 
                f"{sido}의 모든 지역을 일괄 등록하시겠습니까?\n"
                "이 작업은 매우 오래 걸릴 수 있습니다. (수 시간 소요 예상)"):
                self.bulk_add_all_areas_in_sido(sido)
            return
        
        # 3️⃣ 일반 동 선택 처리
        if dong.startswith("  └ "):
            dong = dong.replace("  └ ", "").strip()
        
        if "선택" in [sido, sigungu, dong]:
            print(">>> ❌ 지역을 모두 선택해주세요")
            messagebox.showerror("오류", "지역을 모두 선택해주세요.")
            return
        
        if dong.startswith("  └ "):
            dong = dong.replace("  └ ", "").strip()
        if "선택" in [sido, sigungu, dong]:
            messagebox.showerror("오류", "지역을 모두 선택해주세요.")
            return
        parent_gu = None
        if original_dong.startswith("  └ "):
            dong_list = self.dong_combobox['values']
            for i, item in enumerate(dong_list):
                if item == original_dong:
                    for j in range(i-1, -1, -1):
                        if not dong_list[j].startswith("  └ ") and dong_list[j].endswith('구'):
                            parent_gu = dong_list[j]
                            break
                    break
        sigungu_code_to_use = None
        if parent_gu:
            if hasattr(self, 'gu_info'):
                if sigungu in self.sigungu_to_full_info:
                    _, original_si, _ = self.sigungu_to_full_info[sigungu]
                else:
                    original_si = sigungu.replace('(경)', '').replace('(충)', '').replace('(전)', '').strip()
                gu_key = f"{sido}_{original_si}_{parent_gu}"
                if gu_key in self.gu_info:
                    sigungu_code_to_use = self.gu_info[gu_key]
            if not sigungu_code_to_use:
                region_key = f"{sigungu}_{parent_gu}_{dong}"
                if region_key in self.region_codes:
                    _, sigungu_code_to_use = self.region_codes[region_key]
        elif dong.endswith('구'):
            if hasattr(self, 'gu_info'):
                if sigungu in self.sigungu_to_full_info:
                    _, original_si, _ = self.sigungu_to_full_info[sigungu]
                else:
                    original_si = sigungu.replace('(경)', '').replace('(충)', '').replace('(전)', '').strip()
                gu_key = f"{sido}_{original_si}_{dong}"
                if gu_key in self.gu_info:
                    sigungu_code_to_use = self.gu_info[gu_key]
        else:
            region_code = self.region_codes.get((sido, sigungu, dong))
            if region_code:
                sigungu_code_to_use = region_code[1]
        if not sigungu_code_to_use:
            if sigungu in self.sigungu_to_full_info:
                _, _, sigungu_code_to_use = self.sigungu_to_full_info[sigungu]
            else:
                messagebox.showerror("오류", "해당 지역의 코드를 찾을 수 없습니다.")
                return
        try:
            self.status_var.set("아파트 목록 검색 중...")
            self.root.update_idletasks()
            apt_list = self.get_apt_list_from_api(sigungu_code_to_use, dong)
            if apt_list:
                if self.bulk_monitor_enabled.get():
                    # ▼▼ 추가: 체크박스 켜짐 → 동 내 조회된 모든 단지를 일괄 추가
                    self.bulk_add_all_complexes_in_dong(
                        sigungu_code=sigungu_code_to_use,
                        dong=dong,
                        sido=sido,
                        sigungu=sigungu,
                        apt_list=apt_list  # get_apt_list_from_api() 반환 그대로 투입
                    )
                else:
                    # 기존 동작: 단일 단지 선택 다이얼로그
                    dialog = AptSelectDialog(
                        self.root, 
                        apt_list,
                        self.service_key,
                        sigungu_code_to_use,
                        dong,
                        sido,
                        sigungu,
                        title=f"{dong} 아파트 목록"
                    )
                    self.root.wait_window(dialog.top)
                    if dialog.result:
                        apt_info = dialog.result
                        apt_info['sigungu_code'] = sigungu_code_to_use
                        self.add_apt_to_monitored(apt_info)
            else:
                messagebox.showinfo("알림", f"{dong}에 거래 내역이 있는 아파트가 없습니다.")

        except Exception as e:
            messagebox.showerror("오류", f"아파트 목록 검색 중 오류: {str(e)}")
            import traceback
            traceback.print_exc()

    def bulk_add_all_areas_in_sido(self, sido):
        """시/도 내 모든 시군구의 모든 동 단지를 일괄 등록"""
        # 진행창
        win = tk.Toplevel(self.root)
        win.title(f"{sido} 전체 단지 등록 중...")
        win.geometry("700x250")
        win.transient(self.root)
        win.grab_set()
        
        ttk.Label(win, text=f"{sido}의 모든 시/군/구와 동을 순회하며 단지를 등록합니다.", 
                 font=self.font_large).pack(pady=10)
        
        progress_label = ttk.Label(win, text="준비 중...")
        progress_label.pack(pady=5)
        
        bar = ttk.Progressbar(win, orient="horizontal", length=660, mode="determinate")
        bar.pack(pady=10, padx=20)
        
        info_label = ttk.Label(win, text="")
        info_label.pack(pady=5)
        
        stats_label = ttk.Label(win, text="")
        stats_label.pack(pady=5)
        
        cancel_flag = [False]
        
        def _cancel():
            cancel_flag[0] = True
            try: win.destroy()
            except: pass
        
        ttk.Button(win, text="중단", command=_cancel).pack(pady=10)
        
        def process():
            try:
                # 해당 시/도의 시군구 목록 가져오기
                if sido not in self.sigungu_dict:
                    messagebox.showerror("오류", f"{sido}의 시군구 목록을 찾을 수 없습니다.")
                    win.destroy()
                    return
                
                sigungu_list = self.sigungu_dict[sido]
                total_sigungus = len(sigungu_list)
                total_dongs_processed = 0
                total_apts_added = 0
                
                for sigungu_idx, sigungu in enumerate(sigungu_list):
                    if cancel_flag[0]:
                        break
                    
                    # 시군구 진행률
                    sigungu_progress = (sigungu_idx / total_sigungus) * 100
                    progress_label.config(text=f"시/군/구: {sigungu_idx + 1}/{total_sigungus} - {sigungu}")
                    
                    # 시군구 코드 얻기
                    sigungu_code = None
                    if sigungu in self.sigungu_to_full_info:
                        _, _, sigungu_code = self.sigungu_to_full_info[sigungu]
                    
                    if not sigungu_code:
                        continue
                    
                    # 해당 시군구의 동 목록 가져오기
                    dong_list = []
                    if sigungu in self.dong_dict:
                        all_dongs = self.dong_dict[sigungu]
                        # 구가 아닌 일반 동만 필터링
                        dong_list = [d for d in all_dongs if not d.endswith('구')]
                    
                    for dong_idx, dong_name in enumerate(dong_list):
                        if cancel_flag[0]:
                            break
                        
                        total_dongs_processed += 1
                        
                        # 전체 진행률
                        dong_progress = ((dong_idx + 1) / len(dong_list)) * (100 / total_sigungus)
                        total_progress = sigungu_progress + dong_progress
                        bar['value'] = total_progress
                        
                        info_label.config(text=f"현재 처리: {sigungu} {dong_name}")
                        stats_label.config(text=f"처리한 동: {total_dongs_processed}개, 추가된 단지: {total_apts_added}개 (추정)")
                        win.update_idletasks()
                        
                        try:
                            # 해당 동의 아파트 목록 조회
                            apt_list = self.get_apt_list_from_api(sigungu_code, dong_name)
                            
                            if apt_list:
                                # 각 동의 단지 추가 (기존 함수 활용)
                                # 추가된 개수를 반환하도록 수정 필요
                                before_count = len(self.monitored_apts)
                                
                                self.bulk_add_all_complexes_in_dong(
                                    sigungu_code=sigungu_code,
                                    dong=dong_name,
                                    sido=sido,
                                    sigungu=sigungu,
                                    apt_list=apt_list
                                )
                                
                                after_count = len(self.monitored_apts)
                                total_apts_added += (after_count - before_count)
                            
                            # API 호출 간격 조절
                            time.sleep(0.3)
                            
                        except Exception as e:
                            logging.error(f"{dong_name} 처리 중 오류: {str(e)}")
                            continue
                    
                    # 시군구 단위로 저장
                    self.save_monitored_apts()
                
                # 완료
                self.update_apt_tree()
                
                messagebox.showinfo("완료", 
                    f"{sido} 전체 단지 등록 완료\n"
                    f"처리한 시/군/구: {total_sigungus}개\n"
                    f"처리한 동: {total_dongs_processed}개\n"
                    f"추가된 단지: {total_apts_added}개")
                
            except Exception as e:
                messagebox.showerror("오류", f"처리 중 오류: {str(e)}")
            finally:
                try: win.destroy()
                except: pass
        
        # 별도 스레드에서 실행
        thread = threading.Thread(target=process, daemon=True)
        thread.start()

            
    def bulk_add_all_dongs_in_sigungu(self, sido, sigungu):
        """시/군/구 내 모든 동의 단지를 일괄 등록"""
        # 진행창
        win = tk.Toplevel(self.root)
        win.title(f"{sigungu} 전체 단지 등록 중...")
        win.geometry("600x250")
        win.transient(self.root)
        win.grab_set()
        
        ttk.Label(win, text=f"{sido} {sigungu}의 모든 동을 순회하며 단지를 등록합니다.", 
                 font=self.font_large).pack(pady=10)
        
        progress_label = ttk.Label(win, text="준비 중...")
        progress_label.pack(pady=5)
        
        bar = ttk.Progressbar(win, orient="horizontal", length=560, mode="determinate")
        bar.pack(pady=10, padx=20)
        
        info_label = ttk.Label(win, text="")
        info_label.pack(pady=5)
        
        stats_label = ttk.Label(win, text="")
        stats_label.pack(pady=5)
        
        cancel_flag = [False]
        
        def _cancel():
            cancel_flag[0] = True
            try: win.destroy()
            except: pass
        
        ttk.Button(win, text="중단", command=_cancel).pack(pady=10)
        
        def process():
            try:
                # 시군구 코드 얻기
                sigungu_code = None
                if sigungu in self.sigungu_to_full_info:
                    _, _, sigungu_code = self.sigungu_to_full_info[sigungu]
                
                if not sigungu_code:
                    messagebox.showerror("오류", "시군구 코드를 찾을 수 없습니다.")
                    win.destroy()
                    return
                
                # ===== 핵심 수정 부분: 동 콤보박스의 값을 직접 사용 =====
                dong_combo_values = self.dong_combobox['values']
                dong_list = []
                
                for item in dong_combo_values:
                    # "  └ " 형태의 하위 동 처리
                    if item.startswith("  └ "):
                        dong_name = item.replace("  └ ", "").strip()
                        dong_list.append(dong_name)
                    # 구(區)는 제외하고 일반 동만 추가
                    elif not item.endswith('구'):
                        dong_list.append(item)
                
                # 중복 제거
                dong_list = list(set(dong_list))
                
                if not dong_list:
                    messagebox.showinfo("알림", f"{sigungu}에 등록 가능한 동이 없습니다.")
                    win.destroy()
                    return
                
                total_dongs = len(dong_list)
                total_added = 0
                total_skipped = 0
                
                progress_label.config(text=f"총 {total_dongs}개 동 발견")
                win.update_idletasks()
                
                for dong_idx, dong_name in enumerate(dong_list):
                    if cancel_flag[0]:
                        break
                    
                    # 진행률 업데이트
                    progress = ((dong_idx + 1) / total_dongs) * 100
                    bar['value'] = progress
                    progress_label.config(text=f"동 진행: {dong_idx + 1}/{total_dongs}")
                    info_label.config(text=f"현재 처리 중: {dong_name}")
                    win.update_idletasks()
                    
                    try:
                        # 해당 동의 아파트 목록 조회
                        logging.info(f"처리 중: {sigungu} {dong_name}")
                        apt_list = self.get_apt_list_from_api(sigungu_code, dong_name)
                        
                        if apt_list:
                            # ⭐ silent=True로 호출 (팝업 없이 조용히 처리)
                            result = self.bulk_add_all_complexes_in_dong(
                                sigungu_code=sigungu_code,
                                dong=dong_name,
                                sido=sido,
                                sigungu=sigungu,
                                apt_list=apt_list,
                                silent=True  # ⭐ 추가!
                            )
                            
                            # 결과 누적
                            if result:
                                total_added += result['added']
                                total_skipped += result.get('skipped', 0)
                            
                            stats_label.config(text=f"추가된 단지: {total_added}개")
                            
                            stats_label.config(text=f"추가된 단지: {total_added}개")
                        else:
                            total_skipped += 1
                        
                        # API 호출 간격 조절
                        time.sleep(0.5)
                        
                    except Exception as e:
                        logging.error(f"{dong_name} 처리 중 오류: {str(e)}")
                        total_skipped += 1
                        continue
                    
                    # 10개 동마다 저장
                    if (dong_idx + 1) % 10 == 0:
                        self.save_monitored_apts()
                
                # 완료
                self.save_monitored_apts()
                self.update_apt_tree()
                
                messagebox.showinfo("완료", 
                    f"{sigungu} 전체 단지 등록 완료\n"
                    f"처리한 동: {total_dongs}개\n"
                    f"추가된 단지: {total_added}개\n"
                    f"건너뛴 동: {total_skipped}개")
                
            except Exception as e:
                messagebox.showerror("오류", f"처리 중 오류: {str(e)}")
                logging.error(f"bulk_add_all_dongs_in_sigungu 오류: {str(e)}")
            finally:
                try: win.destroy()
                except: pass
        
        # 별도 스레드에서 실행
        thread = threading.Thread(target=process, daemon=True)
        thread.start()
    

        
    def get_recent_areas_for_apt(self, sigungu_code, dong, apt_name, months=4):
        """최근 months 개월 캐시에서 해당 단지의 전용면적 목록 추출 (빠르고 가벼움)"""
        from collections import Counter
        areas = []
        now = datetime.now()
        for m in range(months):
            deal_ymd = (now - timedelta(days=30*m)).strftime("%Y%m")
            # 기축
            data_ex = self.get_cached_api_data(sigungu_code, deal_ymd, 'existing')
            for it in data_ex:
                if it.get('apt_name','').strip() == apt_name and it.get('dong','').strip() == dong:
                    try:
                        areas.append(str(int(float(it.get('area',0)))))
                    except:
                        pass
            # 신축/분양권
            data_new = self.get_cached_api_data(sigungu_code, deal_ymd, 'new')
            for it in data_new:
                if it.get('apt_name','').strip() == apt_name and it.get('dong','').strip() == dong:
                    try:
                        areas.append(str(int(float(it.get('area',0)))))
                    except:
                        pass
        if not areas:
            return []
        # 가장 많이 등장한 면적(대표면적) 우선 정렬
        c = Counter(areas)
        return [k for k,_ in c.most_common()]


    
    def bulk_add_all_complexes_in_dong(self, *, sigungu_code, dong, sido, sigungu, apt_list, silent=False):
        """해당 동에서 조회된 모든 '단지'를 대표면적 1개씩 골라 모니터링 목록에 추가
        
        Args:
            silent: True면 진행창과 완료 메시지를 표시하지 않음 (일괄 처리용)
        """
        import re
        
        # ⭐ silent 모드 처리
        if not silent:
            # 진행창 표시
            win = tk.Toplevel(self.root)
            win.title(f"{dong} 전체 단지 추가 중…")
            win.geometry("520x170")
            win.transient(self.root)
            win.grab_set()
            ttk.Label(win, text=f"'{sido} {sigungu} {dong}'의 조회된 모든 단지를 모니터링에 추가합니다.").pack(pady=(10,6))
            bar = ttk.Progressbar(win, orient="horizontal", length=480, mode="determinate")
            bar.pack(pady=6)
            info = ttk.Label(win, text="준비 중…")
            info.pack()
            cancel = [False]
            def _cancel():
                cancel[0] = True
                try: win.destroy()
                except: pass
            ttk.Button(win, text="중단", command=_cancel).pack(pady=(8,6))
        else:
            # silent 모드: 더미 객체 사용
            win = None
            bar = type('obj', (object,), {'__setitem__': lambda *args: None})()
            info = type('obj', (object,), {'config': lambda *args: None, 'pack': lambda *args: None})()
            cancel = [False]
        
        # apt_list 항목에서 단지명 파싱
        def parse_apt_name(s):
            # 형태 예: "[신축] 단지명 [도로/지번] (준공: 2019년)" 또는 "단지명 [도로/지번] (준공: ...)"
            txt = s.strip()
            if txt.startswith("[신축]"):
                txt = txt[5:].strip()
            if '[' in txt:
                return txt.split('[')[0].strip()
            # fallback
            return txt.split('(준공')[0].strip()
        
        # 중복 방지: (apt_name, area_int) 기준
        def _areas_equal(a, b, tol=0.15):
            try:
                fa = float(str(a).replace('㎡','').strip())
                fb = float(str(b).replace('㎡','').strip())
                return abs(fa - fb) <= tol
            except:
                return str(a).replace('㎡','').strip() == str(b).replace('㎡','').strip()
        
        def exists_in_monitor(apt_name, area):
            for apt in self.monitored_apts:
                if apt.get('apt_name') == apt_name and _areas_equal(apt.get('area',''), area):
                    return True
            return False
        
        total = len(apt_list)
        added = 0
        skipped = 0
        
        for i, row in enumerate(apt_list, start=1):
            if cancel[0]:
                break
            
            # 진행률 업데이트
            if not silent:
                bar['value'] = (i/total)*100
                info.config(text=f"[{i}/{total}] {parse_apt_name(row)} 처리 중…")
                win.update_idletasks()
            
            apt_name = parse_apt_name(row)
            
            # ▼▼ 추가: 연식(준공연도/분양) 우선 확보 (캐시 → 목록 문자열 파싱)
            build_year = self.resolve_build_year(
                sigungu_code=sigungu_code,
                dong=dong,
                apt_name=apt_name,
                months=12,
                fallback_text=row  # apt_list의 한 줄 원문
            )
            
            # 면적 후보(최근 4개월) 모두 사용
            areas = self.get_recent_areas_for_apt(sigungu_code, dong, apt_name, months=4)
            if not areas:
                skipped += 1
                continue
            
            # 중복 제거 및 숫자 오름차순 정렬
            def _area_to_float(a):
                try:
                    return float(str(a).replace('㎡','').strip())
                except:
                    return None
            
            uniq_areas = []
            seen = set()
            for a in areas:
                key = str(a).replace('㎡','').strip()
                if key not in seen:
                    seen.add(key)
                    uniq_areas.append(key)
            uniq_areas.sort(key=lambda x: (_area_to_float(x) is None, _area_to_float(x) or 0.0))
            
            # 모든 면적에 대해 추가 시도
            for area in uniq_areas:
                if exists_in_monitor(apt_name, area):
                    skipped += 1
                    continue
                
                apt_info = {
                    'apt_name': apt_name,
                    'jibun_addr': dong,
                    'area': area,              # 문자열 "84" 형태
                    'sido': sido,
                    'sigungu': sigungu,
                    'dong': dong,
                    'sigungu_code': sigungu_code,
                    'build_year': build_year   # ← 캐시/목록에서 확보한 연식 반영 ('', '분양', '2012' 등)
                }
                
                # 빠른 초기 수집: 최근 4개월만(조용히)
                trades = self.collect_apt_data_silent_with_cache(apt_info)
                if not trades:
                    skipped += 1
                    continue
                
                max_trade = max(trades, key=lambda x: x.get('price',0))
                apt_info.update({
                    'prev_max_price': 0,
                    'prev_max_date': '',
                    'prev_max_floor': '',
                    'prev_max_dong': '',
                    'last_max_price': max_trade.get('price',0),
                    'max_price_date': (max_trade.get('date') or datetime.now()).strftime('%Y-%m-%d') if isinstance(max_trade.get('date'), datetime) else str(max_trade.get('date')),
                    'max_price_floor': max_trade.get('floor',''),
                    'max_price_dong':  max_trade.get('dong','-'),
                    'last_update': datetime.now().strftime('%Y-%m-%d %H:%M'),
                    'trade_data': trades
                })
                
                self.monitored_apts.append(apt_info)
                added += 1
        
        self.save_monitored_apts()
        self.update_apt_tree()
        
        # ⭐ silent 모드가 아닐 때만 창 닫기 & 메시지 표시
        if not silent:
            try: 
                win.destroy()
            except: 
                pass
            messagebox.showinfo("완료", f"'{dong}' 단지 일괄 추가 완료\n추가: {added}개, 건너뜀: {skipped}개")
        
        # ⭐ silent 모드에서는 결과를 반환
        return {'added': added, 'skipped': skipped}



    
    def test_new_apt_api(self, sigungu_code, deal_ymd):
        """신축 API 응답 구조 테스트"""
        url = (f"https://apis.data.go.kr/1613000/RTMSDataSvcSilvTrade/getRTMSDataSvcSilvTrade"
               f"?serviceKey={self.service_key}"
               f"&LAWD_CD={sigungu_code}"
               f"&DEAL_YMD={deal_ymd}"
               f"&numOfRows=10")
        try:
            response = self.http.get(url, timeout=API_TIMEOUT)
            jitter_sleep()
            if response.status_code == 200:
                print(f"\n=== 신축 API 테스트 결과 ===")
                print(f"상태 코드: {response.status_code}")
                print(f"응답 샘플:\n{response.text[:1000]}")
                root = ET.fromstring(response.text)
                items = root.findall('.//item')
                if items:
                    print(f"\n총 {len(items)}개 항목 발견")
                    print("\n첫 번째 항목 필드:")
                    for child in items[0]:
                        print(f"  {child.tag}: {child.text}")
                else:
                    print("\n항목을 찾을 수 없습니다.")
            else:
                print(f"API 오류: {response.status_code}")
        except Exception as e:
            print(f"테스트 중 오류: {str(e)}")    


    def get_apt_list_from_api(self, sigungu_code, dong):
        """국토부 API에서 아파트 목록 가져오기 (기축 + 신축 통합)"""
        print(f"\n=== get_apt_list_from_api 호출 ===")
        print(f"시군구 코드: {sigungu_code}")
        print(f"동: {dong}")
        
        apt_info = {}
        current_date = datetime.now()
        
        # 최근 3개월만 검색
        for i in range(3):
            search_date = current_date - timedelta(days=30*i)
            deal_ymd = search_date.strftime("%Y%m")
            
            # 기축 아파트 API
            url_existing = (f"http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
                           f"?serviceKey={self.service_key}"
                           f"&LAWD_CD={sigungu_code}"
                           f"&DEAL_YMD={deal_ymd}"
                           f"&numOfRows=1000")
            
            # 신축 아파트(분양권) API - http로 변경
            url_new = (f"http://apis.data.go.kr/1613000/RTMSDataSvcSilvTrade/getRTMSDataSvcSilvTrade"
                       f"?serviceKey={self.service_key}"
                       f"&LAWD_CD={sigungu_code}"
                       f"&DEAL_YMD={deal_ymd}"
                       f"&numOfRows=1000")
            print(f"\n월 {deal_ymd} API 호출:")
            print(f"URL: {url_existing}")
            for url, apt_type in [(url_existing, "기축"), (url_new, "신축")]:
                try:
                    response = self.http.get(url, timeout=API_TIMEOUT)
                    jitter_sleep(200)
                    print(f"{apt_type} API 응답 상태: {response.status_code}")
                    if response.status_code == 200:
                        root = ET.fromstring(response.text)
                        items = root.findall('.//item')
                        print(f"{apt_type} 조회된 전체 항목 수: {len(items)}")
                        print(f"\n{apt_type} API 응답 샘플 (처음 5개):")
                        for idx, item in enumerate(items[:5]):
                            item_dong = item.findtext('umdNm', '').strip()
                            apt_name = item.findtext('aptNm', '').strip()
                            print(f"  [{idx+1}] 동: {item_dong}, 아파트: {apt_name}")
                        print()
                        dong_count = 0
                        for item in items:
                            item_dong = item.findtext('umdNm', '').strip()
                            if item_dong == dong:
                                dong_count += 1
                                apt_name = item.findtext('aptNm', '').strip()
                                if dong_count <= 3:
                                    print(f"  - {apt_name} ({item_dong})")
                                if apt_name and apt_name not in apt_info:
                                    jibun = item.findtext('jibun', '').strip()
                                    jibun_addr = f"{dong} {jibun}"
                                    road = item.findtext('roadName', '').strip()
                                    road_main = item.findtext('roadNameBonbun', '').strip()
                                    road_sub = item.findtext('roadNameBubun', '').strip()
                                    if road:
                                        road_addr = f"{road} {road_main}"
                                        if road_sub:
                                            road_addr += f"-{road_sub}"
                                    else:
                                        road_addr = jibun_addr
                                    build_year = item.findtext('buildYear', '').strip()
                                    if apt_type == "신축" and not build_year:
                                        build_year = "분양"
                                    apt_info[apt_name] = {
                                        'jibun_addr': jibun_addr,
                                        'road_addr': road_addr,
                                        'build_year': build_year,
                                        'type': apt_type
                                    }
                        print(f"'{dong}'의 {apt_type} 거래 수: {dong_count}")
                except Exception as e:
                    logging.error(f"{apt_type} API 호출 중 오류: {str(e)}")
                    print(f"{apt_type} API 호출 오류: {str(e)}")
                    continue
        print(f"\n수집된 아파트 총 {len(apt_info)}개")
        apt_list = []
        new_apts = []
        for apt_name, info in sorted(apt_info.items()):
            if info['type'] == '신축':
                if info['build_year'] and info['build_year'] != "분양":
                    apt_str = f"[신축] {apt_name} [{info['road_addr']} / {info['jibun_addr']}] (준공: {info['build_year']}년)"
                else:
                    apt_str = f"[신축] {apt_name} [{info['road_addr']} / {info['jibun_addr']}] (분양중)"
                new_apts.append(apt_str)
        existing_apts = [f"{apt_name} [{info['road_addr']} / {info['jibun_addr']}] (준공: {info['build_year']}년)" 
                        for apt_name, info in sorted(apt_info.items()) 
                        if info['type'] == '기축' and info['build_year']]
        no_year_apts = [f"{apt_name} [{info['road_addr']} / {info['jibun_addr']}]" 
                       for apt_name, info in sorted(apt_info.items()) 
                       if info['type'] == '기축' and not info['build_year']]
        logging.info(f"검색 결과 - 신축: {len(new_apts)}개, 기축: {len(existing_apts)}개")
        print(f"최종 결과 - 신축: {len(new_apts)}개, 기축: {len(existing_apts)}개, 연도없음: {len(no_year_apts)}개")
        return new_apts + existing_apts + no_year_apts
    
    def add_apt_to_monitored(self, apt_info):
        """모니터링 아파트 목록에 아파트 추가"""
        for apt in self.monitored_apts:
            if apt['apt_name'] == apt_info['apt_name'] and apt['area'] == apt_info['area']:
                messagebox.showinfo("알림", "이미 모니터링 중인 아파트입니다.")
                return
        self.status_var.set(f"{apt_info['apt_name']} 데이터 수집 중...")
        self.root.update_idletasks()
        try:
            trade_data = self.collect_apt_data_with_cache(apt_info)  # ← 변경!
            if trade_data:
                max_price_trade = max(trade_data, key=lambda x: x['price'])
                max_date_str = max_price_trade['date'].strftime('%Y-%m-%d')
                apt_info['prev_max_price'] = 0
                apt_info['prev_max_date'] = ''
                apt_info['prev_max_floor'] = ''
                apt_info['prev_max_dong'] = ''
                apt_info['last_max_price'] = max_price_trade['price']
                apt_info['max_price_date'] = max_date_str
                apt_info['max_price_floor'] = max_price_trade.get('floor', '')
                apt_info['max_price_dong'] = max_price_trade.get('dong', '-')
                apt_info['last_update'] = datetime.now().strftime('%Y-%m-%d %H:%M')
                apt_info['trade_data'] = trade_data
                self.monitored_apts.append(apt_info)
                self.save_monitored_apts()
                self.update_apt_tree()
                self.status_var.set(f"{apt_info['apt_name']} 모니터링 목록에 추가되었습니다.")
            else:
                messagebox.showinfo("알림", f"{apt_info['apt_name']}의 거래 내역이 없습니다.")
                self.status_var.set("거래 내역이 없습니다.")
        except Exception as e:
            messagebox.showerror("오류", f"데이터 수집 중 오류: {str(e)}")
            self.status_var.set("오류 발생")
    
    def collect_apt_data(self, apt_info):
        """모든 월을 꼼꼼하게 수집 (기축 + 신축), 병렬"""
        apt_name = apt_info['apt_name']
        target_area = float(apt_info['area'])
        sigungu_code = apt_info['sigungu_code']
        dong = apt_info['dong']
        all_trades = []
        current_date = datetime.now()
        max_months = 240  # 최대 20년
        
        progress_window = tk.Toplevel(self.root)
        progress_window.title("데이터 수집 중...")
        progress_window.geometry("400x150")
        progress_window.transient(self.root)
        progress_window.grab_set()
        
        ttk.Label(progress_window, text=f"{apt_name} ({target_area}㎡) 실거래 데이터를 수집 중입니다...", wraplength=350).pack(pady=10)
        progress_label = ttk.Label(progress_window, text="0% 완료")
        progress_label.pack(pady=5)
        progress_bar = ttk.Progressbar(progress_window, orient="horizontal", length=350, mode="determinate")
        progress_bar.pack(fill="x", padx=20, pady=10)
        
        cancel_flag = [False]
        def _cancel_collect():
            cancel_flag[0] = True
            try:
                progress_window.destroy()
            except:
                pass
        cancel_button = ttk.Button(progress_window, text="중단", command=_cancel_collect)
        cancel_button.pack(pady=5)
        progress_window.protocol("WM_DELETE_WINDOW", _cancel_collect)
        
        max_workers = min(10, (os.cpu_count() or 4) + 4)
        
        def collect_data():
            try:
                session = build_session()  # 재시도/백오프 적용 세션
                from threading import Lock
                results_lock = Lock()
                all_results = []
            
                def fetch_month_data(month_idx):
                    if cancel_flag[0]:
                        return None
                    search_date = current_date - timedelta(days=30 * month_idx)
                    deal_ymd = search_date.strftime("%Y%m")
                    url_existing = (f"http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
                                   f"?serviceKey={self.service_key}"
                                   f"&LAWD_CD={sigungu_code}"
                                   f"&DEAL_YMD={deal_ymd}"
                                   f"&numOfRows=1000")
                    url_new = (f"https://apis.data.go.kr/1613000/RTMSDataSvcSilvTrade/getRTMSDataSvcSilvTrade"
                               f"?serviceKey={self.service_key}"
                               f"&LAWD_CD={sigungu_code}"
                               f"&DEAL_YMD={deal_ymd}"
                               f"&numOfRows=1000")
                    month_trades = []
                    for url, api_type in [(url_existing, "existing"), (url_new, "new")]:
                        try:
                            response = session.get(url, timeout=API_TIMEOUT)
                            jitter_sleep(120)
                            if response.status_code != 200:
                                logging.error(f"API 응답 오류: {response.status_code} (월: {deal_ymd})")
                                continue
                            root = ET.fromstring(response.text)
                            items = root.findall('.//item')
                            for item in items:
                                item_apt = item.findtext('aptNm', '').strip()
                                item_dong = item.findtext('umdNm', '').strip()
                                if item_apt == apt_name and item_dong == dong:
                                    area = float(item.findtext('excluUseAr', '0'))
                                    if abs(area - target_area) <= 1:
                                        try:
                                            building_dong = item.findtext('kaptdong', '')
                                            if not building_dong:
                                                jibun = item.findtext('jibun', '').strip()
                                                if jibun and '동' in jibun:
                                                    dong_parts = jibun.split('동')
                                                    if len(dong_parts) > 0 and dong_parts[0].isdigit():
                                                        building_dong = dong_parts[0] + '동'
                                            building_dong = building_dong or '-'
                                            trade = {
                                                'date': datetime(
                                                    int(item.findtext('dealYear')),
                                                    int(item.findtext('dealMonth')),
                                                    int(item.findtext('dealDay', '1'))
                                                ),
                                                'price': int(item.findtext('dealAmount').replace(',', '')),
                                                'floor': int(item.findtext('floor', '0')),
                                                'area': area,
                                                'dong': building_dong,
                                                'type': api_type
                                            }
                                            month_trades.append(trade)
                                        except (ValueError, TypeError) as e:
                                            logging.error(f"데이터 처리 오류: {str(e)}")
                                            continue
                        except Exception as e:
                            logging.error(f"월 데이터 조회 중 오류 (월: {deal_ymd}): {str(e)}")
                    with results_lock:
                        all_results.append({'month_idx': month_idx, 'trades': month_trades})
                    return month_trades
                
                def update_progress():
                    completed = len(all_results)
                    progress = min(100, (completed / max_months) * 100)
                    total_trades = sum(len(result['trades']) for result in all_results)
                    progress_bar['value'] = progress
                    progress_label.config(text=f"{progress:.1f}% 완료 - {total_trades}건 수집됨 ({completed}/{max_months}개월)")
                    progress_window.update_idletasks()
                
                batch_size = 12
                for batch_start in range(0, max_months, batch_size):
                    if cancel_flag[0]:
                        break
                    batch_end = min(batch_start + batch_size, max_months)
                    current_batch = list(range(batch_start, batch_end))
                    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
                        futures = {executor.submit(fetch_month_data, month_idx): month_idx for month_idx in current_batch}
                        for future in concurrent.futures.as_completed(futures):
                            month_idx = futures[future]
                            try:
                                _ = future.result()
                                update_progress()
                            except Exception as e:
                                logging.error(f"월 {month_idx} 처리 중 오류: {str(e)}")
                    update_progress()
                    if cancel_flag[0]:
                        break
                    jitter_sleep(300)
                
                all_results.sort(key=lambda x: x['month_idx'])
                for result in all_results:
                    all_trades.extend(result['trades'])
                all_trades.sort(key=lambda x: x['date'])
                progress_bar['value'] = 100
                progress_label.config(text=f"100% 완료 - 총 {len(all_trades)}건 수집됨")
                progress_window.update_idletasks()
                progress_window.after(500, progress_window.destroy)
                    
            except Exception as e:
                messagebox.showerror("오류", f"데이터 수집 중 오류 발생: {str(e)}")
                try:
                    progress_window.destroy()
                except:
                    pass
        
        thread = threading.Thread(target=collect_data, daemon=True)
        thread.start()
        self.root.wait_window(progress_window)
        if cancel_flag[0]:
            return []
        if not all_trades:
            return []
        return sorted(all_trades, key=lambda x: x['date'])


    def collect_apt_data_with_cache(self, apt_info):
        """캐싱을 활용한 아파트 데이터 수집"""
        apt_name = apt_info['apt_name']
        target_area = float(apt_info['area'])
        sigungu_code = apt_info['sigungu_code']
        dong = apt_info['dong']
        all_trades = []
        current_date = datetime.now()
        max_months = 240
        
        # Progress 창
        progress_window = tk.Toplevel(self.root)
        progress_window.title("데이터 수집 중...")
        progress_window.geometry("500x200")
        progress_window.transient(self.root)
        progress_window.grab_set()
        
        ttk.Label(progress_window, text=f"{apt_name} ({target_area}㎡) 실거래 데이터를 수집 중입니다...", 
                 wraplength=450).pack(pady=10)
        progress_label = ttk.Label(progress_window, text="0% 완료")
        progress_label.pack(pady=5)
        
        # ★★★ 캐시 통계 라벨 추가 ★★★
        stats_label = ttk.Label(progress_window, text="API 호출: 0 / 캐시 히트: 0")
        stats_label.pack(pady=5)
        
        progress_bar = ttk.Progressbar(progress_window, orient="horizontal", 
                                      length=450, mode="determinate")
        progress_bar.pack(fill="x", padx=20, pady=10)
        
        cancel_flag = [False]
        def _cancel_collect():
            cancel_flag[0] = True
            try:
                progress_window.destroy()
            except:
                pass
        
        cancel_button = ttk.Button(progress_window, text="중단", command=_cancel_collect)
        cancel_button.pack(pady=5)
        progress_window.protocol("WM_DELETE_WINDOW", _cancel_collect)
        
        # 통계 저장
        initial_api_calls = self.api_call_count
        initial_cache_hits = self.cache_hit_count
        
        def collect_data():
            try:
                for month_idx in range(max_months):
                    if cancel_flag[0]:
                        break
                    
                    search_date = current_date - timedelta(days=30 * month_idx)
                    deal_ymd = search_date.strftime("%Y%m")
                    
                    # ★★★ 캐시 활용 ★★★
                    existing_data = self.get_cached_api_data(sigungu_code, deal_ymd, 'existing')
                    filtered_existing = self.filter_apt_data(existing_data, apt_name, dong, target_area)
                    all_trades.extend(filtered_existing)
                    
                    new_data = self.get_cached_api_data(sigungu_code, deal_ymd, 'new')
                    filtered_new = self.filter_apt_data(new_data, apt_name, dong, target_area)
                    all_trades.extend(filtered_new)
                    
                    # 진행률 업데이트
                    progress = min(100, ((month_idx + 1) / max_months) * 100)
                    progress_bar['value'] = progress
                    progress_label.config(text=f"{progress:.1f}% 완료 - {len(all_trades)}건 수집됨")
                    
                    # ★★★ 통계 업데이트 ★★★
                    api_calls = self.api_call_count - initial_api_calls
                    cache_hits = self.cache_hit_count - initial_cache_hits
                    stats_label.config(text=f"API 호출: {api_calls} / 캐시 히트: {cache_hits}")
                    progress_window.update_idletasks()
                    
                    # 최적화: 6개월간 거래 없으면 종료
                    if len(all_trades) > 0 and month_idx > 6:
                        recent_trades = [t for t in all_trades 
                                       if t['date'] > current_date - timedelta(days=180)]
                        if len(recent_trades) == 0:
                            break
                
                all_trades.sort(key=lambda x: x['date'])
                progress_bar['value'] = 100
                progress_label.config(text=f"100% 완료 - 총 {len(all_trades)}건 수집됨")
                
                # 최종 통계
                final_api_calls = self.api_call_count - initial_api_calls
                final_cache_hits = self.cache_hit_count - initial_cache_hits
                stats_label.config(text=f"완료! API 호출: {final_api_calls} / 캐시 히트: {final_cache_hits}")
                
                progress_window.update_idletasks()
                progress_window.after(500, progress_window.destroy)
                    
            except Exception as e:
                messagebox.showerror("오류", f"데이터 수집 중 오류 발생: {str(e)}")
                try:
                    progress_window.destroy()
                except:
                    pass
        
        thread = threading.Thread(target=collect_data, daemon=True)
        thread.start()
        self.root.wait_window(progress_window)
        
        if cancel_flag[0]:
            return []
        return all_trades
    
    def query_data(self, df, query_string):
        """조건에 맞는 데이터 추출"""
        try:
            filtered_df = df.query(query_string)
            return filtered_df
        except Exception as e:
            logging.error(f"데이터 쿼리 중 오류 발생: {str(e)}")
            return df

    # (주의) history 관련 함수들은 미사용/의존성 문제 가능. 필요한 경우 openpyxl 임포트/구현 보완.
    def load_history(self):
        history_list = []
        print(f"\n=== 히스토리 로드 시작 ===")
        print(f"히스토리 경로: {getattr(self, 'history_path', '(미설정)')}")
        if hasattr(self, 'history_path') and os.path.exists(self.history_path):
            try:
                all_files = os.listdir(self.history_path)
                print(f"히스토리 폴더 내 총 {len(all_files)}개 파일 발견")
                for file in all_files:
                    file_path = os.path.join(self.history_path, file)
                    if not os.path.exists(file_path):
                        print(f"파일 없음, 건너뜀: {file_path}")
                        continue
                    try:
                        if file.startswith('history_compare_'):
                            # 비교 파일 메타만 수집 (openpyxl 의존 제거)
                            history_list.append({
                                'file_path': file_path,
                                'apt_name': f"[비교] {file}",
                                'area': "비교분석",
                                'search_date': os.path.getmtime(file_path),
                                'max_trade': "비교분석",
                                'type': 'compare'
                            })
                        elif file.startswith('history_'):
                            # 단일 분석 파일 (메타만)
                            history_list.append({
                                'file_path': file_path,
                                'apt_name': file,
                                'area': "",
                                'search_date': os.path.getmtime(file_path),
                                'max_trade': "정보없음",
                                'type': 'single'
                            })
                    except Exception as e:
                        print(f"파일 처리 중 오류 ({file}): {str(e)}")
                        continue
                print(f"히스토리 로드 완료: {len(history_list)}개 항목")
            except Exception as e:
                print(f"히스토리 로드 중 오류: {str(e)}")
        else:
            print(f"히스토리 폴더가 없습니다: {getattr(self, 'history_path', '(미설정)')}")
        return sorted(history_list, key=lambda x: x['search_date'], reverse=True)

    def update_history_display(self):
        if not hasattr(self, 'history_tree'):
            return
        for item in self.history_tree.get_children():
            self.history_tree.delete(item)
        print(f"히스토리 목록 업데이트: {len(getattr(self, 'history_list', []))}개 항목")
        sorted_history = sorted(getattr(self, 'history_list', []), key=lambda x: x['search_date'], reverse=True)
        for item in sorted_history:
            search_date = datetime.fromtimestamp(item['search_date'])
            self.history_tree.insert("", "end", values=(
                search_date.strftime("%Y-%m-%d %H:%M"),
                item['apt_name'],
                item['area'],
                item['max_trade']
            ))
        if not getattr(self, 'history_list', []):
            print("히스토리 목록이 비어있습니다.")
        else:
            print(f("히스토리 표시 완료: {len(self.history_list)}개 항목"))
        self.root.update_idletasks()

    def delete_selected_history(self):
        if not hasattr(self, 'history_tree') or not hasattr(self, 'history_list'):
            messagebox.showinfo("알림", "히스토리 UI가 활성화되어 있지 않습니다.")
            return
        selection = self.history_tree.selection()
        if not selection:
            messagebox.showinfo("알림", "삭제할 항목을 선택해주세요.")
            return
        if not messagebox.askyesno("확인", "선택한 히스토리를 삭제하시겠습니까?\n(엑셀 파일과 그래프 이미지가 모두 삭제됩니다)"):
            return
        try:
            deleted_indices = []
            for item in selection:
                idx = self.history_tree.index(item)
                deleted_indices.append(idx)
                if idx < len(self.history_list):
                    history_item = self.history_list[idx]
                    file_path = history_item['file_path']
                    if os.path.exists(file_path):
                        try:
                            os.remove(file_path)
                        except Exception as e:
                            print(f"파일 삭제 실패: {file_path} - {str(e)}")
            deleted_indices.sort(reverse=True)
            for idx in deleted_indices:
                if idx < len(self.history_list):
                    del self.history_list[idx]
            for item in selection:
                self.history_tree.delete(item)
            self.update_history_display()
            messagebox.showinfo("알림", "선택한 히스토리가 삭제되었습니다.")
        except Exception as e:
            print(f"삭제 중 오류: {str(e)}")
            messagebox.showerror("오류", f"히스토리 삭제 중 오류가 발생했습니다: {str(e)}")
    
    def delete_selected_apt(self):
        """선택된 모니터링 아파트 삭제"""
        selection = self.apt_tree.selection()
        if not selection:
            messagebox.showinfo("알림", "삭제할 아파트를 선택해주세요.")
            return
        if messagebox.askyesno("확인", "선택한 아파트를 모니터링 목록에서 삭제하시겠습니까?"):
            deleted_count = 0
            for item_id in selection:
                item_values = self.apt_tree.item(item_id, "values")
                apt_name = item_values[0]
                area_str = item_values[2]
                import re
                area_value = re.search(r'(\d+(?:\.\d+)?)', area_str)
                if area_value:
                    area = area_value.group(1)
                else:
                    area = area_str.replace('㎡', '').strip()
                to_delete = []
                for idx, apt in enumerate(self.monitored_apts):
                    if apt.get('apt_name', '') == apt_name:
                        apt_area = str(apt.get('area', ''))
                        apt_area_clean = apt_area.replace('㎡', '').strip()
                        area_clean = area.replace('㎡', '').strip()
                        try:
                            if float(apt_area_clean) == float(area_clean):
                                to_delete.append(idx)
                        except ValueError:
                            if apt_area_clean == area_clean:
                                to_delete.append(idx)
                for idx in sorted(to_delete, reverse=True):
                    if 0 <= idx < len(self.monitored_apts):
                        del self.monitored_apts[idx]
                        deleted_count += 1
            if deleted_count > 0:
                self.save_monitored_apts()
                self.update_apt_tree()
                self.status_var.set(f"{deleted_count}개 아파트가 모니터링 목록에서 삭제되었습니다.")
            else:
                self.status_var.set("삭제할 항목을 찾을 수 없습니다.")
    
    def clear_all_apts(self):
        """모든 모니터링 아파트 삭제"""
        if not self.monitored_apts:
            messagebox.showinfo("알림", "모니터링 목록이 비어있습니다.")
            return
        if messagebox.askyesno("확인", "모니터링 목록을 모두 삭제하시겠습니까?"):
            self.monitored_apts = []
            self.save_monitored_apts()
            self.update_apt_tree()
            self.status_var.set("모니터링 목록이 모두 삭제되었습니다.")
    
    def recheck_max_prices(self):
        """모니터링 단지의 신고가 재검증 + 새로운 단지 자동 추가 + 연식 보정"""
        if not self.monitored_apts:
            # 모니터링 중인 단지가 없으면 전체 시군구 스캔
            self.scan_and_add_new_complexes()
            return
        
        # 이미 검증된 단지 추적용
        if not hasattr(self, 'rechecked_apts'):
            self.rechecked_apts = set()
        
        # 진행 창
        progress_window = tk.Toplevel(self.root)
        progress_window.title("신고가 재검증 및 신규 단지 탐색 중...")
        progress_window.geometry("600x300")
        progress_window.transient(self.root)
        progress_window.grab_set()
        
        ttk.Label(progress_window, text="신고가 재검증, 신규 단지 탐색, 연식 정보 보정 중...", 
                 wraplength=550).pack(pady=10)
        
        progress_label = ttk.Label(progress_window, text="준비 중...")
        progress_label.pack(pady=5)
        
        progress_bar = ttk.Progressbar(progress_window, orient="horizontal", 
                                      length=550, mode="determinate")
        progress_bar.pack(fill="x", padx=20, pady=10)
        
        result_label = ttk.Label(progress_window, text="")
        result_label.pack(pady=5)
        
        debug_text = tk.Text(progress_window, height=8, width=70)
        debug_text.pack(pady=5, padx=10)
        
        cancel_flag = [False]
        def _cancel():
            cancel_flag[0] = True
            try:
                progress_window.destroy()
            except:
                pass
        
        cancel_button = ttk.Button(progress_window, text="중단", command=_cancel)
        cancel_button.pack(pady=5)
        progress_window.protocol("WM_DELETE_WINDOW", _cancel)
        
        def process_recheck():
            try:
                # ========== 1단계: 기존 단지의 연식 보정 ==========
                debug_text.insert('end', "=== 1단계: 기존 단지 연식 보정 ===\n")
                debug_text.see('end')
                progress_window.update_idletasks()
                
                build_year_updated = 0
                for idx, apt in enumerate(self.monitored_apts):
                    if cancel_flag[0]:
                        break
                    
                    # 진행률 업데이트 (1단계는 전체의 30%)
                    progress = (idx / len(self.monitored_apts)) * 30
                    progress_bar['value'] = progress
                    progress_label.config(text=f"연식 보정 중: {idx + 1}/{len(self.monitored_apts)}")
                    progress_window.update_idletasks()
                    
                    # 연식이 비어있거나 유효하지 않은 경우
                    current_build_year = str(apt.get('build_year', '')).strip()
                    if not current_build_year or current_build_year == '':
                        debug_text.insert('end', f"  {apt.get('apt_name', '')} - 연식 정보 없음, 보정 시도...\n")
                        debug_text.see('end')
                        
                        # 연식 정보 보정 시도
                        resolved_year = self.resolve_build_year(
                            sigungu_code=apt.get('sigungu_code', ''),
                            dong=apt.get('dong', ''),
                            apt_name=apt.get('apt_name', ''),
                            months=24,  # 최근 24개월 데이터 검색
                            fallback_text=None
                        )
                        
                        if resolved_year:
                            apt['build_year'] = resolved_year
                            build_year_updated += 1
                            debug_text.insert('end', f"    ✓ 연식 보정 완료: {resolved_year}\n")
                            debug_text.see('end')
                        else:
                            debug_text.insert('end', f"    ✗ 연식 정보를 찾을 수 없음\n")
                            debug_text.see('end')
                
                debug_text.insert('end', f"\n연식 보정 완료: {build_year_updated}개 단지\n\n")
                debug_text.see('end')
                
                # ========== 2단계: 기존 단지들의 시군구 코드 수집 ==========
                debug_text.insert('end', "=== 2단계: 시군구 코드 수집 ===\n")
                sigungu_codes = set()
                for apt in self.monitored_apts:
                    if 'sigungu_code' in apt:
                        sigungu_codes.add(apt['sigungu_code'])
                
                debug_text.insert('end', f"모니터링 중인 시군구 코드: {sigungu_codes}\n\n")
                debug_text.see('end')
                progress_window.update_idletasks()
                
                # ========== 3단계: 각 시군구의 최근 거래 데이터에서 새로운 단지 탐색 ==========
                debug_text.insert('end', "=== 3단계: 신규 단지 탐색 및 신고가 검증 ===\n")
                new_complexes = []
                total_checked = 0
                corrected_count = 0
                
                current_date = datetime.now()
               
                start_year = 2019
                months_to_check = (current_date.year - start_year) * 12 + current_date.month
                
                for sigungu_idx, sigungu_code in enumerate(sigungu_codes):
                    if cancel_flag[0]:
                        break
                    
                    debug_text.insert('end', f"\n시군구 {sigungu_code} 스캔 중...\n")
                    debug_text.see('end')
                    
                    # 해당 시군구의 모든 거래 데이터 수집
                    all_apt_info = {}
                    
                    for month_idx in range(months_to_check):
                        if cancel_flag[0]:
                            break
                        
                        search_date = current_date - timedelta(days=30 * month_idx)
                        deal_ymd = search_date.strftime("%Y%m")
                        
                        # 캐시 활용하여 데이터 가져오기
                        existing_data = self.get_cached_api_data(sigungu_code, deal_ymd, 'existing')
                        new_data = self.get_cached_api_data(sigungu_code, deal_ymd, 'new')
                        
                        # 각 데이터에서 단지 정보 추출
                        for item in existing_data + new_data:
                            apt_name = item.get('apt_name', '').strip()
                            dong = item.get('dong', '').strip()
                            area = item.get('area', 0)
                            
                            if not apt_name or not dong:
                                continue
                            
                            # 고유 키 생성
                            apt_key = f"{apt_name}_{dong}_{int(area)}"
                            
                            if apt_key not in all_apt_info:
                                all_apt_info[apt_key] = {
                                    'apt_name': apt_name,
                                    'dong': dong,
                                    'area': str(int(area)),
                                    'sigungu_code': sigungu_code,
                                    'trades': [],
                                    'build_year': item.get('build_year', '')  # ★ API에서 연식 정보도 수집
                                }
                            
                            # 거래 정보 추가
                            try:
                                trade = {
                                    'date': datetime(item['year'], item['month'], item.get('day', 1)),
                                    'price': item['price'],
                                    'floor': item.get('floor', 0),
                                    'building_dong': item.get('kaptdong', '-')
                                }
                                all_apt_info[apt_key]['trades'].append(trade)
                            except:
                                continue
                    
                    debug_text.insert('end', f"  발견된 단지: {len(all_apt_info)}개\n")
                    
                    # 3단계: 기존 모니터링 목록과 비교
                    for apt_key, apt_data in all_apt_info.items():
                        if cancel_flag[0]:
                            break
                        
                        apt_name = apt_data['apt_name']
                        dong = apt_data['dong']
                        area = apt_data['area']
                        
                        # 기존 목록에 있는지 확인
                        exists = False
                        existing_apt = None
                        for monitored in self.monitored_apts:
                            if (monitored.get('apt_name') == apt_name and 
                                monitored.get('dong') == dong and
                                str(monitored.get('area', '')).replace('㎡', '').strip() == area):
                                exists = True
                                existing_apt = monitored
                                break
                        
                        if not exists and apt_data['trades']:
                            # 새로운 단지 발견
                            max_trade = max(apt_data['trades'], key=lambda x: x['price'])
                            
                            # sido, sigungu 정보 찾기
                            sido = ''
                            sigungu = ''
                            for apt in self.monitored_apts:
                                if apt.get('sigungu_code') == sigungu_code:
                                    sido = apt.get('sido', '')
                                    sigungu = apt.get('sigungu', '')
                                    break
                            
                            # ★ 연식 정보가 없으면 추가 조회
                            build_year = apt_data.get('build_year', '').strip()
                            if not build_year:
                                build_year = self.resolve_build_year(
                                    sigungu_code=sigungu_code,
                                    dong=dong,
                                    apt_name=apt_name,
                                    months=12
                                )
                            
                            new_apt = {
                                'apt_name': apt_name,
                                'area': area,
                                'sido': sido,
                                'sigungu': sigungu,
                                'dong': dong,
                                'sigungu_code': sigungu_code,
                                'prev_max_price': 0,
                                'prev_max_date': '',
                                'prev_max_floor': '',
                                'prev_max_dong': '',
                                'last_max_price': max_trade['price'],
                                'max_price_date': max_trade['date'].strftime('%Y-%m-%d'),
                                'max_price_floor': max_trade.get('floor', ''),
                                'max_price_dong': max_trade.get('building_dong', '-'),
                                'last_update': datetime.now().strftime('%Y-%m-%d %H:%M'),
                                'trade_data': apt_data['trades'],
                                'build_year': build_year  # ★ 연식 정보 포함
                            }
                            
                            new_complexes.append(new_apt)
                            year_info = f" (연식: {build_year})" if build_year else " (연식 정보 없음)"
                            debug_text.insert('end', f"  [신규] {apt_name} {area}㎡{year_info} - 최고가: {max_trade['price']:,}만원\n")
                            
                        elif exists and existing_apt and apt_data['trades']:
                            # 기존 단지의 신고가 체크
                            max_trade = max(apt_data['trades'], key=lambda x: x['price'])
                            if max_trade['price'] > existing_apt.get('last_max_price', 0):
                                # 신고가 갱신
                                existing_apt['prev_max_price'] = existing_apt.get('last_max_price', 0)
                                existing_apt['prev_max_date'] = existing_apt.get('max_price_date', '')
                                existing_apt['prev_max_floor'] = existing_apt.get('max_price_floor', '')
                                existing_apt['prev_max_dong'] = existing_apt.get('max_price_dong', '')
                                
                                existing_apt['last_max_price'] = max_trade['price']
                                existing_apt['max_price_date'] = max_trade['date'].strftime('%Y-%m-%d')
                                existing_apt['max_price_floor'] = max_trade.get('floor', '')
                                existing_apt['max_price_dong'] = max_trade.get('building_dong', '-')
                                existing_apt['last_update'] = datetime.now().strftime('%Y-%m-%d %H:%M')
                                
                                corrected_count += 1
                                debug_text.insert('end', f"  [신고가] {apt_name} {area}㎡ - {max_trade['price']:,}만원\n")
                        
                        total_checked += 1
                        # 진행률 (30% 이후 ~ 100%)
                        progress = 30 + ((sigungu_idx + (total_checked / len(all_apt_info))) / len(sigungu_codes)) * 70
                        progress_bar['value'] = min(progress, 100)
                        result_label.config(text=f"연식 보정: {build_year_updated}개 / 신규: {len(new_complexes)}개 / 신고가: {corrected_count}개")
                        progress_window.update_idletasks()
                    
                    debug_text.see('end')
                
                # 4단계: 새로운 단지들을 모니터링 목록에 추가
                if new_complexes:
                    self.monitored_apts.extend(new_complexes)
                    debug_text.insert('end', f"\n총 {len(new_complexes)}개 신규 단지 추가됨\n")
                
                # 저장 및 화면 갱신
                self.save_monitored_apts()
                self.update_apt_tree()
                
                progress_bar['value'] = 100
                progress_label.config(text="완료!")
                
                messagebox.showinfo("완료", 
                    f"신고가 재검증이 완료되었습니다.\n"
                    f"연식 보정: {build_year_updated}개\n"
                    f"신규 단지: {len(new_complexes)}개 추가\n"
                    f"신고가 갱신: {corrected_count}개")
                
                progress_window.destroy()
                
            except Exception as e:
                messagebox.showerror("오류", f"재검증 중 오류 발생: {str(e)}")
                debug_text.insert('end', f"\n오류: {str(e)}\n")
                try:
                    progress_window.destroy()
                except:
                    pass
        
        thread = threading.Thread(target=process_recheck, daemon=True)
        thread.start()
    
    def update_all_data(self):
        """모든 모니터링 아파트 데이터 갱신"""
        if not self.monitored_apts:
            messagebox.showinfo("알림", "모니터링 목록이 비어있습니다.")
            return
        
        is_auto_update = threading.current_thread() != threading.main_thread()
        if not is_auto_update:
            self.status_var.set("데이터 갱신 중...")
            self.root.update_idletasks()
        
        new_max_found = False
        apt_with_new_max = []
        
        for i, apt_info in enumerate(self.monitored_apts):
            try:
                if not is_auto_update:
                    self.status_var.set(f"({i+1}/{len(self.monitored_apts)}) {apt_info['apt_name']} 데이터 갱신 중...")
                    self.root.update_idletasks()
                
                old_max_price = apt_info.get('last_max_price', 0)
                old_max_date = apt_info.get('max_price_date', '')
                old_max_dong = apt_info.get('max_price_dong', '')
                old_max_floor = apt_info.get('max_price_floor', '')
                
                trade_data = self.collect_apt_data_silent_with_cache(apt_info)
                
                if trade_data:
                    new_max_price_trade = max(trade_data, key=lambda x: x['price'])
                    
                    # date_str을 먼저 정의
                    date_str = ''
                    if isinstance(new_max_price_trade.get('date'), datetime):
                        date_str = new_max_price_trade['date'].strftime('%Y-%m-%d')
                    else:
                        date_str = str(new_max_price_trade.get('date', ''))
                    
                    if new_max_price_trade['price'] > old_max_price:
                        new_max_found = True
                        
                        # 신고가 정보 딕셔너리 생성 - build_year 확실히 포함
                        # 신고가 정보 딕셔너리 생성 - build_year 확실히 포함
                        build_year = str(apt_info.get('build_year', ''))
                        current_year = datetime.now().year
                        is_young = False
                        
                        # 10년 이하 판별
                        if build_year == '분양':
                            is_young = True
                        elif build_year and build_year.isdigit():
                            if current_year - int(build_year) <= 10:
                                is_young = True
                        
                        new_max_info = {
                            'apt_name': apt_info['apt_name'],
                            'area': apt_info['area'],
                            'old_price': old_max_price,
                            'new_price': new_max_price_trade['price'],
                            'date': date_str,
                            'floor': new_max_price_trade.get('floor', ''),
                            'dong': new_max_price_trade.get('dong', '-'),
                            'sido': apt_info.get('sido', ''),
                            'sigungu': apt_info.get('sigungu', ''),
                            'location_dong': apt_info.get('dong', ''),
                            'build_year': build_year,
                            'is_young': is_young  # 추가
                        }
                        
                        apt_with_new_max.append(new_max_info)
                        
                        # 디버깅을 위한 출력
                        logging.info(f"신고가 발견: {apt_info['apt_name']}, 연식: {apt_info.get('build_year', '없음')}")
                        print(f"신고가 리스트 추가: {new_max_info['apt_name']}, 연식: {new_max_info['build_year']}")
                    
                    # 기존 아파트 정보 업데이트
                    # date_str이 정의되지 않았을 경우를 대비
                    if 'date_str' not in locals():
                        date_str = ''
                        if isinstance(new_max_price_trade.get('date'), datetime):
                            date_str = new_max_price_trade['date'].strftime('%Y-%m-%d')
                        else:
                            date_str = str(new_max_price_trade.get('date', ''))
                    
                    # 기존 아파트 정보 업데이트
                    apt_info['prev_max_price'] = old_max_price
                    apt_info['prev_max_date'] = old_max_date
                    apt_info['prev_max_dong'] = old_max_dong
                    apt_info['prev_max_floor'] = old_max_floor
                    
                    apt_info['last_max_price'] = new_max_price_trade['price']
                    apt_info['max_price_date'] = date_str
                    apt_info['max_price_floor'] = new_max_price_trade.get('floor', '')
                    apt_info['max_price_dong'] = new_max_price_trade.get('dong', '-')
                    apt_info['last_update'] = datetime.now().strftime('%Y-%m-%d %H:%M')
                    apt_info['trade_data'] = trade_data
                    
            except Exception as e:
                logging.error(f"{apt_info['apt_name']} 데이터 갱신 중 오류: {str(e)}")
                continue
        
        self.save_monitored_apts()
        self.update_apt_tree()
        
        if new_max_found:
            # 전체 모니터링 단지 기준 지역구별 순위 계산
            regional_rankings = self.calculate_regional_rankings()
            
            # 신고가 단지에 순위 정보 추가
            for apt in apt_with_new_max:
                sido = apt.get('sido', '')
                sigungu = apt.get('sigungu', '').split('(')[0] if apt.get('sigungu') else ''
                region_key = f"{sido} {sigungu}"
                apt_name = apt.get('apt_name', '')
                
                if region_key in regional_rankings:
                    # 84타입 순위
                    if apt_name in regional_rankings[region_key]['84']:
                        apt['region_84_rank'] = regional_rankings[region_key]['84'][apt_name]
                        apt['region_name'] = sigungu
                    
                    # 59타입 순위
                    if apt_name in regional_rankings[region_key]['59']:
                        apt['region_59_rank'] = regional_rankings[region_key]['59'][apt_name]
                        apt['region_name'] = sigungu
            
            # HTML 생성 전 연식 정보 최종 확인
            print("\n=== 신고가 발견 목록 최종 확인 ===")
            for apt in apt_with_new_max:
                rank_info = []
                if apt.get('region_84_rank'):
                    rank_info.append(f"84타입 {apt.get('region_84_rank')}위")
                if apt.get('region_59_rank'):
                    rank_info.append(f"59타입 {apt.get('region_59_rank')}위")
                rank_str = f", 지역순위={', '.join(rank_info)}" if rank_info else ""
                print(f"- {apt['apt_name']}: 연식={apt.get('build_year', '없음')}, 가격={apt['new_price']:,}만원{rank_str}")
            print("================================\n")
            
            self.show_new_max_notification(apt_with_new_max)
        
        update_time = datetime.now().strftime('%Y-%m-%d %H:%M')
        if is_auto_update:
            logging.info(f"자동 업데이트 완료: {update_time}")
        else:
            self.status_var.set(f"데이터 갱신 완료: {update_time}")
        
    def collect_apt_data_silent_with_cache(self, apt_info):
        """캐싱을 활용한 조용한 데이터 수집"""
        apt_name = apt_info['apt_name']
        target_area = float(apt_info['area'])
        sigungu_code = apt_info['sigungu_code']
        dong = apt_info['dong']
        trades = []
        current_date = datetime.now()
        max_months = 4
        
        try:
            for month in range(max_months):
                search_date = current_date - timedelta(days=30 * month)
                deal_ymd = search_date.strftime("%Y%m")
                
                # ★★★ 캐시 활용 ★★★
                existing_data = self.get_cached_api_data(sigungu_code, deal_ymd, 'existing')
                new_data = self.get_cached_api_data(sigungu_code, deal_ymd, 'new')
                
                trades.extend(self.filter_apt_data(existing_data, apt_name, dong, target_area))
                trades.extend(self.filter_apt_data(new_data, apt_name, dong, target_area))
            
            # 기존 데이터와 병합
            if 'trade_data' in apt_info and apt_info['trade_data']:
                existing_trades = apt_info['trade_data']
                existing_keys = set()
                
                for trade in existing_trades:
                    if 'date' in trade and isinstance(trade['date'], datetime):
                        key = (trade['date'].year, trade['date'].month, trade['date'].day, 
                              trade.get('floor', 0), trade.get('price', 0))
                        existing_keys.add(key)
                
                for trade in trades:
                    trade_key = (trade['date'].year, trade['date'].month, trade['date'].day,
                                trade.get('floor', 0), trade.get('price', 0))
                    if trade_key not in existing_keys:
                        existing_trades.append(trade)
                
                trades = existing_trades
            
            return sorted(trades, key=lambda x: x.get('date', datetime.min))
        except Exception as e:
            logging.error(f"데이터 수집 중 오류: {str(e)}")
            return apt_info.get('trade_data', [])

    def build_notification_html(self, apt_list):
        """신고가 알림용 HTML (날짜 필터 + 상승률 필터 포함)"""
        from html import escape
        from collections import Counter
        import re
        
        now = datetime.now().strftime('%Y-%m-%d %H:%M')
        current_year = datetime.now().year
        
        # 신고가 높은 순으로 정렬
        apt_list = sorted(apt_list, key=lambda x: x.get('new_price', 0), reverse=True)
        total = len(apt_list)
        
        # 지역별 건수 집계 및 연식별 카운트
        region_counter = Counter()
        young_count = 0  # 10년 이하
        very_young_count = 0  # 5년 이하
        old_count = 0  # 2000년대 미만 (1999년 이하)
        
        for apt in apt_list:
            sido = apt.get('sido', '')
            sigungu = apt.get('sigungu', '')
            if sido and sigungu:
                sigungu_clean = sigungu.split('(')[0] if '(' in sigungu else sigungu
                region_key = f"{sido} {sigungu_clean}"
                region_counter[region_key] += 1
            
            # build_year 처리
            build_year = apt.get('build_year', '')
            if build_year == '분양':
                young_count += 1
                very_young_count += 1
                apt['is_young'] = True
                apt['is_very_young'] = True
                apt['is_old'] = False
            elif build_year:
                try:
                    year = int(build_year)
                    years_old = current_year - year
                    
                    # 2000년 미만 체크
                    if year < 2000:
                        old_count += 1
                        apt['is_old'] = True
                    else:
                        apt['is_old'] = False
                    
                    # 기존 연식 체크
                    if years_old <= 5:
                        young_count += 1
                        very_young_count += 1
                        apt['is_young'] = True
                        apt['is_very_young'] = True
                    elif years_old <= 10:
                        young_count += 1
                        apt['is_young'] = True
                        apt['is_very_young'] = False
                    else:
                        apt['is_young'] = False
                        apt['is_very_young'] = False
                except:
                    apt['is_young'] = False
                    apt['is_very_young'] = False
                    apt['is_old'] = False
            else:
                apt['is_young'] = False
                apt['is_very_young'] = False
                apt['is_old'] = False
        
        sorted_regions = sorted(region_counter.items(), key=lambda x: x[1], reverse=True)
        
        # 카드 HTML 생성
        cards_html = []
        for apt in apt_list:
            name = escape(str(apt.get('apt_name', '')))
            area = escape(str(apt.get('area', '')))
            old_price = apt.get('old_price', 0) or 0
            new_price = apt.get('new_price', 0) or 0
            date = escape(str(apt.get('date', '')))
            floor = escape(str(apt.get('floor', '')))
            dong = escape(str(apt.get('dong', '-')))
            build_year = escape(str(apt.get('build_year', '')))
            
            # 지역 정보
            sido = escape(str(apt.get('sido', '')))
            sigungu = escape(str(apt.get('sigungu', '')))
            location_dong = escape(str(apt.get('location_dong', '')))
            location = f"{sido} {sigungu} {location_dong}"
            
            # 가격 정보
            inc = new_price - old_price if old_price else 0
            pct = f"{(inc/old_price*100):.1f}%" if old_price else "-"
            old_str = f"{old_price:,}만원" if old_price else "-"
            new_str = f"{new_price:,}만원"
            inc_str = f"{inc:,}만원" if old_price else "-"
            
            # 연식 표시 설정
            year_badge = ""
            data_year = ""
            if build_year == '분양':
                year_badge = "<span class='year-badge new'>분양</span>"
                data_year = str(current_year)
            elif build_year:
                try:
                    year_int = int(build_year)
                    if year_int < 2000:
                        year_badge = f"<span class='year-badge old'>{build_year}년</span>"
                    elif current_year - year_int <= 5:
                        year_badge = f"<span class='year-badge very-young'>{build_year}년</span>"
                    elif current_year - year_int <= 10:
                        year_badge = f"<span class='year-badge young'>{build_year}년</span>"
                    else:
                        year_badge = f"<span class='year-badge'>{build_year}년</span>"
                    data_year = build_year
                except:
                    year_badge = f"<span class='year-badge'>{build_year}년</span>"
                    data_year = build_year
            
            is_young = "1" if apt.get('is_young', False) else "0"
            is_very_young = "1" if apt.get('is_very_young', False) else "0"
            is_old = "1" if apt.get('is_old', False) else "0"
            
            card = f"""
            <section class="card" data-region="{sido} {sigungu.split('(')[0] if '(' in sigungu else sigungu}" 
                     data-build-year="{data_year}" data-young="{is_young}" 
                     data-very-young="{is_very_young}" data-old="{is_old}">
              <h3>{name} {year_badge} <span class="tag">{area}㎡</span></h3>
              <div class="card-sub">📍 {location}</div>
              <div class="grid">
                <div class="k">이번 실거래</div>
                <div><span class="highlight">{new_str}</span> ({date}{f' | {floor}층' if floor else ''})</div>
                <div class="k">직전 최고가</div>
                <div>{old_str}</div>
                <div class="k">변화</div>
                <div class="rise">{inc_str} 상승 ({pct})</div>
                <div class="k">특이사항</div>
                <div>
                    {f'<span class="badge-rank rank-84">{apt.get("region_name", "")} 84타입 NO.{apt.get("region_84_rank", "")} 단지</span>' if apt.get("region_84_rank") else ''}
                    {f'<span class="badge-rank rank-59">{apt.get("region_name", "")} 59타입 NO.{apt.get("region_59_rank", "")} 단지</span>' if apt.get("region_59_rank") else ''}
                </div>
              </div>
            </section>
            """
            cards_html.append(card)
        
        cards = "\n".join(cards_html)
        
        html_content = f"""<!DOCTYPE html>
        <html lang="ko">
        <head>
        <meta charset="utf-8"/>
        <meta content="width=device-width, initial-scale=1" name="viewport"/>
        <title>부태리 신고가 알림 - {escape(now)}</title>
        <style>
          :root {{
            --bg:#F2F2F7; --fg:#111; --sub:#6b7280; --card:#fff; --bd:#e5e7eb;
            --primary:#007AFF; --primary-dark:#0051D2; --accent:#ff3b30;
          }}
          *{{ box-sizing:border-box }}
          body{{
            margin:0; font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Noto Sans KR","Apple SD Gothic Neo","Malgun Gothic",sans-serif;
            color:var(--fg); background:var(--bg);
          }}
        
          .page{{ max-width:960px; width:92vw; margin:24px auto 80px }}
          .header{{
            background:linear-gradient(135deg,var(--primary),#63A4FF);
            color:#fff; border-radius:16px; padding:20px 20px 16px;
            box-shadow:0 6px 18px rgba(0,0,0,.08);
          }}
          .header h1{{ margin:0 0 8px; font-size:22px; font-weight:800; letter-spacing:-.2px }}
          .header .sub{{ margin:0; opacity:.95; font-size:14px }}
        
          /* 칩 스타일 */
          .chips{{ display:flex; flex-wrap:wrap; gap:8px; margin:12px 0 0 }}
          .chip{{
            display:inline-flex; align-items:center; gap:6px; padding:8px 12px;
            border:1px solid var(--bd); border-radius:999px; background:#fff; color:#111; font-size:13px;
            box-shadow:0 1px 1.5px rgba(0,0,0,.04); cursor:pointer;
            transition: all 0.2s ease;
          }}
          .chip:hover{{ background:#f3f4f6 }}
          .chip.active{{ background:#1976d2; color:#fff; border-color:#1976d2 }}
        
          .region-chips{{ display:flex; flex-wrap:wrap; gap:8px; margin:10px 0 0 }}
          .region-chip{{
            display:inline-flex; align-items:center; gap:6px; padding:8px 12px;
            border:1px solid var(--bd); border-radius:999px; background:#fff; color:#111; font-size:13px;
            box-shadow:0 1px 1.5px rgba(0,0,0,.04); cursor:pointer;
            transition: all 0.2s ease;
          }}
          .region-chip:hover{{ background:#f3f4f6 }}
          .region-chip.active{{ background:#1976d2; color:#fff; border-color:#1976d2 }}
        
          /* 카드 그리드 */
          .cards{{ display:grid; grid-template-columns:repeat(12,1fr); gap:12px; margin-top:16px }}
          .card{{ 
            grid-column:span 12; background:var(--card); border:1px solid var(--bd); 
            border-radius:14px; padding:14px; box-shadow:0 2px 12px rgba(0,0,0,.04);
            transition: transform 0.2s ease, box-shadow 0.2s ease;
          }}
          .card:hover{{ transform: translateY(-2px); box-shadow:0 4px 20px rgba(0,0,0,.08) }}
          @media(min-width:720px){{ .card{{ grid-column:span 6 }} }}
          .card.hidden{{ display:none }}
          .card h3{{ margin:0 0 4px; font-size:16px; display:flex; align-items:center; gap:6px }}
          
          /* 연식 배지 스타일 */
          .year-badge {{
            display: inline-flex;
            align-items: center;
            padding: 2px 8px;
            border-radius: 6px;
            background: #e5e7eb;
            color: #4b5563;
            font-size: 12px;
            font-weight: normal;
          }}
          .year-badge.old {{  /* 2000년 미만 */
            background: #f3e8ff;
            color: #6b21a8;
            font-weight: 600;
          }}
          .year-badge.very-young {{  /* 5년 이하 */
            background: #fce7f3;
            color: #be185d;
            font-weight: 600;
          }}
          .year-badge.young {{  /* 10년 이하 */
            background: #dbeafe;
            color: #1e40af;
            font-weight: 600;
          }}
          .year-badge.new {{  /* 분양 */
            background: #fef3c7;
            color: #92400e;
            font-weight: 600;
          }}
          
          .tag{{ font-size:12px; padding:2px 8px; border-radius:999px; border:1px solid var(--bd); background:#fff; color:#333 }}
          .card .card-sub{{ color:var(--sub); font-size:13px; margin-bottom:6px }}
          .grid{{ display:grid; grid-template-columns:110px 1fr; gap:10px; color:#111; font-size:13px }}
          .grid .k{{ color:var(--sub) }}
          .highlight{{ color:var(--primary-dark); font-weight:700 }}
          .rise{{ color:#e11d48; font-weight:700 }}
          .badge-rank {{
            display:inline-flex; align-items:center; gap:4px; padding:4px 10px;
            margin-left:6px;
            border-radius:999px; color:#fff; font-size:12px; font-weight:600;
            box-shadow:0 2px 4px rgba(0,0,0,0.2);
            animation: pulse 2s infinite;
          }}
          .badge-rank.rank-84 {{
            background:linear-gradient(135deg, #fbbf24, #f59e0b);
            border:1px solid #f59e0b;
          }}
          .badge-rank.rank-59 {{
            background:linear-gradient(135deg, #8b5cf6, #7c3aed);
            border:1px solid #7c3aed;
          }}
          @keyframes pulse {{
            0%, 100% {{ opacity: 1; transform: scale(1); }}
            50% {{ opacity: 0.9; transform: scale(1.02); }}
          }}
    
          /* 툴바 스타일 */
          .toolbar{{ display:flex; gap:8px; align-items:center; margin:8px 0 4px; flex-wrap:wrap }}
          .toolbar input[type="date"], .toolbar input[type="number"]{{
            padding:10px 12px; border:1px solid var(--bd); border-radius:10px; background:#fff; font-size:13px; width:120px;
          }}
          .toolbar .btn{{
            display:inline-flex; align-items:center; padding:10px 12px; border:1px solid var(--bd);
            border-radius:999px; background:#fff; font-size:13px; cursor:pointer; transition: all 0.2s ease;
          }}
          .toolbar .btn:hover{{ background:#f3f4f6 }}
        
          footer{{ margin:24px 0 0; color:#6b7280; font-size:12px; text-align:center }}
        </style>
        </head>
        <body>
          <div class="page">
            <header class="header">
              <h1>부태리 신고가 알림</h1>
              <p class="sub">{escape(now)} 기준 · 총 {total}개 아파트</p>
        
              <!-- 상단 칩: 연식별 필터 -->
              <div class="chips">
                <span class="chip" id="chip-old">2000년대 미만 {old_count}건</span>
                <span class="chip" id="chip-very-young">입주 5년 이하 {very_young_count}건</span>
                <span class="chip" id="chip-young">입주 10년 이하 {young_count}건</span>
              </div>
        
              <!-- 지역 칩 -->
              <div class="region-chips"></div>
            </header>
    
            <!-- 상승률 필터 툴바 (첫 번째 줄) -->
            <div class="toolbar">
              <label style="font-size:13px;color:var(--sub)">상승률(%)</label>
              <input id="rateMin" type="number" step="0.1" min="0" placeholder="최소"/>
              <span style="color:var(--sub)">~</span>
              <input id="rateMax" type="number" step="0.1" min="0" placeholder="최대"/>
              <button class="btn" id="btnRate5">≥ 5%</button>
              <button class="btn" id="btnRate10">≥ 10%</button>
              <button class="btn" id="btnRateReset">상승률 초기화</button>
            </div>
    
            <!-- 날짜 필터 툴바 (두 번째 줄) -->
            <div class="toolbar">
              <label style="font-size:13px;color:var(--sub)">기간</label>
              <input id="dateFrom" type="date" lang="en-CA" placeholder="YYYY-MM-DD" inputmode="numeric" pattern="\\d{{4}}-\\d{{2}}-\\d{{2}}"/>
              <span style="color:var(--sub)">~</span>
              <input id="dateTo" type="date" lang="en-CA" placeholder="YYYY-MM-DD" inputmode="numeric" pattern="\\d{{4}}-\\d{{2}}-\\d{{2}}"/>
              <button class="btn" id="btnToday">오늘</button>
              <button class="btn" id="btn7">최근 7일</button>
              <button class="btn" id="btn30">최근 30일</button>
              <button class="btn" id="btnDateReset">날짜 초기화</button>
              
              <span style="width:8px"></span>
              <button class="btn" id="resetBtn">전체 초기화</button>
            </div>
        
            <section class="cards" id="cards">
              {cards}
            </section>
        
            <footer>© 부태리의 실거래가 모니터링 시스템</footer>
          </div>
        
          <script>
          (function(){{
            const $ = (s, r=document)=>r.querySelector(s);
            const $$ = (s, r=document)=>Array.from(r.querySelectorAll(s));
        
            const cards = $$('section.card');
            const headerSub = $('.header .sub');
    
            /* === 날짜 필터 요소 === */
            const dateFrom = document.getElementById('dateFrom');
            const dateTo   = document.getElementById('dateTo');
            const btnToday = document.getElementById('btnToday');
            const btn7     = document.getElementById('btn7');
            const btn30    = document.getElementById('btn30');
            const btnDateReset = document.getElementById('btnDateReset');
    
            /* === 상승률 필터 요소 === */
            const rateMin = document.getElementById('rateMin');
            const rateMax = document.getElementById('rateMax');
            const btnRate5 = document.getElementById('btnRate5');
            const btnRate10 = document.getElementById('btnRate10');
            const btnRateReset = document.getElementById('btnRateReset');
    
            /* === 전체 초기화 버튼 === */
            const resetBtn = document.getElementById('resetBtn');
            
            function setHeaderCount(n){{
              if(!headerSub) return;
              const base = headerSub.textContent.replace(/· 총\\s+\\d+\\s*개\\s*아파트.*/, '').trim();
              const filterOn = (dateFrom.value || dateTo.value || filterYoung || filterVeryYoung || filterOld || selectedRegion !== '__ALL__' || rateMin.value || rateMax.value);
              const filterBadge = filterOn ? ' (필터 적용)' : '';
              headerSub.textContent = `${{base}} · 총 ${{n}}개 아파트${{filterBadge}}`;
            }}
    
            /* ========= 유틸 ========= */
            function toInputValue(d){{
              const y = d.getFullYear();
              const m = String(d.getMonth()+1).padStart(2,'0');
              const day = String(d.getDate()).padStart(2,'0');
              return `${{y}}-${{m}}-${{day}}`;
            }}
    
            // '이번 실거래 (YYYY-MM-DD | ...)' 날짜 파싱
            function parseTradeDate(card){{
              const priceSpan = card.querySelector('.grid .highlight');
              if(!priceSpan) return null;
              const cell = priceSpan.parentElement;
              const text = (cell.textContent || '').trim();
              const m = text.match(/\\((\\d{{4}}-\\d{{2}}-\\d{{2}})[^\\)]*\\)/);
              return m ? new Date(m[1]) : null;
            }}
    
            // 변화 영역에서 상승률 % 파싱 (예: "... 상승 (9.0%)") → 숫자 9.0
            function parseRisePct(card){{
              const changeEl = card.querySelector('.grid .rise');
              if(!changeEl) return null;
              const text = changeEl.textContent || '';
              const m = text.match(/(\\d+(?:\\.\\d+)?)%/);
              return m ? parseFloat(m[1]) : null;
            }}
        
            // 지역 카운트 및 칩 구성
            const regionCounts = new Map();
            cards.forEach(card=>{{
              const r = card.dataset.region || '';
              if(!r) return;
              regionCounts.set(r, (regionCounts.get(r)||0)+1);
            }});
        
            const chipsWrap = $('.region-chips');
            if (chipsWrap){{
              const allChip = document.createElement('span');
              allChip.className = 'region-chip active';
              allChip.textContent = '전체 보기';
              chipsWrap.appendChild(allChip);
        
              const sorted = Array.from(regionCounts.entries()).sort((a,b)=> b[1]-a[1] || a[0].localeCompare(b[0]));
              sorted.forEach(([name,cnt])=>{{
                const chip = document.createElement('span');
                chip.className = 'region-chip';
                chip.textContent = `${{name}} ${{cnt}}건`;
                chipsWrap.appendChild(chip);
              }});
            }}
        
            // 필터 로직
            let selectedRegion = '__ALL__';
            let filterYoung = false;
            let filterVeryYoung = false;
            let filterOld = false;
        
            function chipRegionText(chip){{
              return chip.textContent.replace(/\\d+\\s*건/g,'').trim();
            }}
        
            function applyFilter(){{
              const fromVal = dateFrom.value ? new Date(dateFrom.value) : null;
              const toVal   = dateTo.value ? new Date(dateTo.value) : null;
    
              const yOldOn   = filterOld;
              const yVYOn    = filterVeryYoung;
              const yYoungOn = filterYoung;
              const yearFilterOn = yOldOn || yVYOn || yYoungOn;
    
              const regionFilterOn = selectedRegion !== '__ALL__';
    
              const minPct = rateMin.value !== '' ? parseFloat(rateMin.value) : null;
              const maxPct = rateMax.value !== '' ? parseFloat(rateMax.value) : null;
              const rateFilterOn = (minPct !== null) || (maxPct !== null);
    
              let shown = 0;
              cards.forEach(card=>{{
                // 날짜 조건(이번 실거래 기준)
                const d = parseTradeDate(card);
                const passDate =
                  (!fromVal || (d && d >= fromVal)) &&
                  (!toVal   || (d && d <= toVal));
    
                // 연차 조건
                const isOld   = card.dataset.old === '1';
                const isVY    = card.dataset.veryYoung === '1';
                const isYoung = card.dataset.young === '1';
                const passYear = !yearFilterOn || (
                  (yOldOn   && isOld)   ||
                  (yVYOn    && isVY)    ||
                  (yYoungOn && isYoung)
                );
    
                // 지역 조건
                const regionOk = (selectedRegion==='__ALL__') ? true : (card.dataset.region === selectedRegion);
                
                // 상승률 조건
                const pct = parseRisePct(card);
                const passRate = !rateFilterOn || (
                  (minPct === null || (pct !== null && pct >= minPct)) &&
                  (maxPct === null || (pct !== null && pct <= maxPct))
                );
    
                const match = passDate && passYear && regionOk && passRate;
                card.classList.toggle('hidden', !match);
                if (match) shown++;
              }});
              setHeaderCount(shown);
            }}
        
            // 지역칩 이벤트
            const allChipEl = $$('.region-chip', chipsWrap).find(c=>c.textContent.trim()==='전체 보기');
            if (allChipEl) allChipEl.addEventListener('click', ()=>{{
              selectedRegion='__ALL__';
              $$('.region-chip').forEach(c=> c.classList.toggle('active', c===allChipEl));
              applyFilter();
            }});
            
            $$('.region-chip', chipsWrap).filter(c=>c!==allChipEl)
              .forEach(c=> c.addEventListener('click', ()=>{{
                selectedRegion = chipRegionText(c);
                $$('.region-chip').forEach(x=> x.classList.toggle('active', x===c));
                applyFilter();
              }}));
        
            // '2000년대 미만' 칩
            const oldChip = $('#chip-old');
            if (oldChip){{
              oldChip.addEventListener('click', ()=>{{
                filterOld = !filterOld;
                filterVeryYoung = false;
                filterYoung = false;
                oldChip.classList.toggle('active', filterOld);
                $('#chip-very-young').classList.remove('active');
                $('#chip-young').classList.remove('active');
                applyFilter();
              }});
            }}
        
            // '입주 5년 이하' 칩
            const veryYoungChip = $('#chip-very-young');
            if (veryYoungChip){{
              veryYoungChip.addEventListener('click', ()=>{{
                filterVeryYoung = !filterVeryYoung;
                filterYoung = false;
                filterOld = false;
                veryYoungChip.classList.toggle('active', filterVeryYoung);
                $('#chip-young').classList.remove('active');
                $('#chip-old').classList.remove('active');
                applyFilter();
              }});
            }}
        
            // '입주 10년 이하' 칩
            const youngChip = $('#chip-young');
            if (youngChip){{
              youngChip.addEventListener('click', ()=>{{
                filterYoung = !filterYoung;
                filterVeryYoung = false;
                filterOld = false;
                youngChip.classList.toggle('active', filterYoung);
                $('#chip-very-young').classList.remove('active');
                $('#chip-old').classList.remove('active');
                applyFilter();
              }});
            }}
    
            // 날짜 입력 이벤트
            dateFrom.addEventListener('change', ()=>{{
              if(dateTo.value && dateFrom.value && dateFrom.value > dateTo.value){{
                dateTo.value = dateFrom.value;
              }}
              applyFilter();
            }});
            dateTo.addEventListener('change', ()=>{{
              if(dateFrom.value && dateTo.value && dateTo.value < dateFrom.value){{
                dateFrom.value = dateTo.value;
              }}
              applyFilter();
            }});
    
            // 빠른 날짜 버튼
            function todayStr(){{ return toInputValue(new Date()); }}
            function daysAgo(n){{
              const d = new Date();
              d.setDate(d.getDate() - n + 1); // 오늘 포함 n일
              return d;
            }}
            btnToday.addEventListener('click', ()=>{{
              const t = todayStr();
              dateFrom.value = t; dateTo.value = t;
              applyFilter();
            }});
            btn7.addEventListener('click', ()=>{{
              dateFrom.value = toInputValue(daysAgo(7));
              dateTo.value   = todayStr();
              applyFilter();
            }});
            btn30.addEventListener('click', ()=>{{
              dateFrom.value = toInputValue(daysAgo(30));
              dateTo.value   = todayStr();
              applyFilter();
            }});
    
            // 날짜 초기화 버튼
            btnDateReset.addEventListener('click', ()=>{{
              dateFrom.value = '';
              dateTo.value = '';
              applyFilter();
            }});
    
            // 상승률 입력/프리셋
            function onRateChange(){{ applyFilter(); }}
            rateMin.addEventListener('input', onRateChange);
            rateMax.addEventListener('input', onRateChange);
            btnRate5.addEventListener('click', ()=>{{ rateMin.value = '5'; rateMax.value = ''; applyFilter(); }});
            btnRate10.addEventListener('click', ()=>{{ rateMin.value = '10'; rateMax.value = ''; applyFilter(); }});
            btnRateReset.addEventListener('click', ()=>{{ rateMin.value = ''; rateMax.value = ''; applyFilter(); }});
    
            // 전체 초기화
            resetBtn.addEventListener('click', ()=>{{
              dateFrom.value = '';
              dateTo.value = '';
              rateMin.value = '';
              rateMax.value = '';
              filterOld = false;
              filterVeryYoung = false;
              filterYoung = false;
              oldChip.classList.remove('active');
              $('#chip-very-young').classList.remove('active');
              $('#chip-young').classList.remove('active');
              selectedRegion = '__ALL__';
              $$('.region-chip').forEach(c=> c.classList.toggle('active', c===allChipEl));
              applyFilter();
            }});
        
            applyFilter();
          }})();
          </script>
        </body>
        </html>"""
        
        return html_content

    def export_notification_html(self, apt_list, *, ask_path=False, silent=False):
        """신고가 알림 HTML 파일 저장 및 열기
        - ask_path=True 이면 저장 위치를 사용자에게 물음
        - silent=True 이면 메시지박스 없이 조용히 저장
        """
        try:
            html_str = self.build_notification_html(apt_list)
            # 저장 경로
            if ask_path:
                initdir = os.path.join(self.download_path, "reports")
                os.makedirs(initdir, exist_ok=True)
                from tkinter import filedialog
                ts = datetime.now().strftime("%Y%m%d_%H%M%S")
                default = f"부태리신고가_{ts}.html"
                filepath = filedialog.asksaveasfilename(
                    initialdir=initdir, initialfile=default,
                    title="신고가 HTML 저장",
                    defaultextension=".html",
                    filetypes=[("HTML files","*.html"), ("All files","*.*")]
                )
                if not filepath:
                    return None
            else:
                reports_dir = os.path.join(self.download_path, "reports")
                os.makedirs(reports_dir, exist_ok=True)
                ts = datetime.now().strftime("%Y%m%d_%H%M%S")
                filepath = os.path.join(reports_dir, f"부태리신고가_{ts}.html")
            # 쓰기
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(html_str)
            # 열기
            try:
                if os.name == "nt":
                    os.startfile(filepath)  # Windows
                else:
                    webbrowser.open("file://" + filepath)
            except Exception:
                pass
            if not silent:
                messagebox.showinfo("저장 완료", f"HTML 저장: {filepath}")
            logging.info(f"[HTML 저장] {filepath}")
            return filepath
        except Exception as e:
            logging.error(f"HTML 저장 중 오류: {str(e)}")
            if not silent:
                messagebox.showerror("오류", f"HTML 저장 실패: {str(e)}")
            return None


    
    def show_new_max_notification(self, apt_list):
        """신고가 발견 시 9:16 비율 알림 창 (페이지네이션 + 캡처 기능)"""
        if not apt_list:
            return
        
        # 신고가 높은 순으로 정렬
        apt_list = sorted(apt_list, key=lambda x: x.get('new_price', 0), reverse=True)
        current_time = datetime.now()
        is_duplicate = False
        for existing in self.notifications_history:
            existing_time = existing.get('timestamp', datetime.now())
            if isinstance(existing_time, str):
                try:
                    existing_time = datetime.strptime(existing_time, '%Y-%m-%d %H:%M:%S')
                except:
                    continue
            if (abs((current_time - existing_time).total_seconds()) < 60 and 
                len(existing.get('apt_list', [])) == len(apt_list)):
                is_duplicate = True
                break
        if not is_duplicate:
            notification_data = {'timestamp': current_time, 'apt_list': apt_list.copy()}
            self.notifications_history.append(notification_data)
            if len(self.notifications_history) > 50:
                self.notifications_history = self.notifications_history[-50:]
            self.save_notifications_history()
        apts_per_page = 2
        total_pages = (len(apt_list) + apts_per_page - 1) // apts_per_page
        current_page = 0
        notification_window = tk.Toplevel(self.root)
        notification_window.title("📱 부태리의 신고가")
        base_width = 400
        base_height = int(base_width * 16 / 9)
        screen_width = notification_window.winfo_screenwidth()
        screen_height = notification_window.winfo_screenheight()
        max_width = min(500, int(screen_width * 0.4))
        max_height = min(900, int(screen_height * 0.85))
        if base_height > max_height:
            window_height = max_height
            window_width = int(window_height * 9 / 16)
        elif base_width > max_width:
            window_width = max_width
            window_height = int(window_width * 16 / 9)
        else:
            window_width = base_width
            window_height = base_height
        min_width = 300
        min_height = int(min_width * 16 / 9)
        if window_width < min_width:
            window_width = min_width
            window_height = min_height
        elif window_height < min_height:
            window_height = min_height
            window_width = int(window_height * 9 / 16)
        x = (screen_width - window_width) // 2
        y = (screen_height - window_height) // 2
        notification_window.geometry(f"{window_width}x{window_height}+{x}+{y}")
        notification_window.resizable(False, False)
        notification_window.transient(self.root)
        notification_window.grab_set()
        notification_window.attributes('-topmost', True)
        try:
            dpi = notification_window.winfo_fpixels('1i')
            scale_factor = max(1.0, dpi / 96.0)
        except:
            scale_factor = 1.0
        def scaled_font(family, size, weight='normal'):
            scaled_size = max(8, int(size * scale_factor))
            return (family, scaled_size, weight)
        colors = {
            'primary': '#007AFF',
            'primary_dark': '#0051D2',
            'background': '#F2F2F7',
            'surface': '#FFFFFF',
            'card_bg': '#FFFFFF',
            'error': '#FF3B30',
            'error_light': '#FFEBEE',
            'success': '#34C759',
            'text_primary': '#000000',
            'text_secondary': '#8E8E93',
            'separator': '#E5E5EA',
            'capture': '#FF9500'
        }
        main_frame = tk.Frame(notification_window, bg=colors['background'])
        main_frame.pack(fill='both', expand=True)
        header_height = int(window_height * 0.16)
        header_frame = tk.Frame(main_frame, bg=colors['primary'], height=header_height)
        header_frame.pack(fill='x')
        header_frame.pack_propagate(False)
        header_content = tk.Frame(header_frame, bg=colors['primary'])
        header_content.pack(expand=True)
        header_main = tk.Frame(header_content, bg=colors['primary'])
        header_main.pack(expand=True)
        icon_size = max(24, int(28 * scale_factor))
        icon_label = tk.Label(header_main, text="🔔", font=scaled_font('Segoe UI Emoji', icon_size), bg=colors['primary'], fg='white')
        icon_label.pack(pady=(5, 3))
        title_label = tk.Label(header_main, text="부태리의 신고가", font=scaled_font('Malgun Gothic', 18, 'bold'), bg=colors['primary'], fg='white')
        title_label.pack()
        page_info_label = tk.Label(header_main, text="", font=scaled_font('Malgun Gothic', 10), bg=colors['primary'], fg='#E3F2FD')
        page_info_label.pack(pady=(3, 10))
        content_height = int(window_height * 0.68)
        content_container = tk.Frame(main_frame, bg=colors['background'], height=content_height)
        content_container.pack(fill='both', expand=True, padx=15, pady=10)
        content_container.pack_propagate(False)
        content_frame = tk.Frame(content_container, bg=colors['background'])
        content_frame.pack(fill='both', expand=True)
        def update_page():
            for widget in content_frame.winfo_children():
                widget.destroy()
            start_idx = current_page * apts_per_page
            end_idx = min(start_idx + apts_per_page, len(apt_list))
            current_apts = apt_list[start_idx:end_idx]
            if total_pages > 1:
                page_text = f"{current_page + 1}/{total_pages} 페이지 | {len(apt_list)}개 아파트"
            else:
                page_text = f"{len(apt_list)}개 아파트에서 최고가 갱신"
            page_info_label.config(text=page_text)
            for i, apt in enumerate(current_apts):
                card_frame = tk.Frame(content_frame, bg=colors['card_bg'], relief='flat', bd=0, highlightbackground=colors['separator'], highlightthickness=1)
                card_frame.pack(fill='x', pady=(0, 6 if i < len(current_apts) - 1 else 0))
                card_content = tk.Frame(card_frame, bg=colors['card_bg'])
                card_content.pack(fill='both', expand=True, padx=12, pady=10)
                apt_header = tk.Frame(card_content, bg=colors['card_bg'])
                apt_header.pack(fill='x', pady=(0, 6))
                apt_name_size = max(10, int(12 * scale_factor))
                apt_name_label = tk.Label(apt_header, text=f"🏢 {apt['apt_name']}", font=scaled_font('Malgun Gothic', apt_name_size, 'bold'), bg=colors['card_bg'], fg=colors['text_primary'], anchor='w')
                apt_name_label.pack(anchor='w')
                
                # 지역 정보 추가
                location_text = ""
                if 'sido' in apt and 'sigungu' in apt and 'dong' in apt:
                    location_text = f"📍 {apt['sido']} {apt['sigungu']} {apt['dong']}"
                elif 'location' in apt:
                    location_text = f"📍 {apt['location']}"
                    
                if location_text:
                    location_size = max(7, int(9 * scale_factor))
                    location_label = tk.Label(apt_header, text=location_text, font=scaled_font('Malgun Gothic', location_size), bg=colors['card_bg'], fg=colors['text_secondary'], anchor='w')
                    location_label.pack(anchor='w', pady=(1, 0))
                
                area_size = max(8, int(10 * scale_factor))
                area_label = tk.Label(apt_header, text=f"📐 {apt['area']}㎡", font=scaled_font('Malgun Gothic', area_size), bg=colors['card_bg'], fg=colors['text_secondary'], anchor='w')
                area_label.pack(anchor='w', pady=(1, 0))
                price_section = tk.Frame(card_content, bg=colors['card_bg'])
                price_section.pack(fill='x', pady=(0, 6))
                if apt['old_price'] > 0:
                    old_price_frame = tk.Frame(price_section, bg=colors['card_bg'])
                    old_price_frame.pack(fill='x', pady=(0, 3))
                    old_label_size = max(8, int(9 * scale_factor))
                    tk.Label(old_price_frame, text="이전 최고가", font=scaled_font('Malgun Gothic', old_label_size), bg=colors['card_bg'], fg=colors['text_secondary']).pack(side='left')
                    tk.Label(old_price_frame, text=f"{apt['old_price']:,}만원", font=scaled_font('Malgun Gothic', old_label_size), bg=colors['card_bg'], fg=colors['text_secondary']).pack(side='right')
                new_price_frame = tk.Frame(price_section, bg=colors['card_bg'])
                new_price_frame.pack(fill='x')
                new_label_size = max(9, int(11 * scale_factor))
                tk.Label(new_price_frame, text="📈 신고가", font=scaled_font('Malgun Gothic', new_label_size, 'bold'), bg=colors['card_bg'], fg=colors['error']).pack(side='left')
                new_price_size = max(11, int(13 * scale_factor))
                tk.Label(new_price_frame, text=f"{apt['new_price']:,}만원", font=scaled_font('Malgun Gothic', new_price_size, 'bold'), bg=colors['card_bg'], fg=colors['error']).pack(side='right')
                if apt['old_price'] > 0:
                    increase = apt['new_price'] - apt['old_price']
                    increase_percent = (increase / apt['old_price']) * 100
                    highlight_frame = tk.Frame(card_content, bg=colors['error_light'], relief='flat', bd=1)
                    highlight_frame.pack(fill='x', pady=(6, 0))
                    highlight_content = tk.Frame(highlight_frame, bg=colors['error_light'])
                    highlight_content.pack(fill='x', padx=6, pady=4)
                    increase_size = max(8, int(10 * scale_factor))
                    increase_label = tk.Label(highlight_content, text=f"🔥 {increase:,}만원 상승 (+{increase_percent:.1f}%)", font=scaled_font('Malgun Gothic', increase_size, 'bold'), bg=colors['error_light'], fg='#C62828')
                    increase_label.pack(anchor='w')
                    detail_info = f"📅 {apt['date']}"
                    if apt.get('floor'):
                        detail_info += f" | {apt['floor']}층"
                    if apt.get('dong') and apt['dong'] != '-':
                        detail_info += f" | {apt['dong']}"
                    detail_size = max(7, int(8 * scale_factor))
                    detail_label = tk.Label(highlight_content, text=detail_info, font=scaled_font('Malgun Gothic', detail_size), bg=colors['error_light'], fg=colors['text_secondary'])
                    detail_label.pack(anchor='w', pady=(1, 0))
            if total_pages > 1:
                prev_button.config(state='normal' if current_page > 0 else 'disabled')
                next_button.config(state='normal' if current_page < total_pages - 1 else 'disabled')
            if total_pages > 1:
                page_display.config(text=f"{current_page + 1} / {total_pages}")
            else:
                page_display.config(text="")
        
        bottom_height = int(window_height * 0.16)
        bottom_container = tk.Frame(main_frame, bg=colors['background'], height=bottom_height)
        bottom_container.pack(fill='x', padx=15, pady=(0, 15))
        bottom_container.pack_propagate(False)
        if total_pages > 1:
            nav_frame = tk.Frame(bottom_container, bg=colors['background'])
            nav_frame.pack(fill='x', pady=(0, 8))
            def go_prev():
                nonlocal current_page
                if current_page > 0:
                    current_page -= 1
                    update_page()
            prev_button = tk.Button(nav_frame, text="◀", font=scaled_font('Malgun Gothic', 12, 'bold'), bg=colors['primary'], fg='white', activebackground=colors['primary_dark'], relief='flat', bd=0, width=4, height=1, cursor='hand2', command=go_prev)
            prev_button.pack(side='left')
            page_display = tk.Label(nav_frame, text="", font=scaled_font('Malgun Gothic', 11, 'bold'), bg=colors['background'], fg=colors['text_primary'])
            page_display.pack(side='left', expand=True)
            def go_next():
                nonlocal current_page
                if current_page < total_pages - 1:
                    current_page += 1
                    update_page()
            next_button = tk.Button(nav_frame, text="▶", font=scaled_font('Malgun Gothic', 12, 'bold'), bg=colors['primary'], fg='white', activebackground=colors['primary_dark'], relief='flat', bd=0, width=4, height=1, cursor='hand2', command=go_next)
            next_button.pack(side='right')
        else:
            prev_button = tk.Button(bottom_container)
            next_button = tk.Button(bottom_container)
            page_display = tk.Label(bottom_container)
            prev_button.pack_forget()
            next_button.pack_forget()
            page_display.pack_forget()
        def capture_screen():
            try:
                notification_window.update_idletasks()
                time.sleep(0.1)
                x = notification_window.winfo_rootx()
                y = notification_window.winfo_rooty()
                width = notification_window.winfo_width()
                try:
                    from ctypes import windll
                    windll.shcore.SetProcessDpiAwareness(1)
                except:
                    pass
                header_y = header_frame.winfo_rooty()
                header_height = header_frame.winfo_height()
                content_y = content_container.winfo_rooty()
                content_height = content_container.winfo_height()
                capture_y = header_y
                capture_bottom = content_y + content_height
                print(f"창 위치: x={x}, y={y}")
                print(f"창 크기: width={width}, height={notification_window.winfo_height()}")
                print(f"헤더 위치: y={header_y}, height={header_height}")
                print(f"콘텐츠 위치: y={content_y}, height={content_height}")
                print(f"캡처 영역: y={capture_y}, height={capture_bottom - capture_y}")
                try:
                    bbox = (x, capture_y, x + width, capture_bottom)
                    screenshot = ImageGrab.grab(bbox=bbox)
                    if screenshot.size[0] <= 0 or screenshot.size[1] <= 0:
                        raise ValueError("캡처된 이미지 크기가 유효하지 않습니다.")
                    capture_folder = os.path.join(self.download_path, "captures")
                    os.makedirs(capture_folder, exist_ok=True)
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    page_info = f"_page{current_page + 1}of{total_pages}" if total_pages > 1 else ""
                    filename = f"부태리신고가_{timestamp}{page_info}.png"
                    filepath = os.path.join(capture_folder, filename)
                    screenshot.save(filepath, 'PNG', quality=95)
                    capture_button.config(text="✓", fg=colors['success'])
                    notification_window.after(3000, lambda: capture_button.config(text="📸", fg='white'))
                    print(f"📸 화면 캡처 성공: {filepath}")
                    print(f"   캡처 크기: {screenshot.size[0]}x{screenshot.size[1]}")
                except ImportError:
                    capture_button.config(text="❌", fg=colors['error'])
                    notification_window.after(3000, lambda: capture_button.config(text="📸", fg='white'))
                    messagebox.showerror("오류", "화면 캡처를 위해 Pillow 라이브러리가 필요합니다.\npip install Pillow 명령으로 설치해주세요.")
            except Exception as e:
                capture_button.config(text="❌", fg=colors['error'])
                notification_window.after(3000, lambda: capture_button.config(text="📸", fg='white'))
                print(f"캡처 오류: {str(e)}")
                import traceback
                traceback.print_exc()
                messagebox.showerror("오류", f"화면 캡처 중 오류가 발생했습니다:\n{str(e)}")
        # --- 버튼 행 컨테이너 ---
        button_frame = tk.Frame(bottom_container, bg=colors['background'])
        button_frame.pack(fill='x')
        button_size = max(12, int(14 * scale_factor))
        
        # 1) 확인 버튼
        confirm_button = tk.Button(
            button_frame,
            text="확인",
            font=scaled_font('Malgun Gothic', button_size, 'bold'),
            bg=colors['primary'], fg='white', activebackground=colors['primary_dark'],
            activeforeground='white', relief='flat', bd=0, pady=12, cursor='hand2',
            command=notification_window.destroy
        )
        confirm_button.pack(side='left', fill='both', expand=True, padx=(0, 5))
        
        # 2) (원하는 위치에 따라 pack 순서 조절) 캡처 버튼
        capture_button = tk.Button(
            button_frame,
            text="📸",
            font=scaled_font('Segoe UI Emoji', 14),
            bg=colors['capture'], fg='white', activebackground='#FF7A00',
            relief='flat', bd=0, width=4, pady=12, cursor='hand2',
            command=capture_screen
        )
        capture_button.pack(side='right')
        
        # 3) HTML 저장 버튼 (지금 순서면 📸 왼쪽에 위치)
        html_button = tk.Button(
            button_frame,
            text="📝 HTML 저장",
            font=scaled_font('Malgun Gothic', button_size, 'bold'),
            bg='#10B981', fg='white', activebackground='#0E9F6E', relief='flat', bd=0,
            pady=12, cursor='hand2',
            command=lambda: self.export_notification_html(apt_list, ask_path=False, silent=False)
        )
        html_button.pack(side='right', padx=(6, 0))
        
        # --- 호버 핸들러 (버튼 만든 뒤 정의해도 되고, 앞에 있어도 됩니다) ---
        def on_enter(e):
            if e.widget == confirm_button:
                confirm_button.config(bg=colors['primary_dark'])
            elif e.widget == capture_button:
                capture_button.config(bg='#FF7A00')
            elif e.widget == html_button:
                html_button.config(bg='#0E9F6E')
        
        def on_leave(e):
            if e.widget == confirm_button:
                confirm_button.config(bg=colors['primary'])
            elif e.widget == capture_button:
                capture_button.config(bg=colors['capture'])
            elif e.widget == html_button:
                html_button.config(bg='#10B981')
        
        # --- 바인딩 (버튼 '생성 후'에 있어야 함) ---
        confirm_button.bind("<Enter>", on_enter)
        confirm_button.bind("<Leave>", on_leave)
        capture_button.bind("<Enter>", on_enter)
        capture_button.bind("<Leave>", on_leave)
        html_button.bind("<Enter>", on_enter)
        html_button.bind("<Leave>", on_leave)
        if total_pages > 1:
            prev_button.bind("<Enter>", on_enter)
            prev_button.bind("<Leave>", on_leave)
            next_button.bind("<Enter>", on_enter)
            next_button.bind("<Leave>", on_leave)
        def on_key_press(event):
            if event.keysym == 'Escape' or event.keysym == 'Return':
                notification_window.destroy()
            elif event.keysym == 'Left' and total_pages > 1 and current_page > 0:
                go_prev()
            elif event.keysym == 'Right' and total_pages > 1 and current_page < total_pages - 1:
                go_next()
            elif event.keysym == 'F12' or (event.state & 0x4 and event.keysym == 's'):
                capture_screen()
        notification_window.bind('<Key>', on_key_press)
        notification_window.focus_set()
        update_page()
        notification_window.after(100, lambda: notification_window.focus_force())
        try:
            title = "📱 부태리의 신고가"
            message = f"{len(apt_list)}개 아파트에서 신고가가 발견되었습니다!"
            notification.notify(title=title, message=message, app_name="실거래가 모니터", timeout=10)
        except Exception as e:
            logging.error(f"시스템 알림 표시 중 오류: {str(e)}")
        print(f"📱 부태리 신고가 알림창 크기: {window_width}x{window_height} (9:16 비율)")
        print(f"📄 총 {total_pages}페이지, 페이지당 {apts_per_page}개 아파트")
        print(f"📸 캡처 기능: F12 키 또는 우측 하단 📸 버튼")
    
    def on_closing(self):
        """프로그램 종료 시 처리"""
        try:
            self.save_monitored_apts()
            self.save_notifications_history()
            if messagebox.askokcancel("종료", "프로그램을 종료하시겠습니까?"):
                self.root.destroy()
        except Exception as e:
            logging.error(f"종료 처리 중 오류: {str(e)}")
            self.root.destroy()


class AptSelectDialog:
    def __init__(self, parent, apt_list, service_key, sigungu_code, dong, sido, sigungu, title="아파트 선택"):
        self.parent = parent
        self.service_key = service_key
        self.sigungu_code = sigungu_code
        self.dong = dong
        self.sido = sido
        self.sigungu = sigungu
        self.apt_list = apt_list
        self.result = None
        self.selected_apt = None
        
        self.top = tk.Toplevel(parent)
        self.top.title(title)
        self.top.attributes('-topmost', True)
        
        width = 800
        height = 500
        screen_width = self.top.winfo_screenwidth()
        screen_height = self.top.winfo_screenheight()
        x = (screen_width - width) // 2
        y = (screen_height - height) // 2
        self.top.geometry(f"{width}x{height}+{x}+{y}")
        
        self.font_normal = ('Malgun Gothic', 9)
        self.font_large = ('Malgun Gothic', 11)
        self.font_button = ('Malgun Gothic', 9)
        
        search_frame = ttk.Frame(self.top, padding="5")
        search_frame.pack(fill='x', padx=5, pady=5)
        ttk.Label(search_frame, text=f"{self.dong} 아파트 목록", font=self.font_large).pack(side='left')
        ttk.Label(search_frame, text="검색:", font=self.font_normal).pack(side='left', padx=(20, 0))
        self.search_var = tk.StringVar()
        self.search_var.trace('w', self.filter_apartments)
        search_entry = ttk.Entry(search_frame, textvariable=self.search_var, width=40, font=self.font_normal)
        search_entry.pack(side='left', fill='x', expand=True, padx=5)
        
        list_frame = ttk.Frame(self.top, padding="5")
        list_frame.pack(fill='both', expand=True, padx=5, pady=5)
        scrollbar = ttk.Scrollbar(list_frame)
        scrollbar.pack(side='right', fill='y')
        self.listbox = tk.Listbox(list_frame, yscrollcommand=scrollbar.set, font=self.font_normal)
        self.listbox.pack(fill='both', expand=True)
        scrollbar.config(command=self.listbox.yview)
        
        self.update_listbox(apt_list)
        
        button_frame = ttk.Frame(self.top, padding="5")
        button_frame.pack(fill='x', padx=5, pady=5)
        select_button = ttk.Button(button_frame, text="선택", command=self.on_button_select)
        select_button.pack(side='right', padx=5)
        cancel_button = ttk.Button(button_frame, text="취소", command=self.top.destroy)
        cancel_button.pack(side='right', padx=5)
        self.listbox.bind('<Double-Button-1>', self.on_select)
    
    def update_listbox(self, items):
        self.listbox.delete(0, tk.END)
        for item in items:
            self.listbox.insert(tk.END, item)
    
    def filter_apartments(self, *args):
        search_text = self.search_var.get().lower()
        filtered_list = [apt for apt in self.apt_list if search_text in apt.lower()]
        self.update_listbox(filtered_list)
    
    def on_button_select(self):
        if not self.listbox.curselection():
            messagebox.showinfo("알림", "아파트를 선택해주세요.")
            return
        self.on_select(None)
    
    def on_select(self, event):
        if self.listbox.curselection():
            full_text = self.listbox.get(self.listbox.curselection())
            is_new_apt = False
            if full_text.startswith("[신축]"):
                is_new_apt = True
                full_text = full_text[5:].strip()
            if '[' in full_text and ']' in full_text:
                address_info = full_text[full_text.find('[')+1:full_text.find(']')]
                addr_parts = address_info.split(' / ')
                if len(addr_parts) > 1:
                    jibun_addr = addr_parts[1]
                else:
                    jibun_addr = addr_parts[0]
                simple_addr = ' '.join(jibun_addr.split()[-2:])
            else:
                simple_addr = ""
            apt_name = full_text.split('[')[0].strip()
            build_year = ""
            if "(준공:" in full_text:
                build_year_part = full_text.split("(준공:")[1].strip()
                build_year = build_year_part.split("년")[0].strip()
            elif "(분양중)" in full_text:
                build_year = "분양"
            self.selected_apt = apt_name
            self.simple_addr = simple_addr
            self.build_year = build_year
            self.is_new_apt = is_new_apt
            self.show_area_dialog()
    
    def show_area_dialog(self):
        """전용면적 목록 다이얼로그 표시"""
        # 전용면적 목록 가져오기
        area_list = self.get_areas_for_apt(self.selected_apt)
        
        if not area_list:
            messagebox.showinfo("알림", "해당 아파트의 전용면적 정보를 찾을 수 없습니다.")
            return
        
        area_dialog = tk.Toplevel(self.top)
        area_dialog.title(f"{self.selected_apt} - 전용면적 선택")
        area_dialog.attributes('-topmost', True)
        
        width = 300
        height = 200
        x = self.top.winfo_x() + 50
        y = self.top.winfo_y() + 50
        area_dialog.geometry(f"{width}x{height}+{x}+{y}")
        
        list_frame = ttk.Frame(area_dialog, padding="5")
        list_frame.pack(fill='both', expand=True, padx=5, pady=5)
        
        scrollbar = ttk.Scrollbar(list_frame)
        scrollbar.pack(side='right', fill='y')
        
        listbox = tk.Listbox(list_frame, yscrollcommand=scrollbar.set, font=self.font_normal)
        listbox.pack(fill='both', expand=True)
        scrollbar.config(command=listbox.yview)
        
        for area in sorted(area_list, key=lambda x: float(x)):
            listbox.insert(tk.END, f"{area}㎡")
        
        # 첫 번째 항목 선택 (사용자 편의)
        if len(area_list) > 0:
            listbox.selection_set(0)
        
        def on_area_select(event=None):
            # 선택한 항목이 없으면 첫 번째 항목 자동 선택
            if not listbox.curselection() and len(area_list) > 0:
                listbox.selection_set(0)
                
            if not listbox.curselection():
                messagebox.showinfo("알림", "전용면적을 선택해주세요.")
                return
                
            selected_area = listbox.get(listbox.curselection())
            area_value = selected_area.replace('㎡', '').strip()
            
            # 아파트 정보 구성
            apt_info = {
                'apt_name': self.selected_apt,
                'jibun_addr': self.simple_addr,
                'area': area_value,
                'sido': self.sido,
                'sigungu': self.sigungu,
                'dong': self.dong,
                'sigungu_code': self.sigungu_code,
                'build_year': self.build_year
            }
            
            # 결과 저장
            self.result = apt_info
            
            # 창 닫기
            area_dialog.destroy()
            self.top.destroy()
        
        # 이벤트 바인딩
        listbox.bind('<Double-1>', on_area_select)
        
        # 선택 버튼 프레임
        button_frame = ttk.Frame(area_dialog, padding="5")
        button_frame.pack(fill='x', padx=5, pady=5)
        
        # 선택 버튼
        select_button = ttk.Button(button_frame, text="선택", command=on_area_select)
        select_button.pack(side='right', padx=5)
        
        cancel_button = ttk.Button(button_frame, text="취소", command=area_dialog.destroy)
        cancel_button.pack(side='right', padx=5)
        
        # Enter 키 이벤트 바인딩 추가
        listbox.bind('<Return>', on_area_select)
        area_dialog.bind('<Return>', on_area_select)
        
        # 다이얼로그가 닫힐 때 처리
        def on_dialog_close():
            area_dialog.destroy()
        
        area_dialog.protocol("WM_DELETE_WINDOW", on_dialog_close)
        
        # 포커스 설정
        listbox.focus_set()
        
        # 모달 다이얼로그로 처리
        area_dialog.transient(self.top)
        area_dialog.grab_set()
        self.top.wait_window(area_dialog)
        
    def get_areas_for_apt(self, apt_name):
        """해당 아파트의 전용면적 목록 가져오기 (최근 3개월)"""
        areas = set()
        current_date = datetime.now()
        
        # [신축] 태그 제거
        if apt_name.startswith("[신축] "):
            apt_name = apt_name[6:]  # "[신축] " 제거
        
        # 진행 상황 표시 창
        progress_window = tk.Toplevel(self.top)
        progress_window.title("데이터 수집 중...")
        progress_window.geometry("300x100")
        progress_window.transient(self.top)
        progress_window.grab_set()
        
        ttk.Label(progress_window, text=f"{apt_name} 전용면적 정보를 수집 중입니다...").pack(pady=10)
        
        progress_bar = ttk.Progressbar(progress_window, orient="horizontal", length=300, mode="determinate")
        progress_bar.pack(fill="x", padx=20, pady=10)
        
        cancel_flag = [False]
        progress_window.protocol("WM_DELETE_WINDOW", lambda: setattr(cancel_flag, 0, True) or progress_window.destroy())
        
        if not hasattr(self, 'apt_area_cache'):
            self.apt_area_cache = {}
        
        if apt_name in self.apt_area_cache:
            progress_bar['value'] = 100
            progress_window.update_idletasks()
            time.sleep(0.3)
            progress_window.destroy()
            return self.apt_area_cache[apt_name]
        
        concurrent_requests = 12
        
        session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(
            pool_connections=concurrent_requests,
            pool_maxsize=concurrent_requests * 2,
            max_retries=1
        )
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        
        # 최근 3개월만 조회
        max_months = 4
        
        def collect_areas():
            nonlocal areas
            
            try:
                all_requests = []
                for month in range(max_months):
                    search_date = current_date - timedelta(days=30 * month)
                    deal_ymd = search_date.strftime("%Y%m")
                    
                    # 기축 매매 API
                    trade_url = (f"http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade"
                               f"?serviceKey={self.service_key}"
                               f"&LAWD_CD={self.sigungu_code}"
                               f"&DEAL_YMD={deal_ymd}"
                               f"&numOfRows=1000")
                    
                    # 신축(분양권) 매매 API
                    new_trade_url = (f"http://apis.data.go.kr/1613000/RTMSDataSvcSilvTrade/getRTMSDataSvcSilvTrade"
                                   f"?serviceKey={self.service_key}"
                                   f"&LAWD_CD={self.sigungu_code}"
                                   f"&DEAL_YMD={deal_ymd}"
                                   f"&numOfRows=1000")
                    
                    # 기축 전세 API
                    rent_url = (f"http://apis.data.go.kr/1613000/RTMSDataSvcAptRent/getRTMSDataSvcAptRent"
                               f"?serviceKey={self.service_key}"
                               f"&LAWD_CD={self.sigungu_code}"
                               f"&DEAL_YMD={deal_ymd}"
                               f"&numOfRows=1000")
                    
                    all_requests.append((trade_url, 'trade', month))
                    all_requests.append((new_trade_url, 'new_trade', month))
                    all_requests.append((rent_url, 'rent', month))
                
                with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent_requests) as executor:
                    future_to_request = {
                        executor.submit(session.get, url, timeout=2): (req_type, month_idx) 
                        for url, req_type, month_idx in all_requests
                    }
                    
                    total_requests = len(all_requests)
                    processed = 0
                    
                    for future in concurrent.futures.as_completed(future_to_request):
                        processed += 1
                        req_type, month_idx = future_to_request[future]
                        
                        progress = min(100, (processed / total_requests) * 100)
                        progress_bar['value'] = progress
                        progress_window.update_idletasks()
                        
                        if len(areas) >= 5:
                            continue
                            
                        if cancel_flag[0]:
                            continue
                        
                        try:
                            response = future.result()
                            
                            if response.status_code == 200:
                                try:
                                    root = ET.fromstring(response.text)
                                    
                                    for item in root.findall('.//item'):
                                        item_apt = item.findtext('aptNm', '').strip()
                                        
                                        if item_apt == apt_name:
                                            item_area = float(item.findtext('excluUseAr', '0'))
                                            if item_area > 0:
                                                # 소수점 없는 정수로 변환
                                                area_int = str(int(item_area))
                                                areas.add(area_int)
                                except ET.ParseError:
                                    pass
                        
                        except Exception as e:
                            continue
                
                progress_bar['value'] = 100
                progress_window.update_idletasks()
                
                self.apt_area_cache[apt_name] = sorted(list(areas), key=float)
                
            except Exception as e:
                logging.error(f"전용면적 정보 수집 중 오류: {str(e)}")
            
            time.sleep(0.3)
            if not cancel_flag[0]:
                progress_window.destroy()
        

        thread = threading.Thread(target=collect_areas)
        thread.daemon = True
        thread.start()
        
        self.top.wait_window(progress_window)
        
        if cancel_flag[0] or not areas:
            return []
        
        if apt_name not in self.apt_area_cache:
            self.apt_area_cache[apt_name] = sorted(list(areas), key=float)
        
        return sorted(list(areas), key=float)


def main():
    app = RealEstateMonitorApp()
    app.root.mainloop()

if __name__ == "__main__":
    main()

2025-10-25 13:42:22 [INFO] - 신고가 히스토리를 불러왔습니다. (12개)
2025-10-25 13:44:21 [INFO] - API 호출: ('11680', '202510', 'existing') (총 호출: 1)
2025-10-25 13:44:36 [WARNING] - Retrying (Retry(total=4, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPConnectionPool(host='apis.data.go.kr', port=80): Read timed out. (read timeout=15)")': /1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade?serviceKey=Vs5lXsSo6iEI8no3pP%2FT0udWF9s7Cc8oP1SIWnEI5F4h6dKq92fLvnKmxkoWGJxSeW2%2FSOLQECGxOJzWcjJEXQ%3D%3D&LAWD_CD=11680&DEAL_YMD=202510&numOfRows=1000
2025-10-25 13:44:53 [WARNING] - Retrying (Retry(total=3, connect=3, read=1, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPConnectionPool(host='apis.data.go.kr', port=80): Read timed out. (read timeout=15)")': /1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade?serviceKey=Vs5lXsSo6iEI8no3pP%2FT0udWF9s7Cc8oP1SIWnEI5F4h6dKq92fLvnKmxkoWGJxSeW2%2FSOLQECGxOJzWcjJEXQ%3D%3D&LAWD_CD=11680&DEAL_